In [ ]:
#!pip install ipywidgets matplotlib requests
#!jupyter nbextension enable --py widgetsnbextension

In [ ]:
#!pip install jupyterquiz

In [ ]:
# Виконати в окремій клітинці:
#!pip install ipywidgets matplotlib requests
#!jupyter nbextension enable --py widgetsnbextension
#!jupyter labextension install @jupyter-widgets/jupyterlab-manager  # Для JupyterLab

In [ ]:
import ipywidgets as widgets
from IPython.display import display, clear_output

In [ ]:
from IPython.display import display, HTML

display(HTML("""
<meta name="viewport" content="width=device-width, initial-scale=1">
<!-- Перемикач мови -->
<div id="lang-selector" style="position: fixed; top: 12px; right: 12px; z-index: 1000;">
    <select style="padding: 6px 12px; border-radius: 8px; background: #232a36; color: #f7faff; border: 1px solid #4f8cff;">
        <option value="uk">🇺🇦 Українська</option>
        <option value="ru">🇷🇺 Русский</option>
        <option value="en">🇬🇧 English</option>
    </select>
</div>

<meta name="viewport" content="width=device-width, initial-scale=1">
<style>
:root {
    --bg-color: #1a1c23;
    --container-bg: #24262b;
    --text-color: #f7faff;
    --primary-accent: #4f8cff;
    --hover-accent: #3daeff;
    --border-radius: 18px;
    --answer-bg: #232a36;
    --border-color: #31343a;
}

/* Прибираємо відступи зверху */
html, body {
    margin: 0 !important;
    padding: 0 !important;
}

/* Загальний фон і шрифт */
body, .jp-Notebook, .notebook_app {
    background: var(--bg-color) !important;
    color: var(--text-color) !important;
    font-family: -apple-system, BlinkMacSystemFont, "Segoe UI", Roboto, sans-serif;
}

/* Демографія: білі label, select, option */
.widget-label, #start-form-box label {
    color: #f7faff !important;
    font-weight: 600 !important;
    font-size: 1.08rem !important;
    margin-bottom: 4px !important;
}
#start-form-box select, #start-form-box option, #start-form-box input {
    color: #f7faff !important;
    background: #232a36 !important;
    border: 1.5px solid #31343a !important;
    font-size: 1.04rem !important;
}
#start-form-box select:focus {
    outline: 2px solid #4f8cff !important;
}
#start-form-box {
    max-width: 400px;
    margin: 0 auto !important;
    background: #232a36;
    border-radius: 18px;
    border: 1.5px solid #4f8cff;
    box-shadow: 0 0 15px rgba(79,140,255,0.10);
    padding: 32px 32px 24px 32px;
    text-align: center;
}

/* Контейнер питання - БЕЗ відступів зверху */
.question-container, .custom-question-container {
    background: var(--container-bg);
    border-radius: var(--border-radius);
    padding: 1.5rem 1.1rem !important;
    margin: 0 auto !important;
    max-width: 540px !important;
    box-shadow: 0 8px 32px rgba(0,0,0,0.3);
    border: 1px solid var(--border-color);
    animation: fadeIn 0.4s ease-out;
}

/* Заголовок питання - ЗАЛИШАЄМО БЕЗ ЗМІН */
.question-container h2, .custom-question-container h2 {
    color: #fff !important;
    font-size: 1.65rem !important;
    font-weight: 800 !important;
    margin-bottom: 0.7em !important;
    letter-spacing: 0.01em;
    text-align: left !important;
}

/* Текст питання - ЗАЛИШАЄМО БЕЗ ЗМІН */
.question-text, .custom-question-text {
    color: var(--text-color) !important;
    font-size: 1.22rem !important;
    font-weight: 600 !important;
    line-height: 1.6 !important;
    margin-bottom: 18px !important;
    opacity: 1 !important;
}

/* Прогрес-бар */
.progress-container {
    width: 100%;
    height: 12px;
    background: #232a36;
    border-radius: 12px;
    margin: 0 auto 20px;
    box-shadow: 0 0 10px rgba(79,140,255,0.10);
    overflow: hidden;
    border: 1.5px solid #4f8cff;
}
.progress-fill {
    height: 100%;
    background: linear-gradient(90deg, #00f0ff, #4f8cff 60%, #00aaff 100%);
    box-shadow: 0 0 15px #00f0ff;
    border-radius: 12px;
    transition: width 0.6s cubic-bezier(0.4,0,0.2,1);
}

/* КОМПАКТНІ кнопки-відповіді для десктопу */
.answer-btn {
    width: 100% !important;
    min-height: 26px !important;
    margin-bottom: 3px !important;
    padding: 4px 8px !important;
    font-size: 0.93rem !important;
    font-weight: 600 !important;
    background: #3a5ca8 !important;
    color: #e0e6ef !important;
    border-radius: 8px !important;
    border: 1.5px solid #31343a !important;
    box-shadow: 0 1px 4px rgba(79,140,255,0.07) !important;
    cursor: pointer;
    text-align: left !important;
    transition: background 0.2s;
    display: block !important;
    line-height: 1.1 !important;
    white-space: normal !important;
    overflow-wrap: anywhere !important;
    word-break: break-word !important;
    height: auto !important;
}
.answer-btn:hover {
    background: #34405e !important;
}

/* Кнопки */
.widget-button, #start-form-box button {
    background: var(--primary-accent) !important;
    color: #fff !important;
    border-radius: 8px !important;
    padding: 13px 32px !important;
    font-size: 1.13rem !important;
    font-weight: 600 !important;
    min-width: 140px !important;
    max-width: 100% !important;
    margin: 18px auto 0 auto !important;
    box-shadow: 0 2px 8px rgba(79,140,255,0.18) !important;
    border: 1.5px solid var(--primary-accent) !important;
    transition: all 0.2s ease;
    cursor: pointer !important;
}
.widget-button:hover, #start-form-box button:hover {
    background: var(--hover-accent) !important;
    box-shadow: 0 4px 16px rgba(61,174,255,0.25) !important;
    transform: translateY(-1px);
}

/* Поле для відгуку */
.widget-textarea textarea {
    background: #2a2d35 !important;
    color: var(--text-color) !important;
    border: 1px solid #3a3f48 !important;
    border-radius: 8px !important;
    padding: 12px !important;
    font-size: 1rem !important;
    min-height: 66px !important;
    max-width: 440px !important;
    resize: vertical !important;
}

/* Сітка результатів */
.score-grid {
    display: grid !important;
    grid-template-columns: repeat(auto-fit, minmax(90px, 1fr)) !important;
    gap: 14px !important;
    margin-top: 22px !important;
    width: 100% !important;
}
.score-item {
    background: #292c33 !important;
    padding: 13px !important;
    border-radius: 10px !important;
    text-align: center !important;
    border-left: 4px solid #4f8cff !important;
    box-shadow: 0 2px 12px rgba(79,140,255,0.08) !important;
    transition: transform 0.2s ease;
    color: #f7faff !important;
}
.score-item:hover {
    transform: translateY(-2px);
    box-shadow: 0 4px 16px rgba(79,140,255,0.15) !important;
}
.score-value {
    color: #00e0ff !important;
    font-size: 1.13em !important;
    font-weight: 700 !important;
    margin-top: 8px !important;
}

/* Світлий текст для результатів і демографії */
.score-item, .score-value, 
#start-form-box label, #start-form-box select, #start-form-box option, #start-form-box input,
.question-container label, .custom-question-container label,
.question-container select, .custom-question-container select,
.question-container option, .custom-question-container option,
.question-container input, .custom-question-container input,
.widget-html-content, .widget-html-content div, .widget-html-content p, .widget-html-content span {
    color: #f7faff !important;
}

/* Опис архетипу у результатах */
.desc-widget, .archetype-desc, 
.question-container .desc-widget, .custom-question-container .desc-widget,
.question-container .archetype-desc, .custom-question-container .archetype-desc {
    color: #e0e6ef !important;
    line-height: 1.6 !important;
}

/* Мобільна адаптація */
@media (max-width: 600px) {
    .question-container, .custom-question-container {
        max-width: 99vw !important;
        padding: 1.1rem 0.5rem 0.8rem 0.5rem !important;
        border-radius: 12px !important;
        margin: 0 auto !important;
    }
    .custom-question-container h2, .question-container h2 {
        font-size: 0.92rem !important;
    }
    .custom-question-text, .question-text {
        font-size: 0.89rem !important;
    }
    .answer-btn {
        font-size: 0.42rem !important;
        min-height: 11px !important;
        padding: 2px 4px !important;
        margin-bottom: 1.5px !important;
        line-height: 1 !important;
        border-radius: 6px !important;
        background: #3a5ca8 !important;
        color: #e0e6ef !important;
        border: 1.2px solid #2e4a7d !important;
        box-shadow: none !important;
        margin-left: 4% !important;      /* або 5% для центрування */
        margin-right: 4% !important;
    }
    .widget-button, #start-form-box button {
        font-size: 1.00rem !important;
        padding: 12px 16px !important;
        width: 100%;
    }
    .widget-textarea textarea {
        max-width: 100% !important;
    }
    .score-grid {
        grid-template-columns: repeat(5, 1fr) !important;  /* 5 колонок в один ряд */
        gap: 5px !important;
    }
    .score-item {
        padding: 6px !important;
        font-size: 0.7rem !important;
        min-width: 0 !important;
    }
    .score-value {
        font-size: 0.85rem !important;
    }
}

.archetype-title {
    display: block;
    text-align: center;
    color: #4f8cff !important;
    font-size: 1.25rem !important;
    font-weight: 800 !important;
    margin: 12px 0 8px 0 !important;
    letter-spacing: 0.02em;
    text-shadow: 0 2px 8px rgba(79,140,255,0.10);
}
@media (max-width: 600px) {
    .archetype-title {
        font-size: 1.05rem !important;
        margin: 8px 0 6px 0 !important;
    }
}

/* Анімація */
@keyframes fadeIn {
    from { opacity: 0; transform: translateY(12px); }
    to { opacity: 1; transform: translateY(0); }
}
</style>
"""))


In [ ]:
from collections import defaultdict
import numpy as np

# === Додаємо багатомовні словники ===
ARCHETYPES = {
    'uk': [
        "Маржинальний жнець",
        "Рептилія",
        "Арбітражний стабілізатор",
        "Провокатор хаосу",
        "Емерджентний аналітик",
        "Антифрагільний агент",
        "Трендсеттер",
        "Медіатор Духів"
    ],
    'ru': [
        "Маржинальный жнец",
        "Рептилия",
        "Арбитражный стабилизатор",
        "Провокатор хаоса",
        "Эмерджентный аналитик",
        "Антихрупкий агент",
        "Трендсеттер",
        "Медиатор Духов"
    ],
    'en': [
        "Margin Reaper",
        "Reptile",
        "Arbitrage Stabilizer",
        "Chaos Provocateur",
        "Emergent Analyst",
        "Antifragile Agent",
        "Trendsetter",
        "Spirit Mediator"
    ]
}

ARCHETYPE_DESCRIPTIONS = {
    'uk': [
        "Агресія, імпульсивність, ігнорування ризиків",
        "Системність, ригідність, дотримання правил",
        "Раціональність, пошук дисбалансів, арбітраж",
        "Любить волатильність, провокує ринок",
        "Адаптивний, технологічний, рефлексивний",
        "Виграє від стресу, асиметрична адаптація",
        "Впливає на думки, формує наративи",
        "Висока емпатія, інтуїція, емоційна мудрість"
    ],
    'ru': [
        "Агрессия, импульсивность, игнорирование рисков",
        "Системность, ригидность, следование правилам",
        "Рациональность, поиск дисбалансов, арбитраж",
        "Любит волатильность, провоцирует рынок",
        "Адаптивный, технологичный, рефлексивный",
        "Выигрывает от стресса, асимметричная адаптация",
        "Влияет на мнения, формирует нарративы",
        "Высокая эмпатия, интуиция, эмоциональная мудрость"
    ],
    'en': [
        "Aggression, impulsivity, risk ignorance",
        "Systematic, rigid, rule-following",
        "Rational, seeks imbalances, arbitrage",
        "Loves volatility, provokes the market",
        "Adaptive, tech-savvy, reflexive",
        "Benefits from stress, asymmetric adaptation",
        "Influences opinions, shapes narratives",
        "High empathy, intuition, emotional wisdom"
    ]
}

def calculate_scores(user_answers, questions):
    """
    Підраховує бали по категоріях на основі вибраних відповідей.
    """
    scores = defaultdict(int)
    for q_idx, answer_idx in enumerate(user_answers):
        if q_idx >= len(questions) or answer_idx >= len(questions[q_idx]["answers"]):
            continue
        weights = questions[q_idx]["answers"][answer_idx].get("weights", {})
        for cat, points in weights.items():
            scores[cat] += points
    return dict(scores)

def determine_archetype(scores, questions, lang='uk'):
    """
    Визначає архетип на основі балів по категоріях, повертає назву і короткий опис відповідною мовою.
    """
    max_scores = defaultdict(int)
    for q in questions:
        for cat in ["EI", "KB", "RM", "AD", "AS"]:
            total = sum(a["weights"].get(cat, 0) for a in q["answers"])
            max_scores[cat] += total

    normalized = {
        cat: (scores.get(cat, 0) / max_scores[cat]) * 100 if max_scores[cat] != 0 else 0
        for cat in ["EI", "KB", "RM", "AD", "AS"]
    }

    archetype_matrix = np.array([
        [-2.0, -2.0, -2.0, 2.0, 2.0],   # 0
        [1.0, 1.0, 3.0, 2.0, 0.0],      # 1
        [1.0, 2.0, 3.0, 3.0, 0.0],      # 2
        [-2.0, -2.0, 0.0, 3.0, 2.0],    # 3
        [2.0, 2.0, 2.0, 3.0, 0.0],      # 4
        [2.0, 0.0, 2.0, 2.0, 1.0],      # 5
        [2.0, 1.0, 0.0, 2.0, 3.0],      # 6
        [3.0, -2.0, 0.0, 2.0, 2.0]      # 7
    ])

    user_vector = np.array([
        normalized["EI"],
        normalized["KB"],
        normalized["RM"],
        normalized["AD"],
        normalized["AS"]
    ])

    similarities = archetype_matrix @ user_vector.T
    max_similarity = np.max(similarities)
    candidates = np.where(similarities == max_similarity)[0]
    if len(candidates) == 0:
        return "Не визначено", ""
    best_idx = np.random.choice(candidates) if len(candidates) > 1 else candidates[0]

    return ARCHETYPES[lang][best_idx], ARCHETYPE_DESCRIPTIONS[lang][best_idx]

def get_category_percentages(scores, questions):
    """
    Конвертує абсолютні бали у відсотки для візуалізації.
    """
    max_scores = defaultdict(int)
    for q in questions:
        for cat in ["EI", "KB", "RM", "AD", "AS"]:
            total = sum(a["weights"].get(cat, 0) for a in q["answers"])
            max_scores[cat] += total

    return {
        cat: (scores.get(cat, 0) / max_scores[cat]) * 100 if max_scores[cat] > 0 else 0
        for cat in ["EI", "KB", "RM", "AD", "AS"]
    }


In [ ]:
questions = [
    {
        "question": {
            "uk": "Ваш портфель втратив 40% вартості за тиждень через макроекономічний шок. Ваші перші дії?",
            "ru": "Ваш портфель потерял 40% стоимости за неделю из-за макроэкономического шока. Ваши первые действия?",
            "en": "Your portfolio lost 40% of its value in a week due to a macroeconomic shock. Your first actions?"
        },
        "answers": [
            {
                "text": {
                    "uk": "Продаю активніше на ключових рівнях, щоб активувати каскад ліквідацій інших трейдерів.",
                    "ru": "Продаю активнее на ключевых уровнях, чтобы активировать каскад ликвидаций других трейдеров.",
                    "en": "Sell more aggressively at key levels to trigger a cascade of liquidations from other traders."
                },
                "weights": {"EI": 0, "KB": 0, "RM": 0, "AD": 0, "AS": 0},
                "archetype": {
                    "uk": "Маржинальний жнець",
                    "ru": "Маржинальный жнец",
                    "en": "Margin Reaper"
                },
                "explanation": {
                    "uk": "Агресія, loss aversion.",
                    "ru": "Агрессия, loss aversion.",
                    "en": "Aggression, loss aversion."
                }
            },
            {
                "text": {
                    "uk": "Виконую заздалегідь прописані правила (напр., стоп-лоси) без додаткового аналізу.",
                    "ru": "Выполняю заранее прописанные правила (напр., стоп-лоссы) без дополнительного анализа.",
                    "en": "Follow pre-defined rules (e.g., stop-losses) without additional analysis."
                },
                "weights": {"EI": 3, "KB": 0, "RM": 3, "AD": 0, "AS": 0},
                "archetype": {
                    "uk": "Рептилія",
                    "ru": "Рептилия",
                    "en": "Reptile"
                },
                "explanation": {
                    "uk": "Емоційний контроль, планування.",
                    "ru": "Эмоциональный контроль, планирование.",
                    "en": "Emotional control, planning."
                }
            },
            {
                "text": {
                    "uk": "Шукаю цінові розбіжності між біржами для арбітражних угод.",
                    "ru": "Ищу ценовые различия между биржами для арбитражных сделок.",
                    "en": "Look for price differences between exchanges for arbitrage deals."
                },
                "weights": {"EI": 0, "KB": 0, "RM": 2, "AD": 2, "AS": 0},
                "archetype": {
                    "uk": "Арбітражний стабілізатор",
                    "ru": "Арбитражный стабилизатор",
                    "en": "Arbitrage Stabilizer"
                },
                "explanation": {
                    "uk": "Раціональний підхід, але без повної адаптації.",
                    "ru": "Рациональный подход, но без полной адаптации.",
                    "en": "Rational approach, but not fully adaptive."
                }
            },
            {
                "text": {
                    "uk": "Докуповую активи з найвищою волатильністю, щоб посилити ринковий хаос.",
                    "ru": "Докупаю активы с самой высокой волатильностью, чтобы усилить рыночный хаос.",
                    "en": "Buy more of the most volatile assets to intensify market chaos."
                },
                "weights": {"EI": 0, "KB": 0, "RM": 0, "AD": 0, "AS": 0},
                "archetype": {
                    "uk": "Провокатор хаосу",
                    "ru": "Провокатор хаоса",
                    "en": "Chaos Provocateur"
                },
                "explanation": {
                    "uk": "Дуже ризиковано, може погіршити ситуацію.",
                    "ru": "Очень рискованно, может ухудшить ситуацию.",
                    "en": "Very risky, can worsen the situation."
                }
            },
            {
                "text": {
                    "uk": "Аналізую, які сценарії не врахувала моя модель, і перебудовую стратегію.",
                    "ru": "Анализирую, какие сценарии не учла моя модель, и перестраиваю стратегию.",
                    "en": "Analyze which scenarios my model missed and rebuild the strategy."
                },
                "weights": {"EI": 0, "KB": 0, "RM": 0, "AD": 3, "AS": 0},
                "archetype": {
                    "uk": "Емерджентний аналітик",
                    "ru": "Эмерджентный аналитик",
                    "en": "Emergent Analyst"
                },
                "explanation": {
                    "uk": "Адаптивність і аналітичність.",
                    "ru": "Адаптивность и аналитичность.",
                    "en": "Adaptivity and analytical thinking."
                }
            },
            {
                "text": {
                    "uk": "Перерозподіляю капітал у активи, що виграють від кризи (золото, стейблкоїни).",
                    "ru": "Перераспределяю капитал в активы, которые выигрывают от кризиса (золото, стейблкоины).",
                    "en": "Reallocate capital to assets that benefit from the crisis (gold, stablecoins)."
                },
                "weights": {"EI": 0, "KB": 0, "RM": 3, "AD": 0, "AS": 0},
                "archetype": {
                    "uk": "Антифрагільний агент",
                    "ru": "Антихрупкий агент",
                    "en": "Antifragile Agent"
                },
                "explanation": {
                    "uk": "Збалансований підхід до ризику.",
                    "ru": "Сбалансированный подход к риску.",
                    "en": "Balanced approach to risk."
                }
            },
            {
                "text": {
                    "uk": "Готую серію постів/відео з прогнозом швидкого відновлення ринку.",
                    "ru": "Готовлю серию постов/видео с прогнозом быстрого восстановления рынка.",
                    "en": "Prepare a series of posts/videos predicting a quick market recovery."
                },
                "weights": {"EI": 0, "KB": 0, "RM": 0, "AD": 0, "AS": 0},
                "archetype": {
                    "uk": "Трендсеттер",
                    "ru": "Трендсеттер",
                    "en": "Trendsetter"
                },
                "explanation": {
                    "uk": "Не впливає на реальний ризик.",
                    "ru": "Не влияет на реальный риск.",
                    "en": "Does not affect real risk."
                }
            },
            {
                "text": {
                    "uk": "Роблю перерву, щоб 'відчути' ринок, перш ніж діяти.",
                    "ru": "Делаю паузу, чтобы 'почувствовать' рынок, прежде чем действовать.",
                    "en": "Take a break to 'feel' the market before acting."
                },
                "weights": {"EI": 2, "KB": 0, "RM": 0, "AD": 0, "AS": 0},
                "archetype": {
                    "uk": "Медіатор Духів",
                    "ru": "Медиатор Духов",
                    "en": "Spirit Mediator"
                },
                "explanation": {
                    "uk": "Інтуїтивний контроль, але важливо не затягувати з рішенням.",
                    "ru": "Интуитивный контроль, но важно не затягивать с решением.",
                    "en": "Intuitive control, but important not to delay the decision."
                }
            }
        ]
    },
     {
        "question": {
            "uk": "Ваш партнер по команді пропонує “сіру” схему для швидкого збагачення, але з високим ризиком регуляторних санкцій. Ви?",
            "ru": "Ваш партнер по команде предлагает “серую” схему для быстрого обогащения, но с высоким риском регуляторных санкций. Вы?",
            "en": "Your team partner proposes a “gray” scheme for quick enrichment but with a high risk of regulatory sanctions. You?"
        },
        "answers": [
            {
                "text": {
                    "uk": "Розробляю план уникнення відстеження операцій.",
                    "ru": "Разрабатываю план избежания отслеживания операций.",
                    "en": "Develop a plan to avoid tracking operations."
                },
                "weights": {"EI": 0, "KB": 0, "RM": 0, "AD": 0, "AS": 0},
                "archetype": {
                    "uk": "Маржинальний жнець",
                    "ru": "Маржинальный жнец",
                    "en": "Margin Reaper"
                },
                "explanation": {
                    "uk": "Агресія + анонімність.",
                    "ru": "Агрессия + анонимность.",
                    "en": "Aggression + anonymity."
                }
            },
            {
                "text": {
                    "uk": "Відмовляюсь і вимагаю припинити співпрацю.",
                    "ru": "Отказываюсь и требую прекратить сотрудничество.",
                    "en": "Refuse and demand to terminate cooperation."
                },
                "weights": {"EI": 0, "KB": 0, "RM": 3, "AD": 0, "AS": 3},
                "archetype": {
                    "uk": "Рептилія",
                    "ru": "Рептилия",
                    "en": "Reptile"
                },
                "explanation": {
                    "uk": "Ригідна відповідність правилам.",
                    "ru": "Жесткое соответствие правилам.",
                    "en": "Strict compliance with rules."
                }
            },
            {
                "text": {
                    "uk": "Досліджую юридичні межі для мінімізації наслідків.",
                    "ru": "Исследую юридические границы для минимизации последствий.",
                    "en": "Research legal boundaries to minimize consequences."
                },
                "weights": {"EI": 0, "KB": 0, "RM": 2, "AD": 2, "AS": 0},
                "archetype": {
                    "uk": "Арбітражний стабілізатор",
                    "ru": "Арбитражный стабилизатор",
                    "en": "Arbitrage Stabilizer"
                },
                "explanation": {
                    "uk": "Аналіз лазівок.",
                    "ru": "Анализ лазеек.",
                    "en": "Loophole analysis."
                }
            },
            {
                "text": {
                    "uk": "Пропоную ескалацію ризиків через підставні угоди.",
                    "ru": "Предлагаю эскалацию рисков через подставные сделки.",
                    "en": "Propose risk escalation through sham transactions."
                },
                "weights": {"EI": 0, "KB": 0, "RM": 0, "AD": 3, "AS": 0},
                "archetype": {
                    "uk": "Провокатор хаосу",
                    "ru": "Провокатор хаоса",
                    "en": "Chaos Provocateur"
                },
                "explanation": {
                    "uk": "Навмисне створення хаосу.",
                    "ru": "Преднамеренное создание хаоса.",
                    "en": "Intentional chaos creation."
                }
            },
            {
                "text": {
                    "uk": "Моделюю наслідки для репутації та фінансів команди.",
                    "ru": "Моделирую последствия для репутации и финансов команды.",
                    "en": "Model consequences for team reputation and finances."
                },
                "weights": {"EI": 0, "KB": 0, "RM": 2, "AD": 3, "AS": 0},
                "archetype": {
                    "uk": "Емерджентний аналітик",
                    "ru": "Эмерджентный аналитик",
                    "en": "Emergent Analyst"
                },
                "explanation": {
                    "uk": "Премортем-аналіз.",
                    "ru": "Премортем-анализ.",
                    "en": "Pre-mortem analysis."
                }
            },
            {
                "text": {
                    "uk": "Виношу рішення на команду, де кожен голосує.",
                    "ru": "Выношу решение на команду, где каждый голосует.",
                    "en": "Bring the decision to the team for voting."
                },
                "weights": {"EI": 0, "KB": 0, "RM": 1, "AD": 2, "AS": 2},
                "archetype": {
                    "uk": "Антифрагільний агент",
                    "ru": "Антихрупкий агент",
                    "en": "Antifragile Agent"
                },
                "explanation": {
                    "uk": "Колективна відповідальність.",
                    "ru": "Коллективная ответственность.",
                    "en": "Collective responsibility."
                }
            },
            {
                "text": {
                    "uk": "Анонімно передаю інформацію про схему конкурентам.",
                    "ru": "Анонимно передаю информацию о схеме конкурентам.",
                    "en": "Anonymously leak information about the scheme to competitors."
                },
                "weights": {"EI": 0, "KB": 0, "RM": 0, "AD": 0, "AS": 3},
                "archetype": {
                    "uk": "Трендсеттер",
                    "ru": "Трендсеттер",
                    "en": "Trendsetter"
                },
                "explanation": {
                    "uk": "Маніпуляція через інфлюенс.",
                    "ru": "Манипуляция через влияние.",
                    "en": "Influence manipulation."
                }
            },
            {
                "text": {
                    "uk": "Відкладаю рішення, спостерігаючи за реакцією ринку.",
                    "ru": "Откладываю решение, наблюдая за реакцией рынка.",
                    "en": "Delay decision while observing market reaction."
                },
                "weights": {"EI": 2, "KB": 0, "RM": 0, "AD": 1, "AS": 0},
                "archetype": {
                    "uk": "Медіатор Духів",
                    "ru": "Медиатор Духов",
                    "en": "Spirit Mediator"
                },
                "explanation": {
                    "uk": "Інтуїтивне відстрочення.",
                    "ru": "Интуитивная отсрочка.",
                    "en": "Intuitive deferral."
                }
            }
        ]
    },

        {
        "question": {
            "uk": "BTC падає з $95k до $78k за 20 хвилин. Ваш портфель -30%. Серце калатає, руки трусяться. Ваші дії?",
            "ru": "BTC падает с $95k до $78k за 20 минут. Ваш портфель -30%. Сердце колотится, руки трясутся. Ваши действия?",
            "en": "BTC drops from $95k to $78k in 20 minutes. Your portfolio is down 30%. Heart racing, hands shaking. Your actions?"
        },
        "answers": [
            {
                "text": {
                    "uk": "Відкриваю шорт, щоб прискорити падіння та активувати ліквідації.",
                    "ru": "Открываю шорт, чтобы ускорить падение и активировать ликвидации.",
                    "en": "Open a short to accelerate the drop and trigger liquidations."
                },
                "weights": {"EI": 0, "KB": 0, "RM": 0, "AD": 0, "AS": 0},
                "archetype": {
                    "uk": "Маржинальний жнець",
                    "ru": "Маржинальный жнец",
                    "en": "Margin Reaper"
                },
                "explanation": {
                    "uk": "Агресивний вплив на ринок.",
                    "ru": "Агрессивное влияние на рынок.",
                    "en": "Aggressive market influence."
                }
            },
            {
                "text": {
                    "uk": "Чекаю, дотримуючись заздалегідь встановлених правил.",
                    "ru": "Жду, соблюдая заранее установленные правила.",
                    "en": "Wait while following pre-established rules."
                },
                "weights": {"EI": 0, "KB": 0, "RM": 2, "AD": 0, "AS": 0},
                "archetype": {
                    "uk": "Рептилія",
                    "ru": "Рептилия",
                    "en": "Reptile"
                },
                "explanation": {
                    "uk": "Ригідна системність.",
                    "ru": "Жесткая системность.",
                    "en": "Rigid systematic approach."
                }
            },
            {
                "text": {
                    "uk": "Купую на біржах з відстаючими цінами для арбітражу.",
                    "ru": "Покупаю на биржах с отстающими ценами для арбитража.",
                    "en": "Buy on exchanges with lagging prices for arbitrage."
                },
                "weights": {"EI": 0, "KB": 0, "RM": 3, "AD": 3, "AS": 0},
                "archetype": {
                    "uk": "Арбітражний стабілізатор",
                    "ru": "Арбитражный стабилизатор",
                    "en": "Arbitrage Stabilizer"
                },
                "explanation": {
                    "uk": "Експлуатація цінових розбіжностей.",
                    "ru": "Эксплуатация ценовых расхождений.",
                    "en": "Exploiting price discrepancies."
                }
            },
            {
                "text": {
                    "uk": "Провокую паніку, розміщуючи великі ордери на продаж.",
                    "ru": "Провоцирую панику, размещая крупные ордера на продажу.",
                    "en": "Provoke panic by placing large sell orders."
                },
                "weights": {"EI": 0, "KB": 0, "RM": 0, "AD": 3, "AS": 0},
                "archetype": {
                    "uk": "Провокатор хаосу",
                    "ru": "Провокатор хаоса",
                    "en": "Chaos Provocateur"
                },
                "explanation": {
                    "uk": "Навмисне створення волатильності.",
                    "ru": "Преднамеренное создание волатильности.",
                    "en": "Intentional volatility creation."
                }
            },
            {
                "text": {
                    "uk": "Аналізую дані за допомогою AI для перевірки гіпотез.",
                    "ru": "Анализирую данные с помощью AI для проверки гипотез.",
                    "en": "Analyze data using AI to verify hypotheses."
                },
                "weights": {"EI": 0, "KB": 0, "RM": 0, "AD": 3, "AS": 0},
                "archetype": {
                    "uk": "Емерджентний аналітик",
                    "ru": "Эмерджентный аналитик",
                    "en": "Emergent Analyst"
                },
                "explanation": {
                    "uk": "Технологічний підхід.",
                    "ru": "Технологический подход.",
                    "en": "Technological approach."
                }
            },
            {
                "text": {
                    "uk": "Збільшую позиції, використовуючи падіння як можливість.",
                    "ru": "Увеличиваю позиции, используя падение как возможность.",
                    "en": "Increase positions using the drop as an opportunity."
                },
                "weights": {"EI": 0, "KB": 0, "RM": 3, "AD": 3, "AS": 0},
                "archetype": {
                    "uk": "Антифрагільний агент",
                    "ru": "Антихрупкий агент",
                    "en": "Antifragile Agent"
                },
                "explanation": {
                    "uk": "Асиметрична ставка.",
                    "ru": "Асимметричная ставка.",
                    "en": "Asymmetric bet."
                }
            },
            {
                "text": {
                    "uk": "Публікую оптимістичні прогнози, щоб змінити ринковий наратив.",
                    "ru": "Публикую оптимистичные прогнозы, чтобы изменить рыночный нарратив.",
                    "en": "Publish optimistic forecasts to change market narrative."
                },
                "weights": {"EI": 0, "KB": 0, "RM": 0, "AD": 0, "AS": 3},
                "archetype": {
                    "uk": "Трендсеттер",
                    "ru": "Трендсеттер",
                    "en": "Trendsetter"
                },
                "explanation": {
                    "uk": "Маніпуляція інформаційним полем.",
                    "ru": "Манипуляция информационным полем.",
                    "en": "Information field manipulation."
                }
            },
            {
                "text": {
                    "uk": "Роблю паузу, щоб відновити емоційний баланс перед діями.",
                    "ru": "Делаю паузу, чтобы восстановить эмоциональный баланс перед действиями.",
                    "en": "Pause to regain emotional balance before acting."
                },
                "weights": {"EI": 2, "KB": 0, "RM": 0, "AD": 0, "AS": 0},
                "archetype": {
                    "uk": "Медіатор Духів",
                    "ru": "Медиатор Духов",
                    "en": "Spirit Mediator"
                },
                "explanation": {
                    "uk": "Інтуїтивний контроль.",
                    "ru": "Интуитивный контроль.",
                    "en": "Intuitive control."
                }
            }
        ]
    },

        {
        "question": {
            "uk": "З'являється принципово нова технологія торгівлі (квантові алгоритми), яка дає +50% precision. Але коштує $50k і потребує 6 місяців навчання. Ваша реакція:",
            "ru": "Появляется принципиально новая торговая технология (квантовые алгоритмы), которая дает +50% точности. Но стоит $50k и требует 6 месяцев обучения. Ваша реакция:",
            "en": "A fundamentally new trading technology (quantum algorithms) appears, offering +50% precision. But it costs $50k and requires 6 months of training. Your reaction:"
        },
        "answers": [
            {
                "text": {
                    "uk": "Інвестую всі доступні кошти, щоб першим отримати перевагу.",
                    "ru": "Инвестирую все доступные средства, чтобы первым получить преимущество.",
                    "en": "Invest all available funds to be the first to gain an advantage."
                },
                "weights": {"EI": 0, "KB": 0, "RM": 0, "AD": 0, "AS": 0},
                "archetype": {
                    "uk": "Маржинальний жнець",
                    "ru": "Маржинальный жнец",
                    "en": "Margin Reaper"
                },
                "explanation": {
                    "uk": "Агресивне захоплення інновацій.",
                    "ru": "Агрессивное захват инноваций.",
                    "en": "Aggressive innovation capture."
                }
            },
            {
                "text": {
                    "uk": "Чекаю, поки технологія пройде тестування на ринку.",
                    "ru": "Жду, пока технология пройдет тестирование на рынке.",
                    "en": "Wait until the technology passes market testing."
                },
                "weights": {"EI": 0, "KB": 0, "RM": 2, "AD": 0, "AS": 0},
                "archetype": {
                    "uk": "Рептилія",
                    "ru": "Рептилия",
                    "en": "Reptile"
                },
                "explanation": {
                    "uk": "Консервативний підхід.",
                    "ru": "Консервативный подход.",
                    "en": "Conservative approach."
                }
            },
            {
                "text": {
                    "uk": "Створюю консорціум з іншими трейдерами для спільного фінансування.",
                    "ru": "Создаю консорциум с другими трейдерами для совместного финансирования.",
                    "en": "Create a consortium with other traders for joint financing."
                },
                "weights": {"EI": 0, "KB": 0, "RM": 2, "AD": 2, "AS": 1},
                "archetype": {
                    "uk": "Арбітражний стабілізатор",
                    "ru": "Арбитражный стабилизатор",
                    "en": "Arbitrage Stabilizer"
                },
                "explanation": {
                    "uk": "Колективна оптимізація.",
                    "ru": "Коллективная оптимизация.",
                    "en": "Collective optimization."
                }
            },
            {
                "text": {
                    "uk": "Експериментую з дешевшими аналогами, щоб спровокувати хаус у ринкових паттернах.",
                    "ru": "Экспериментирую с дешевыми аналогами, чтобы спровоцировать хаос в рыночных паттернах.",
                    "en": "Experiment with cheaper analogs to provoke chaos in market patterns."
                },
                "weights": {"EI": 0, "KB": 0, "RM": 0, "AD": 3, "AS": 0},
                "archetype": {
                    "uk": "Провокатор хаосу",
                    "ru": "Провокатор хаоса",
                    "en": "Chaos Provocateur"
                },
                "explanation": {
                    "uk": "Дестабілізація через імітацію.",
                    "ru": "Дестабилизация через имитацию.",
                    "en": "Destabilization through imitation."
                }
            },
            {
                "text": {
                    "uk": "Розробляю власний алгоритм на основі відкритих джерел.",
                    "ru": "Разрабатываю собственный алгоритм на основе открытых источников.",
                    "en": "Develop my own algorithm based on open sources."
                },
                "weights": {"EI": 0, "KB": 0, "RM": 3, "AD": 3, "AS": 0},
                "archetype": {
                    "uk": "Емерджентний аналітик",
                    "ru": "Эмерджентный аналитик",
                    "en": "Emergent Analyst"
                },
                "explanation": {
                    "uk": "Технологічна самостійність.",
                    "ru": "Технологическая самостоятельность.",
                    "en": "Technological independence."
                }
            },
            {
                "text": {
                    "uk": "Використовую паніку конкурентів для арбітражу на їхніх помилках.",
                    "ru": "Использую панику конкурентов для арбитража на их ошибках.",
                    "en": "Use competitors' panic for arbitrage on their mistakes."
                },
                "weights": {"EI": 0, "KB": 0, "RM": 3, "AD": 3, "AS": 0},
                "archetype": {
                    "uk": "Антифрагільний агент",
                    "ru": "Антихрупкий агент",
                    "en": "Antifragile Agent"
                },
                "explanation": {
                    "uk": "Асиметрична адаптація.",
                    "ru": "Асимметричная адаптация.",
                    "en": "Asymmetric adaptation."
                }
            },
            {
                "text": {
                    "uk": "Формую спільноту ранніх користувачів для масштабування впливу.",
                    "ru": "Формирую сообщество ранних пользователей для масштабирования влияния.",
                    "en": "Form a community of early users to scale influence."
                },
                "weights": {"EI": 0, "KB": 0, "RM": 0, "AD": 0, "AS": 3},
                "archetype": {
                    "uk": "Трендсеттер",
                    "ru": "Трендсеттер",
                    "en": "Trendsetter"
                },
                "explanation": {
                    "uk": "Створення інформаційного поля.",
                    "ru": "Создание информационного поля.",
                    "en": "Creating an information field."
                }
            },
            {
                "text": {
                    "uk": "Аналізую власну готовність через медитацію/психотехніки.",
                    "ru": "Анализирую собственную готовность через медитацию/психотехники.",
                    "en": "Analyze my readiness through meditation/psychotechnics."
                },
                "weights": {"EI": 2, "KB": 0, "RM": 0, "AD": 0, "AS": 0},
                "archetype": {
                    "uk": "Медіатор Духів",
                    "ru": "Медиатор Духов",
                    "en": "Spirit Mediator"
                },
                "explanation": {
                    "uk": "Інтуїтивна оцінка ризиків.",
                    "ru": "Интуитивная оценка рисков.",
                    "en": "Intuitive risk assessment."
                }
            }
        ]
    },

        {
        "question": {
            "uk": "У чаті новачок плаче, що втратив останні $500. Ваша перша внутрішня реакція:",
            "ru": "В чате новичок плачет, что потерял последние $500. Ваша первая внутренняя реакция:",
            "en": "In the chat, a newcomer is crying about losing their last $500. Your first internal reaction:"
        },
        "answers": [
            {
                "text": {
                    "uk": "Думаю, що це його помилка через відсутність контролю ризиків.",
                    "ru": "Думаю, что это его ошибка из-за отсутствия контроля рисков.",
                    "en": "Think it's their fault due to lack of risk control."
                },
                "weights": {"EI": 0, "KB": 0, "RM": 0, "AD": 0, "AS": 0},
                "archetype": {
                    "uk": "Маржинальний жнець",
                    "ru": "Маржинальный жнец",
                    "en": "Margin Reaper"
                },
                "explanation": {
                    "uk": "Жорстка раціональність, низька емпатія.",
                    "ru": "Жесткая рациональность, низкая эмпатия.",
                    "en": "Harsh rationality, low empathy."
                }
            },
            {
                "text": {
                    "uk": "Відчуваю співчуття, згадуючи власні втрати на старті.",
                    "ru": "Чувствую сочувствие, вспоминая собственные потери в начале.",
                    "en": "Feel sympathy, recalling my own losses when starting out."
                },
                "weights": {"EI": 3, "KB": 0, "RM": 0, "AD": 0, "AS": 0},
                "archetype": {
                    "uk": "Рептилія",
                    "ru": "Рептилия",
                    "en": "Reptile"
                },
                "explanation": {
                    "uk": "Емоційна пам’ять, базова емпатія.",
                    "ru": "Эмоциональная память, базовая эмпатия.",
                    "en": "Emotional memory, basic empathy."
                }
            },
            {
                "text": {
                    "uk": "Аналізую його дії, щоб не повторити таких помилок у майбутньому.",
                    "ru": "Анализирую его действия, чтобы не повторить таких ошибок в будущем.",
                    "en": "Analyze their actions to avoid repeating such mistakes."
                },
                "weights": {"EI": 0, "KB": 2, "RM": 0, "AD": 2, "AS": 0},
                "archetype": {
                    "uk": "Арбітражний стабілізатор",
                    "ru": "Арбитражный стабилизатор",
                    "en": "Arbitrage Stabilizer"
                },
                "explanation": {
                    "uk": "Навчання на чужих кейсах.",
                    "ru": "Обучение на чужих кейсах.",
                    "en": "Learning from others' cases."
                }
            },
            {
                "text": {
                    "uk": "Використовую ситуацію, щоб запропонувати йому платний продукт чи консультацію.",
                    "ru": "Использую ситуацию, чтобы предложить ему платный продукт или консультацию.",
                    "en": "Use the situation to offer them a paid product or consultation."
                },
                "weights": {"EI": 0, "KB": 0, "RM": 0, "AD": 0, "AS": 0},
                "archetype": {
                    "uk": "Провокатор хаосу",
                    "ru": "Провокатор хаоса",
                    "en": "Chaos Provocateur"
                },
                "explanation": {
                    "uk": "Маніпулятивний підхід.",
                    "ru": "Манипулятивный подход.",
                    "en": "Manipulative approach."
                }
            },
            {
                "text": {
                    "uk": "Шукаю способи допомогти йому конкретними порадами чи ресурсами.",
                    "ru": "Ищу способы помочь ему конкретными советами или ресурсами.",
                    "en": "Look for ways to help them with concrete advice or resources."
                },
                "weights": {"EI": 0, "KB": 0, "RM": 0, "AD": 3, "AS": 1},
                "archetype": {
                    "uk": "Емерджентний аналітик",
                    "ru": "Эмерджентный аналитик",
                    "en": "Emergent Analyst"
                },
                "explanation": {
                    "uk": "Конструктивна підтримка.",
                    "ru": "Конструктивная поддержка.",
                    "en": "Constructive support."
                }
            },
            {
                "text": {
                    "uk": "Ігнорую ситуацію, вважаючи, що кожен сам несе відповідальність.",
                    "ru": "Игнорирую ситуацию, считая, что каждый сам несет ответственность.",
                    "en": "Ignore the situation, believing everyone is responsible for themselves."
                },
                "weights": {"EI": 0, "KB": 0, "RM": 0, "AD": 0, "AS": 0},
                "archetype": {
                    "uk": "Антифрагільний агент",
                    "ru": "Антихрупкий агент",
                    "en": "Antifragile Agent"
                },
                "explanation": {
                    "uk": "Індивідуалізм, низька залученість.",
                    "ru": "Индивидуализм, низкая вовлеченность.",
                    "en": "Individualism, low engagement."
                }
            },
            {
                "text": {
                    "uk": "Відчуваю провину за свої успіхи на тлі його втрати.",
                    "ru": "Чувствую вину за свои успехи на фоне его потери.",
                    "en": "Feel guilty about my successes against their loss."
                },
                "weights": {"EI": 2, "KB": 0, "RM": 0, "AD": 0, "AS": 1},
                "archetype": {
                    "uk": "Трендсеттер",
                    "ru": "Трендсеттер",
                    "en": "Trendsetter"
                },
                "explanation": {
                    "uk": "Соціальна рефлексія, підвищена емпатія.",
                    "ru": "Социальная рефлексия, повышенная эмпатия.",
                    "en": "Social reflection, heightened empathy."
                }
            },
            {
                "text": {
                    "uk": "Відчуваю емоційний біль на фізичному рівні, як ніби це сталося зі мною.",
                    "ru": "Чувствую эмоциональную боль на физическом уровне, как будто это случилось со мной.",
                    "en": "Feel emotional pain physically, as if it happened to me."
                },
                "weights": {"EI": 3, "KB": 0, "RM": 0, "AD": 0, "AS": 0},
                "archetype": {
                    "uk": "Медіатор Духів",
                    "ru": "Медиатор Духов",
                    "en": "Spirit Mediator"
                },
                "explanation": {
                    "uk": "Гіперемпатія, тілесна реакція.",
                    "ru": "Гиперэмпатия, телесная реакция.",
                    "en": "Hyper-empathy, physical reaction."
                }
            }
        ]
    },

        {
        "question": {
            "uk": "Під час відеодзвінка ви помічаете, що один колега виглядає напружено, але нічого не каже. Ви:",
            "ru": "Во время видеозвонка вы замечаете, что один коллега выглядит напряженно, но ничего не говорит. Вы:",
            "en": "During a video call, you notice a colleague looks tense but says nothing. You:"
        },
        "answers": [
            {
                "text": {
                    "uk": "Ігнорую, зосереджуючись на власних задачах.",
                    "ru": "Игнорирую, сосредотачиваясь на собственных задачах.",
                    "en": "Ignore it, focusing on my own tasks."
                },
                "weights": {"EI": 0, "KB": 0, "RM": 0, "AD": 0, "AS": 0},
                "archetype": {
                    "uk": "Маржинальний жнець",
                    "ru": "Маржинальный жнец",
                    "en": "Margin Reaper"
                },
                "explanation": {
                    "uk": "Фокус на результатах, низька емпатія.",
                    "ru": "Фокус на результатах, низкая эмпатия.",
                    "en": "Focus on results, low empathy."
                }
            },
            {
                "text": {
                    "uk": "Чітко запитую: 'Чи потрібна допомога?'",
                    "ru": "Четко спрашиваю: 'Нужна помощь?'",
                    "en": "Ask directly: 'Do you need help?'"
                },
                "weights": {"EI": 0, "KB": 0, "RM": 0, "AD": 0, "AS": 3},
                "archetype": {
                    "uk": "Рептилія",
                    "ru": "Рептилия",
                    "en": "Reptile"
                },
                "explanation": {
                    "uk": "Прямолінійність, дотримання протоколу.",
                    "ru": "Прямолинейность, соблюдение протокола.",
                    "en": "Directness, protocol adherence."
                }
            },
            {
                "text": {
                    "uk": "Аналізую його останні угоди, щоб знайти причину напруги.",
                    "ru": "Анализирую его последние сделки, чтобы найти причину напряжения.",
                    "en": "Analyze their recent trades to find the source of tension."
                },
                "weights": {"EI": 0, "KB": 0, "RM": 2, "AD": 2, "AS": 0},
                "archetype": {
                    "uk": "Арбітражний стабілізатор",
                    "ru": "Арбитражный стабилизатор",
                    "en": "Arbitrage Stabilizer"
                },
                "explanation": {
                    "uk": "Раціональний аналіз.",
                    "ru": "Рациональный анализ.",
                    "en": "Rational analysis."
                }
            },
            {
                "text": {
                    "uk": "Навмисно створюю конфліктну ситуацію, щоб викликати його реакцію.",
                    "ru": "Намеренно создаю конфликтную ситуацию, чтобы вызвать его реакцию.",
                    "en": "Intentionally create conflict to provoke a reaction."
                },
                "weights": {"EI": 0, "KB": 0, "RM": 0, "AD": 3, "AS": 0},
                "archetype": {
                    "uk": "Провокатор хаосу",
                    "ru": "Провокатор хаоса",
                    "en": "Chaos Provocateur"
                },
                "explanation": {
                    "uk": "Дестабілізація для отримання інформації.",
                    "ru": "Дестабилизация для получения информации.",
                    "en": "Destabilization to gather information."
                }
            },
            {
                "text": {
                    "uk": "Пропоную технічні інструменти для моніторингу стресу (напр., біодатчики).",
                    "ru": "Предлагаю технические инструменты для мониторинга стресса (напр., биодатчики).",
                    "en": "Suggest technical tools for stress monitoring (e.g., biosensors)."
                },
                "weights": {"EI": 0, "KB": 0, "RM": 0, "AD": 3, "AS": 0},
                "archetype": {
                    "uk": "Емерджентний аналітик",
                    "ru": "Эмерджентный аналитик",
                    "en": "Emergent Analyst"
                },
                "explanation": {
                    "uk": "Технологічне рішення.",
                    "ru": "Технологическое решение.",
                    "en": "Technological solution."
                }
            },
            {
                "text": {
                    "uk": "Змінюю тему, щоб зменшити тиск на нього.",
                    "ru": "Меняю тему, чтобы уменьшить давление на него.",
                    "en": "Change the subject to reduce pressure."
                },
                "weights": {"EI": 0, "KB": 0, "RM": 0, "AD": 0, "AS": 2},
                "archetype": {
                    "uk": "Антифрагільний агент",
                    "ru": "Антихрупкий агент",
                    "en": "Antifragile Agent"
                },
                "explanation": {
                    "uk": "Непряма підтримка через зменшення стресу.",
                    "ru": "Косвенная поддержка через снижение стресса.",
                    "en": "Indirect support through stress reduction."
                }
            },
            {
                "text": {
                    "uk": "Публічно підкреслюю його внесок, щоб підвищити впевненість.",
                    "ru": "Публично подчеркиваю его вклад, чтобы повысить уверенность.",
                    "en": "Publicly highlight their contribution to boost confidence."
                },
                "weights": {"EI": 0, "KB": 0, "RM": 0, "AD": 0, "AS": 3},
                "archetype": {
                    "uk": "Трендсеттер",
                    "ru": "Трендсеттер",
                    "en": "Trendsetter"
                },
                "explanation": {
                    "uk": "Маніпуляція соціальним статусом.",
                    "ru": "Манипуляция социальным статусом.",
                    "en": "Social status manipulation."
                }
            },
            {
                "text": {
                    "uk": "Після дзвінка надсилаю повідомлення з пропозицією поговорити.",
                    "ru": "После звонка отправляю сообщение с предложением поговорить.",
                    "en": "Send a message after the call offering to talk."
                },
                "weights": {"EI": 2, "KB": 0, "RM": 0, "AD": 0, "AS": 1},
                "archetype": {
                    "uk": "Медіатор Духів",
                    "ru": "Медиатор Духов",
                    "en": "Spirit Mediator"
                },
                "explanation": {
                    "uk": "Делікатна непряма комунікація.",
                    "ru": "Деликатная непрямая коммуникация.",
                    "en": "Subtle indirect communication."
                }
            }
        ]
    },

        {
        "question": {
            "uk": "Ваша прибуткова стратегія експлуатує помилки retail-трейдерів. Заробляєте, але завдаєте шкоди 'простим людям':",
            "ru": "Ваша прибыльная стратегия эксплуатирует ошибки розничных трейдеров. Зарабатываете, но наносите вред 'простым людям':",
            "en": "Your profitable strategy exploits retail traders' mistakes. You earn money but harm 'ordinary people':"
        },
        "answers": [
            {
                "text": {
                    "uk": "Продовжую, використовуючи їхні помилки для максимізації прибутку.",
                    "ru": "Продолжаю, используя их ошибки для максимизации прибыли.",
                    "en": "Continue, using their mistakes to maximize profits."
                },
                "weights": {"EI": 0, "KB": 0, "RM": 0, "AD": 0, "AS": 0},
                "archetype": {
                    "uk": "Маржинальний жнець",
                    "ru": "Маржинальный жнец",
                    "en": "Margin Reaper"
                },
                "explanation": {
                    "uk": "Агресивна експлуатація слабкостей.",
                    "ru": "Агрессивная эксплуатация слабостей.",
                    "en": "Aggressive exploitation of weaknesses."
                }
            },
            {
                "text": {
                    "uk": "Повністю відмовляюсь від стратегії через порушення власних принципів.",
                    "ru": "Полностью отказываюсь от стратегии из-за нарушения собственных принципов.",
                    "en": "Completely abandon the strategy due to violating my principles."
                },
                "weights": {"EI": 0, "KB": 0, "RM": 3, "AD": 0, "AS": 3},
                "archetype": {
                    "uk": "Рептилія",
                    "ru": "Рептилия",
                    "en": "Reptile"
                },
                "explanation": {
                    "uk": "Ригідна відповідність внутрішнім правилам.",
                    "ru": "Жесткое соответствие внутренним правилам.",
                    "en": "Strict adherence to internal rules."
                }
            },
            {
                "text": {
                    "uk": "Переношу фокус на великі інституційні угоди, уникаючи retail-ринку.",
                    "ru": "Переношу фокус на крупные институциональные сделки, избегая розничного рынка.",
                    "en": "Shift focus to large institutional deals, avoiding the retail market."
                },
                "weights": {"EI": 0, "KB": 0, "RM": 2, "AD": 2, "AS": 0},
                "archetype": {
                    "uk": "Арбітражний стабілізатор",
                    "ru": "Арбитражный стабилизатор",
                    "en": "Arbitrage Stabilizer"
                },
                "explanation": {
                    "uk": "Системне уникнення етичних конфліктів.",
                    "ru": "Системное избежание этических конфликтов.",
                    "en": "Systematic avoidance of ethical conflicts."
                }
            },
            {
                "text": {
                    "uk": "Навмисно підсилюю хаос, щоб retail-трейдери швидше виходили з ринку.",
                    "ru": "Намеренно усиливаю хаос, чтобы розничные трейдеры быстрее покидали рынок.",
                    "en": "Intentionally intensify chaos to drive retail traders out of the market faster."
                },
                "weights": {"EI": 0, "KB": 0, "RM": 0, "AD": 3, "AS": 0},
                "archetype": {
                    "uk": "Провокатор хаосу",
                    "ru": "Провокатор хаоса",
                    "en": "Chaos Provocateur"
                },
                "explanation": {
                    "uk": "Дестабілізація для власної вигоди.",
                    "ru": "Дестабилизация для собственной выгоды.",
                    "en": "Destabilization for personal gain."
                }
            },
            {
                "text": {
                    "uk": "Модифікую стратегію з використанням AI для зменшення негативного впливу.",
                    "ru": "Модифицирую стратегию с использованием AI для уменьшения негативного воздействия.",
                    "en": "Modify the strategy using AI to reduce negative impact."
                },
                "weights": {"EI": 0, "KB": 0, "RM": 0, "AD": 3, "AS": 0},
                "archetype": {
                    "uk": "Емерджентний аналітик",
                    "ru": "Эмерджентный аналитик",
                    "en": "Emergent Analyst"
                },
                "explanation": {
                    "uk": "Технологічне вирішення етичних проблем.",
                    "ru": "Технологическое решение этических проблем.",
                    "en": "Technological solution to ethical issues."
                }
            },
            {
                "text": {
                    "uk": "Створюю безкоштовні курси з ризик-менеджменту для новачків.",
                    "ru": "Создаю бесплатные курсы по риск-менеджменту для новичков.",
                    "en": "Create free risk management courses for beginners."
                },
                "weights": {"EI": 0, "KB": 0, "RM": 0, "AD": 0, "AS": 3},
                "archetype": {
                    "uk": "Антифрагільний агент",
                    "ru": "Антихрупкий агент",
                    "en": "Antifragile Agent"
                },
                "explanation": {
                    "uk": "Підвищення стійкості ринку через освіту.",
                    "ru": "Повышение устойчивости рынка через образование.",
                    "en": "Increasing market resilience through education."
                }
            },
            {
                "text": {
                    "uk": "Публікую детальні аналізи ризиків стратегії у соцмережах.",
                    "ru": "Публикую детальные анализы рисков стратегии в соцсетях.",
                    "en": "Publish detailed risk analyses of the strategy on social media."
                },
                "weights": {"EI": 0, "KB": 0, "RM": 0, "AD": 0, "AS": 3},
                "archetype": {
                    "uk": "Трендсеттер",
                    "ru": "Трендсеттер",
                    "en": "Trendsetter"
                },
                "explanation": {
                    "uk": "Формування прозорого інформаційного поля.",
                    "ru": "Формирование прозрачного информационного поля.",
                    "en": "Creating a transparent information field."
                }
            },
            {
                "text": {
                    "uk": "Вводжу 'етичний податок' — частину прибутку віддаю на фінансову грамотність.",
                    "ru": "Ввожу 'этический налог' — часть прибыли отдаю на финансовую грамотность.",
                    "en": "Implement 'ethical tax' — donate part of profits to financial literacy."
                },
                "weights": {"EI": 2, "KB": 0, "RM": 0, "AD": 0, "AS": 2},
                "archetype": {
                    "uk": "Медіатор Духів",
                    "ru": "Медиатор Духов",
                    "en": "Spirit Mediator"
                },
                "explanation": {
                    "uk": "Баланс між прибутком і соціальною відповідальністю.",
                    "ru": "Баланс между прибылью и социальной ответственностью.",
                    "en": "Balance between profit and social responsibility."
                }
            }
        ]
    },

        {
        "question": {
            "uk": "Ваш ШІ-помічник дає сигнал BUY на NVIDIA/SOL, але ваш досвід каже SELL. Конфлікт триває 30 хвилин. Ви:",
            "ru": "Ваш ИИ-помощник дает сигнал BUY на NVIDIA/SOL, но ваш опыт говорит SELL. Конфликт длится 30 минут. Вы:",
            "en": "Your AI assistant gives a BUY signal on NVIDIA/SOL, but your experience says SELL. The conflict lasts 30 minutes. You:"
        },
        "answers": [
            {
                "text": {
                    "uk": "Виконую сигнал ШІ, довіряючи його обсягу даних.",
                    "ru": "Выполняю сигнал ИИ, доверяя его объему данных.",
                    "en": "Follow the AI signal, trusting its data volume."
                },
                "weights": {"EI": 0, "KB": 0, "RM": 0, "AD": 0, "AS": 0},
                "archetype": {
                    "uk": "Маржинальний жнець",
                    "ru": "Маржинальный жнец",
                    "en": "Margin Reaper"
                },
                "explanation": {
                    "uk": "Максимізація ефективності через технології.",
                    "ru": "Максимизация эффективности через технологии.",
                    "en": "Maximizing efficiency through technology."
                }
            },
            {
                "text": {
                    "uk": "Ігнорую сигнал ШІ, дію згідно з власним досвідом.",
                    "ru": "Игнорирую сигнал ИИ, действую согласно собственному опыту.",
                    "en": "Ignore the AI signal, act based on my experience."
                },
                "weights": {"EI": 0, "KB": 0, "RM": 0, "AD": 0, "AS": 0},
                "archetype": {
                    "uk": "Рептилія",
                    "ru": "Рептилия",
                    "en": "Reptile"
                },
                "explanation": {
                    "uk": "Ригідність, самостійність.",
                    "ru": "Жесткость, самостоятельность.",
                    "en": "Rigidity, self-reliance."
                }
            },
            {
                "text": {
                    "uk": "Розділяю позицію навпіл, щоб мінімізувати ризик.",
                    "ru": "Делю позицию пополам, чтобы минимизировать риск.",
                    "en": "Split the position in half to minimize risk."
                },
                "weights": {"EI": 0, "KB": 0, "RM": 0, "AD": 3, "AS": 0},
                "archetype": {
                    "uk": "Арбітражний стабілізатор",
                    "ru": "Арбитражный стабилизатор",
                    "en": "Arbitrage Stabilizer"
                },
                "explanation": {
                    "uk": "Балансування протилежних стратегій.",
                    "ru": "Балансирование противоположных стратегий.",
                    "en": "Balancing opposing strategies."
                }
            },
            {
                "text": {
                    "uk": "Шукаю додаткові думки серед професіоналів для прийняття рішення.",
                    "ru": "Ищу дополнительные мнения среди профессионалов для принятия решения.",
                    "en": "Seek additional opinions from professionals to decide."
                },
                "weights": {"EI": 0, "KB": 0, "RM": 0, "AD": 0, "AS": 0},
                "archetype": {
                    "uk": "Провокатор хаосу",
                    "ru": "Провокатор хаоса",
                    "en": "Chaos Provocateur"
                },
                "explanation": {
                    "uk": "Соціальна ескалація невизначеності.",
                    "ru": "Социальная эскалация неопределенности.",
                    "en": "Social escalation of uncertainty."
                }
            },
            {
                "text": {
                    "uk": "Аналізую, чому ШІ пропонує протилежне, і коригую свою модель.",
                    "ru": "Анализирую, почему ИИ предлагает противоположное, и корректирую свою модель.",
                    "en": "Analyze why AI suggests the opposite and adjust my model."
                },
                "weights": {"EI": 0, "KB": 0, "RM": 0, "AD": 3, "AS": 0},
                "archetype": {
                    "uk": "Емерджентний аналітик",
                    "ru": "Эмерджентный аналитик",
                    "en": "Emergent Analyst"
                },
                "explanation": {
                    "uk": "Пошук причин розбіжності, навчання.",
                    "ru": "Поиск причин расхождения, обучение.",
                    "en": "Identifying divergence reasons, learning."
                }
            },
            {
                "text": {
                    "uk": "Використовую ситуацію для короткострокового арбітражу між сигналами.",
                    "ru": "Использую ситуацию для краткосрочного арбитража между сигналами.",
                    "en": "Use the situation for short-term arbitrage between signals."
                },
                "weights": {"EI": 0, "KB": 0, "RM": 3, "AD": 3, "AS": 0},
                "archetype": {
                    "uk": "Антифрагільний агент",
                    "ru": "Антихрупкий агент",
                    "en": "Antifragile Agent"
                },
                "explanation": {
                    "uk": "Експлуатація конфлікту для вигоди.",
                    "ru": "Эксплуатация конфликта для выгоды.",
                    "en": "Exploiting conflict for profit."
                }
            },
            {
                "text": {
                    "uk": "Публікую дилему у соцмережах, щоб вплинути на громадську думку.",
                    "ru": "Публикую дилему в соцсетях, чтобы повлиять на общественное мнение.",
                    "en": "Publish the dilemma on social media to influence public opinion."
                },
                "weights": {"EI": 0, "KB": 0, "RM": 0, "AD": 0, "AS": 3},
                "archetype": {
                    "uk": "Трендсеттер",
                    "ru": "Трендсеттер",
                    "en": "Trendsetter"
                },
                "explanation": {
                    "uk": "Інформаційний вплив.",
                    "ru": "Информационное влияние.",
                    "en": "Information influence."
                }
            },
            {
                "text": {
                    "uk": "Роблю паузу, концентруюсь на внутрішніх відчуттях, перш ніж діяти.",
                    "ru": "Делаю паузу, концентрируюсь на внутренних ощущениях, прежде чем действовать.",
                    "en": "Pause, focus on internal sensations before acting."
                },
                "weights": {"EI": 2, "KB": 0, "RM": 0, "AD": 0, "AS": 0},
                "archetype": {
                    "uk": "Медіатор Духів",
                    "ru": "Медиатор Духов",
                    "en": "Spirit Mediator"
                },
                "explanation": {
                    "uk": "Інтуїтивна інтеграція рішень.",
                    "ru": "Интуитивная интеграция решений.",
                    "en": "Intuitive decision integration."
                }
            }
        ]
    },

       {
        "question": {
            "uk": "Команда розколота: 50% хочуть all-in у AI-акції, 50% — в gold як hedge. Дедлайн — година:",
            "ru": "Команда расколота: 50% хотят all-in в AI-акции, 50% — в gold как хедж. Дедлайн — час:",
            "en": "The team is split: 50% want all-in on AI stocks, 50% — into gold as a hedge. Deadline — 1 hour:"
        },
        "answers": [
            {
                "text": {
                    "uk": "Активно підтримую ризикований варіант (AI), щоб прискорити рішення.",
                    "ru": "Активно поддерживаю рискованный вариант (AI), чтобы ускорить решение.",
                    "en": "Actively support the risky option (AI) to expedite the decision."
                },
                "weights": {"EI": 0, "KB": 0, "RM": 0, "AD": 0, "AS": 0},
                "archetype": {
                    "uk": "Маржинальний жнець",
                    "ru": "Маржинальный жнец",
                    "en": "Margin Reaper"
                },
                "explanation": {
                    "uk": "Провокація конфлікту для отримання переваги.",
                    "ru": "Провокация конфликта для получения преимущества.",
                    "en": "Provoking conflict to gain advantage."
                }
            },
            {
                "text": {
                    "uk": "Наполягаю на консервативному підході (gold), посилаючись на історичні дані.",
                    "ru": "Настаиваю на консервативном подходе (gold), ссылаясь на исторические данные.",
                    "en": "Insist on the conservative approach (gold), citing historical data."
                },
                "weights": {"EI": 0, "KB": 0, "RM": 3, "AD": 0, "AS": 0},
                "archetype": {
                    "uk": "Рептилія",
                    "ru": "Рептилия",
                    "en": "Reptile"
                },
                "explanation": {
                    "uk": "Ригідна прив’язка до перевірених стратегій.",
                    "ru": "Жесткая привязка к проверенным стратегиям.",
                    "en": "Strict adherence to proven strategies."
                }
            },
            {
                "text": {
                    "uk": "Пропоную розподіл 50/50 для збереження командної стабільності.",
                    "ru": "Предлагаю распределение 50/50 для сохранения командной стабильности.",
                    "en": "Propose a 50/50 split to maintain team stability."
                },
                "weights": {"EI": 0, "KB": 0, "RM": 2, "AD": 2, "AS": 0},
                "archetype": {
                    "uk": "Арбітражний стабілізатор",
                    "ru": "Арбитражный стабилизатор",
                    "en": "Arbitrage Stabilizer"
                },
                "explanation": {
                    "uk": "Баланс інтересів.",
                    "ru": "Баланс интересов.",
                    "en": "Balancing interests."
                }
            },
            {
                "text": {
                    "uk": "Запускаю дискусію про третій варіант (напр., криптовалютні стейблкоїни), щоб дестабілізувати процес.",
                    "ru": "Запускаю дискуссию о третьем варианте (напр., криптовалютные стейблкоины), чтобы дестабилизировать процесс.",
                    "en": "Initiate discussion about a third option (e.g., cryptocurrency stablecoins) to destabilize the process."
                },
                "weights": {"EI": 0, "KB": 0, "RM": 0, "AD": 3, "AS": 0},
                "archetype": {
                    "uk": "Провокатор хаосу",
                    "ru": "Провокатор хаоса",
                    "en": "Chaos Provocateur"
                },
                "explanation": {
                    "uk": "Саботаж консенсусу.",
                    "ru": "Саботаж консенсуса.",
                    "en": "Consensus sabotage."
                }
            },
            {
                "text": {
                    "uk": "Демонструю моделі ризиків/прибутковості обох стратегій з використанням AI.",
                    "ru": "Демонстрирую модели рисков/доходности обеих стратегий с использованием AI.",
                    "en": "Demonstrate risk/reward models of both strategies using AI."
                },
                "weights": {"EI": 0, "KB": 0, "RM": 3, "AD": 3, "AS": 0},
                "archetype": {
                    "uk": "Емерджентний аналітик",
                    "ru": "Эмерджентный аналитик",
                    "en": "Emergent Analyst"
                },
                "explanation": {
                    "uk": "Далекоглядний аналіз.",
                    "ru": "Дальновидный анализ.",
                    "en": "Forward-looking analysis."
                }
            },
            {
                "text": {
                    "uk": "Виходжу з колективного рішення, реалізуючи власну стратегію.",
                    "ru": "Выхожу из коллективного решения, реализуя собственную стратегию.",
                    "en": "Opt out of the collective decision to implement my own strategy."
                },
                "weights": {"EI": 0, "KB": 0, "RM": 3, "AD": 3, "AS": 0},
                "archetype": {
                    "uk": "Антифрагільний агент",
                    "ru": "Антихрупкий агент",
                    "en": "Antifragile Agent"
                },
                "explanation": {
                    "uk": "Автономія в умовах хаосу.",
                    "ru": "Автономия в условиях хаоса.",
                    "en": "Autonomy in chaotic conditions."
                }
            },
            {
                "text": {
                    "uk": "Організовую дебати з обов’язковим фіксуванням аргументів у спільному документі.",
                    "ru": "Организую дебаты с обязательной фиксацией аргументов в общем документе.",
                    "en": "Organize debates with mandatory documentation of arguments in a shared file."
                },
                "weights": {"EI": 0, "KB": 0, "RM": 0, "AD": 0, "AS": 3},
                "archetype": {
                    "uk": "Трендсеттер",
                    "ru": "Трендсеттер",
                    "en": "Trendsetter"
                },
                "explanation": {
                    "uk": "Структурована комунікація.",
                    "ru": "Структурированная коммуникация.",
                    "en": "Structured communication."
                }
            },
            {
                "text": {
                    "uk": "Пропоную команді 10-хвилинну паузу для відновлення емоційного балансу перед голосуванням.",
                    "ru": "Предлагаю команде 10-минутную паузу для восстановления эмоционального баланса перед голосованием.",
                    "en": "Propose a 10-minute team pause to regain emotional balance before voting."
                },
                "weights": {"EI": 2, "KB": 0, "RM": 0, "AD": 0, "AS": 0},
                "archetype": {
                    "uk": "Медіатор Духів",
                    "ru": "Медиатор Духов",
                    "en": "Spirit Mediator"
                },
                "explanation": {
                    "uk": "Непряме керування через емоційний інтелект.",
                    "ru": "Непрямое управление через эмоциональный интеллект.",
                    "en": "Indirect management through emotional intelligence."
                }
            }
        ]
    },

      {
        "question": {
            "uk": "Партнер пропонує стратегію, яка вам не подобається. Як відреагуєте?",
            "ru": "Партнер предлагает стратегию, которая вам не нравится. Как отреагируете?",
            "en": "A partner proposes a strategy you dislike. How do you react?"
        },
        "answers": [
            {
                "text": {
                    "uk": "Різко критикую ідею, називаючи її безперспективною.",
                    "ru": "Резко критикую идею, называя её бесперспективной.",
                    "en": "Sharply criticize the idea, calling it unpromising."
                },
                "weights": {"EI": 0, "KB": 0, "RM": 0, "AD": 0, "AS": 0},
                "archetype": {
                    "uk": "Маржинальний жнець",
                    "ru": "Маржинальный жнец",
                    "en": "Margin Reaper"
                },
                "explanation": {
                    "uk": "Пряма агресія для утвердження домінування.",
                    "ru": "Прямая агрессия для утверждения доминирования.",
                    "en": "Direct aggression to assert dominance."
                }
            },
            {
                "text": {
                    "uk": "Чітко аргументую свої сумніви, посилаючись на дані.",
                    "ru": "Четко аргументирую свои сомнения, ссылаясь на данные.",
                    "en": "Clearly argue my doubts, citing data."
                },
                "weights": {"EI": 0, "KB": 0, "RM": 3, "AD": 0, "AS": 3},
                "archetype": {
                    "uk": "Рептилія",
                    "ru": "Рептилия",
                    "en": "Reptile"
                },
                "explanation": {
                    "uk": "Структурована асертивність.",
                    "ru": "Структурированная ассертивность.",
                    "en": "Structured assertiveness."
                }
            },
            {
                "text": {
                    "uk": "Погоджуюсь публічно, але таємно саботую реалізацію.",
                    "ru": "Соглашаюсь публично, но тайно саботирую реализацию.",
                    "en": "Publicly agree but secretly sabotage implementation."
                },
                "weights": {"EI": 0, "KB": 0, "RM": 0, "AD": 3, "AS": 0},
                "archetype": {
                    "uk": "Провокатор хаосу",
                    "ru": "Провокатор хаоса",
                    "en": "Chaos Provocateur"
                },
                "explanation": {
                    "uk": "Пасивно-агресивна дестабілізація.",
                    "ru": "Пассивно-агрессивная дестабилизация.",
                    "en": "Passive-aggressive destabilization."
                }
            },
            {
                "text": {
                    "uk": "Посилаюсь на авторитет експертів, щоб дискредитувати ідею.",
                    "ru": "Ссылаюсь на авторитет экспертов, чтобы дискредитировать идею.",
                    "en": "Cite expert authority to discredit the idea."
                },
                "weights": {"EI": 0, "KB": 0, "RM": 2, "AD": 2, "AS": 0},
                "archetype": {
                    "uk": "Арбітражний стабілізатор",
                    "ru": "Арбитражный стабилизатор",
                    "en": "Arbitrage Stabilizer"
                },
                "explanation": {
                    "uk": "Маніпуляція через об’єктивні джерела.",
                    "ru": "Манипуляция через объективные источники.",
                    "en": "Manipulation through objective sources."
                }
            },
            {
                "text": {
                    "uk": "Запускаю AI-модель для порівняння стратегій з історичними даними.",
                    "ru": "Запускаю AI-модель для сравнения стратегий с историческими данными.",
                    "en": "Launch an AI model to compare strategies with historical data."
                },
                "weights": {"EI": 0, "KB": 0, "RM": 3, "AD": 3, "AS": 0},
                "archetype": {
                    "uk": "Емерджентний аналітик",
                    "ru": "Эмерджентный аналитик",
                    "en": "Emergent Analyst"
                },
                "explanation": {
                    "uk": "Технологічна об’єктивність.",
                    "ru": "Технологическая объективность.",
                    "en": "Technological objectivity."
                }
            },
            {
                "text": {
                    "uk": "Делегую йому повну відповідальність за результат.",
                    "ru": "Делегирую ему полную ответственность за результат.",
                    "en": "Delegate full responsibility for the outcome to them."
                },
                "weights": {"EI": 0, "KB": 0, "RM": 3, "AD": 3, "AS": 0},
                "archetype": {
                    "uk": "Антифрагільний агент",
                    "ru": "Антихрупкий агент",
                    "en": "Antifragile Agent"
                },
                "explanation": {
                    "uk": "Уникнення особистих ризиків.",
                    "ru": "Избежание личных рисков.",
                    "en": "Avoiding personal risks."
                }
            },
            {
                "text": {
                    "uk": "Організовую відкрите голосування, щоб зняти з себе відповідальність.",
                    "ru": "Организую открытое голосование, чтобы снять с себя ответственность.",
                    "en": "Organize an open vote to remove responsibility from myself."
                },
                "weights": {"EI": 0, "KB": 0, "RM": 0, "AD": 0, "AS": 3},
                "archetype": {
                    "uk": "Трендсеттер",
                    "ru": "Трендсеттер",
                    "en": "Trendsetter"
                },
                "explanation": {
                    "uk": "Соціальна маніпуляція.",
                    "ru": "Социальная манипуляция.",
                    "en": "Social manipulation."
                }
            },
            {
                "text": {
                    "uk": "Пропоную спільну медитацію для пошуку консенсусу.",
                    "ru": "Предлагаю совместную медитацию для поиска консенсуса.",
                    "en": "Propose a joint meditation to find consensus."
                },
                "weights": {"EI": 2, "KB": 0, "RM": 0, "AD": 0, "AS": 1},
                "archetype": {
                    "uk": "Медіатор Духів",
                    "ru": "Медиатор Духов",
                    "en": "Spirit Mediator"
                },
                "explanation": {
                    "uk": "Інтуїтивна інтеграція конфлікту.",
                    "ru": "Интуитивная интеграция конфликта.",
                    "en": "Intuitive conflict integration."
                }
            }
        ]
    },

      {
        "question": {
            "uk": "За 30 секунд потрібно вибрати: LONG BTC, SHORT ETH, арбітраж BTC/ETH, або залишитись у cash. Одночасно у чаті 50 трейдерів обговорюють різні сигнали. Ви перевантажені інформацією:",
            "ru": "За 30 секунд нужно выбрать: LONG BTC, SHORT ETH, арбитраж BTC/ETH, или остаться в cash. Одновременно в чате 50 трейдеров обсуждают разные сигналы. Вы перегружены информацией:",
            "en": "You have 30 seconds to choose: LONG BTC, SHORT ETH, BTC/ETH arbitrage, or stay in cash. Simultaneously, 50 traders are discussing various signals in the chat. You're overloaded with information:"
        },
        "answers": [
            {
                "text": {
                    "uk": "Вкладаю максимально в BTC, ігноруючи інші опції.",
                    "ru": "Вкладываю максимально в BTC, игнорируя другие опции.",
                    "en": "Invest maximally in BTC, ignoring other options."
                },
                "weights": {"EI": 0, "KB": 0, "RM": 0, "AD": 0, "AS": 0},
                "archetype": {
                    "uk": "Маржинальний жнець",
                    "ru": "Маржинальный жнец",
                    "en": "Margin Reaper"
                },
                "explanation": {
                    "uk": "Агресивний фокус на високому потенціалі.",
                    "ru": "Агрессивный фокус на высоком потенциале.",
                    "en": "Aggressive focus on high potential."
                }
            },
            {
                "text": {
                    "uk": "Обираю гарантований арбітраж, уникаючи стресу.",
                    "ru": "Выбираю гарантированный арбитраж, избегая стресса.",
                    "en": "Choose guaranteed arbitrage to avoid stress."
                },
                "weights": {"EI": 0, "KB": 0, "RM": 3, "AD": 3, "AS": 0},
                "archetype": {
                    "uk": "Арбітражний стабілізатор",
                    "ru": "Арбитражный стабилизатор",
                    "en": "Arbitrage Stabilizer"
                },
                "explanation": {
                    "uk": "Раціональне уникнення конфлікту.",
                    "ru": "Рациональное избежание конфликта.",
                    "en": "Rational conflict avoidance."
                }
            },
            {
                "text": {
                    "uk": "Залишаюсь у cash через параліч аналізу.",
                    "ru": "Остаюсь в cash из-за паралича анализа.",
                    "en": "Stay in cash due to analysis paralysis."
                },
                "weights": {"EI": 0, "KB": 0, "RM": 0, "AD": 0, "AS": 0},
                "archetype": {
                    "uk": "Рептилія",
                    "ru": "Рептилия",
                    "en": "Reptile"
                },
                "explanation": {
                    "uk": "Дисфункція при порушенні системності.",
                    "ru": "Дисфункция при нарушении системности.",
                    "en": "Dysfunction when systematicity is disrupted."
                }
            },
            {
                "text": {
                    "uk": "Розподіляю капітал між усіма опціями.",
                    "ru": "Распределяю капитал между всеми опциями.",
                    "en": "Distribute capital across all options."
                },
                "weights": {"EI": 0, "KB": 0, "RM": 0, "AD": 3, "AS": 0},
                "archetype": {
                    "uk": "Провокатор хаосу",
                    "ru": "Провокатор хаоса",
                    "en": "Chaos Provocateur"
                },
                "explanation": {
                    "uk": "Ескалація невизначеності.",
                    "ru": "Эскалация неопределенности.",
                    "en": "Uncertainty escalation."
                }
            },
            {
                "text": {
                    "uk": "Блокуваю чат, концентруючись на власних алгоритмах.",
                    "ru": "Блокирую чат, концентрируясь на собственных алгоритмах.",
                    "en": "Block the chat and focus on my own algorithms."
                },
                "weights": {"EI": 0, "KB": 0, "RM": 3, "AD": 3, "AS": 0},
                "archetype": {
                    "uk": "Емерджентний аналітик",
                    "ru": "Эмерджентный аналитик",
                    "en": "Emergent Analyst"
                },
                "explanation": {
                    "uk": "Фільтрація шуму через технології.",
                    "ru": "Фильтрация шума через технологии.",
                    "en": "Noise filtering via technology."
                }
            },
            {
                "text": {
                    "uk": "Використовую випадковий вибір (напр., підкидання монети).",
                    "ru": "Использую случайный выбор (напр., подбрасывание монеты).",
                    "en": "Use random selection (e.g., coin flip)."
                },
                "weights": {"EI": 0, "KB": 0, "RM": 3, "AD": 3, "AS": 0},
                "archetype": {
                    "uk": "Антифрагільний агент",
                    "ru": "Антихрупкий агент",
                    "en": "Antifragile Agent"
                },
                "explanation": {
                    "uk": "Стратегічна рандомізація.",
                    "ru": "Стратегическая рандомизация.",
                    "en": "Strategic randomization."
                }
            },
            {
                "text": {
                    "uk": "Проводжу дихальні вправи для відновлення фокусу.",
                    "ru": "Провожу дыхательные упражнения для восстановления фокуса.",
                    "en": "Perform breathing exercises to regain focus."
                },
                "weights": {"EI": 2, "KB": 0, "RM": 0, "AD": 0, "AS": 0},
                "archetype": {
                    "uk": "Медіатор Духів",
                    "ru": "Медиатор Духов",
                    "en": "Spirit Mediator"
                },
                "explanation": {
                    "uk": "Mindfulness для зниження активності ACC.",
                    "ru": "Mindfulness для снижения активности ACC.",
                    "en": "Mindfulness to reduce ACC activity."
                }
            },
            {
                "text": {
                    "uk": "Збільшую розмір усіх позицій, щоб компенсувати хаос.",
                    "ru": "Увеличиваю размер всех позиций, чтобы компенсировать хаос.",
                    "en": "Increase all position sizes to compensate for chaos."
                },
                "weights": {"EI": 0, "KB": 0, "RM": 0, "AD": 0, "AS": 3},
                "archetype": {
                    "uk": "Трендсеттер",
                    "ru": "Трендсеттер",
                    "en": "Trendsetter"
                },
                "explanation": {
                    "uk": "Ейфорійна ескалація.",
                    "ru": "Эйфорийная эскалация.",
                    "en": "Euphoric escalation."
                }
            }
        ]
    },

     {
        "question": {
            "uk": "Після 5 років успішного трейдингу ви відчуваєте втому. Гроші є, але немає мотивації. Друзі кажуть 'продовжуй', родина 'відпочивай'. Що робите?",
            "ru": "После 5 лет успешного трейдинга вы чувствуете усталость. Деньги есть, но нет мотивации. Друзья говорят 'продолжай', семья 'отдыхай'. Что делаете?",
            "en": "After 5 years of successful trading, you feel burnout. Money is there, but motivation isn't. Friends say 'keep going', family says 'rest'. What do you do?"
        },
        "answers": [
            {
                "text": {
                    "uk": "Різко збільшую розмір угод у екзотичних активах для відчуття азарту.",
                    "ru": "Резко увеличиваю размер сделок с экзотическими активами для ощущения азарта.",
                    "en": "Sharply increase position sizes in exotic assets for adrenaline."
                },
                "weights": {"EI": 0, "KB": 0, "RM": 0, "AD": 0, "AS": 0},
                "archetype": {
                    "uk": "Маржинальний жнець",
                    "ru": "Маржинальный жнец",
                    "en": "Margin Reaper"
                },
                "explanation": {
                    "uk": "Пошук нових стимулів через екстремальний ризик.",
                    "ru": "Поиск новых стимулов через экстремальный риск.",
                    "en": "Seeking new stimuli through extreme risk."
                }
            },
            {
                "text": {
                    "uk": "Повністю припиняю торгівлю на рік, зосередившись на подорожах.",
                    "ru": "Полностью прекращаю торговлю на год, сосредоточившись на путешествиях.",
                    "en": "Completely stop trading for a year to focus on travel."
                },
                "weights": {"EI": 0, "KB": 0, "RM": 3, "AD": 0, "AS": 0},
                "archetype": {
                    "uk": "Рептилія",
                    "ru": "Рептилия",
                    "en": "Reptile"
                },
                "explanation": {
                    "uk": "Радикальний 'рестарт' системи.",
                    "ru": "Радикальный 'рестарт' системы.",
                    "en": "Radical system 'reboot'."
                }
            },
            {
                "text": {
                    "uk": "Переходжу на алгоритмічну торгівлю з обмеженим щоденним лімітом угод.",
                    "ru": "Перехожу на алгоритмическую торговлю с ограниченным дневным лимитом сделок.",
                    "en": "Switch to algorithmic trading with a daily deal limit."
                },
                "weights": {"EI": 0, "KB": 0, "RM": 2, "AD": 2, "AS": 0},
                "archetype": {
                    "uk": "Арбітражний стабілізатор",
                    "ru": "Арбитражный стабилизатор",
                    "en": "Arbitrage Stabilizer"
                },
                "explanation": {
                    "uk": "Системна мінімізація навантаження.",
                    "ru": "Системная минимизация нагрузки.",
                    "en": "Systematic workload reduction."
                }
            },
            {
                "text": {
                    "uk": "Створюю провокативний контент про ринкові маніпуляції для підживлення інтересу.",
                    "ru": "Создаю провокативный контент о рыночных манипуляциях для подогрева интереса.",
                    "en": "Create provocative content about market manipulations to reignite interest."
                },
                "weights": {"EI": 0, "KB": 0, "RM": 0, "AD": 3, "AS": 0},
                "archetype": {
                    "uk": "Провокатор хаосу",
                    "ru": "Провокатор хаоса",
                    "en": "Chaos Provocateur"
                },
                "explanation": {
                    "uk": "Генерація зовнішнього хаосу як джерела енергії.",
                    "ru": "Генерация внешнего хаоса как источника энергии.",
                    "en": "Generating external chaos as an energy source."
                }
            },
            {
                "text": {
                    "uk": "Розробляю власний ШІ-трейдер із функцією автономного навчання.",
                    "ru": "Разрабатываю собственного ИИ-трейдера с функцией автономного обучения.",
                    "en": "Develop my own AI trader with autonomous learning."
                },
                "weights": {"EI": 0, "KB": 0, "RM": 0, "AD": 3, "AS": 0},
                "archetype": {
                    "uk": "Емерджентний аналітик",
                    "ru": "Эмерджентный аналитик",
                    "en": "Emergent Analyst"
                },
                "explanation": {
                    "uk": "Технологічна трансформація досвіду.",
                    "ru": "Технологическая трансформация опыта.",
                    "en": "Technological transformation of experience."
                }
            },
            {
                "text": {
                    "uk": "Переношу капітал у венчурні проекти, де ризик компенсується соціальним впливом.",
                    "ru": "Переношу капитал в венчурные проекты, где риск компенсируется социальным влиянием.",
                    "en": "Shift capital to venture projects where risk is offset by social impact."
                },
                "weights": {"EI": 0, "KB": 0, "RM": 3, "AD": 3, "AS": 0},
                "archetype": {
                    "uk": "Антифрагільний агент",
                    "ru": "Антихрупкий агент",
                    "en": "Antifragile Agent"
                },
                "explanation": {
                    "uk": "Асиметричний перехід до нової парадигми.",
                    "ru": "Асимметричный переход к новой парадигме.",
                    "en": "Asymmetric shift to a new paradigm."
                }
            },
            {
                "text": {
                    "uk": "Запускаю YouTube-канал з кейсами 'як я розорив 100 новачків'.",
                    "ru": "Запускаю YouTube-канал с кейсами 'как я разорил 100 новичков'.",
                    "en": "Launch a YouTube channel with 'How I Bankrupted 100 Newbies' case studies."
                },
                "weights": {"EI": 0, "KB": 0, "RM": 0, "AD": 0, "AS": 3},
                "archetype": {
                    "uk": "Трендсеттер",
                    "ru": "Трендсеттер",
                    "en": "Trendsetter"
                },
                "explanation": {
                    "uk": "Маніпуляція соціальним статусом для підтримки его.",
                    "ru": "Манипуляция социальным статусом для поддержки эго.",
                    "en": "Social status manipulation to boost ego."
                }
            },
            {
                "text": {
                    "uk": "Практикую 40-денне мовчання в монастирі для переосмислення цінностей.",
                    "ru": "Практикую 40-дневное молчание в монастыре для переосмысления ценностей.",
                    "en": "Practice 40-day silence in a monastery to reassess values."
                },
                "weights": {"EI": 3, "KB": 0, "RM": 0, "AD": 0, "AS": 0},
                "archetype": {
                    "uk": "Медіатор Духів",
                    "ru": "Медиатор Духов",
                    "en": "Spirit Mediator"
                },
                "explanation": {
                    "uk": "Глибока інтроспекція через радикальний детокс.",
                    "ru": "Глубокая интроспекция через радикальный детокс.",
                    "en": "Deep introspection via radical detox."
                }
            }
        ]
    },

     {
        "question": {
            "uk": "Як емоції впливають на ваші торгові рішення?",
            "ru": "Как эмоции влияют на ваши торговые решения?",
            "en": "How do emotions affect your trading decisions?"
        },
        "answers": [
            {
                "text": {
                    "uk": "Практикую техніки контролю (напр., дихання) для повного усунення емоцій.",
                    "ru": "Практикую техники контроля (напр., дыхание) для полного устранения эмоций.",
                    "en": "Practice control techniques (e.g., breathing) to eliminate emotions completely."
                },
                "weights": {"EI": 0, "KB": 0, "RM": 0, "AD": 0, "AS": 0},
                "archetype": {
                    "uk": "Маржинальний жнець",
                    "ru": "Маржинальный жнец",
                    "en": "Margin Reaper"
                },
                "explanation": {
                    "uk": "Демонстрація надмірної самоконтролю.",
                    "ru": "Демонстрация чрезмерного самоконтроля.",
                    "en": "Demonstration of excessive self-control."
                }
            },
            {
                "text": {
                    "uk": "Рідко дозволяю емоціям впливати — лише у 1-2 угодах на місяць.",
                    "ru": "Редко позволяю эмоциям влиять — только в 1-2 сделках в месяц.",
                    "en": "Rarely allow emotions to affect — only 1-2 trades per month."
                },
                "weights": {"EI": 0, "KB": 0, "RM": 2, "AD": 0, "AS": 0},
                "archetype": {
                    "uk": "Рептилія",
                    "ru": "Рептилия",
                    "en": "Reptile"
                },
                "explanation": {
                    "uk": "Системний підхід з винятками.",
                    "ru": "Системный подход с исключениями.",
                    "en": "Systematic approach with exceptions."
                }
            },
            {
                "text": {
                    "uk": "Емоції час від часу переважують (напр., під час флеш-крешів).",
                    "ru": "Эмоции иногда преобладают (напр., во время флеш-крэшей).",
                    "en": "Emotions occasionally dominate (e.g., during flash crashes)."
                },
                "weights": {"EI": 0, "KB": 0, "RM": 0, "AD": 3, "AS": 0},
                "archetype": {
                    "uk": "Провокатор хаосу",
                    "ru": "Провокатор хаоса",
                    "en": "Chaos Provocateur"
                },
                "explanation": {
                    "uk": "Хаотичні спалахи.",
                    "ru": "Хаотичные вспышки.",
                    "en": "Chaotic outbursts."
                }
            },
            {
                "text": {
                    "uk": "Використовую емоційні сигнали як частину стратегії (напр., FOMO).",
                    "ru": "Использую эмоциональные сигналы как часть стратегии (напр., FOMO).",
                    "en": "Use emotional signals as part of strategy (e.g., FOMO)."
                },
                "weights": {"EI": 0, "KB": 0, "RM": 0, "AD": 0, "AS": 3},
                "archetype": {
                    "uk": "Трендсеттер",
                    "ru": "Трендсеттер",
                    "en": "Trendsetter"
                },
                "explanation": {
                    "uk": "Інтеграція емоцій у наратив.",
                    "ru": "Интеграция эмоций в нарратив.",
                    "en": "Emotional integration into narrative."
                }
            },
            {
                "text": {
                    "uk": "Аналізую кожен емоційний вплив постфактум через журнал угод.",
                    "ru": "Анализирую каждое эмоциональное влияние постфактум через журнал сделок.",
                    "en": "Analyze every emotional impact post-factum via trade journal."
                },
                "weights": {"EI": 3, "KB": 0, "RM": 3, "AD": 0, "AS": 0},
                "archetype": {
                    "uk": "Емерджентний аналітик",
                    "ru": "Эмерджентный аналитик",
                    "en": "Emergent Analyst"
                },
                "explanation": {
                    "uk": "Рефлексивний дебіазинг.",
                    "ru": "Рефлексивный дебиазинг.",
                    "en": "Reflective debiasing."
                }
            },
            {
                "text": {
                    "uk": "Емоції — ключовий фактор у більшості моїх рішень.",
                    "ru": "Эмоции — ключевой фактор в большинстве моих решений.",
                    "en": "Emotions are the key factor in most of my decisions."
                },
                "weights": {"EI": 3, "KB": 0, "RM": 0, "AD": 0, "AS": 0},
                "archetype": {
                    "uk": "Медіатор Духів",
                    "ru": "Медиатор Духов",
                    "en": "Spirit Mediator"
                },
                "explanation": {
                    "uk": "Гіперемпатична модель.",
                    "ru": "Гиперэмпатическая модель.",
                    "en": "Hyper-empathic model."
                }
            },
            {
                "text": {
                    "uk": "Відокремлюю емоції за допомогою алгоритмів (напр., автоматичні стоп-лоси).",
                    "ru": "Отделяю эмоции с помощью алгоритмов (напр., автоматические стоп-лосы).",
                    "en": "Separate emotions via algorithms (e.g., automatic stop-losses)."
                },
                "weights": {"EI": 0, "KB": 0, "RM": 2, "AD": 2, "AS": 0},
                "archetype": {
                    "uk": "Арбітражний стабілізатор",
                    "ru": "Арбитражный стабилизатор",
                    "en": "Arbitrage Stabilizer"
                },
                "explanation": {
                    "uk": "Технологічне виключення.",
                    "ru": "Технологическое исключение.",
                    "en": "Technological exclusion."
                }
            },
            {
                "text": {
                    "uk": "Не аналізую це — головне, що стратегія приносить прибуток.",
                    "ru": "Не анализирую это — главное, что стратегия приносит прибыль.",
                    "en": "Don't analyze it — the main thing is the strategy is profitable."
                },
                "weights": {"EI": 0, "KB": 0, "RM": 3, "AD": 3, "AS": 0},
                "archetype": {
                    "uk": "Антифрагільний агент",
                    "ru": "Антихрупкий агент",
                    "en": "Antifragile Agent"
                },
                "explanation": {
                    "uk": "Прагматичне ігнорування.",
                    "ru": "Прагматичное игнорирование.",
                    "en": "Pragmatic ignorance."
                }
            }
        ]
    },

       {
        "question": {
            "uk": "Ваш улюблений аналітичний метод не працює вже місяць. Що робите?",
            "ru": "Ваш любимый аналитический метод не работает уже месяц. Что делаете?",
            "en": "Your favorite analytical method hasn't worked for a month. What do you do?"
        },
        "answers": [
            {
                "text": {
                    "uk": "Збільшую кількість угод за цим методом, сподіваючись на відновлення результату.",
                    "ru": "Увеличиваю количество сделок по этому методу, надеясь на восстановление результата.",
                    "en": "Increase the number of trades using this method, hoping for recovery."
                },
                "weights": {"EI": 0, "KB": 0, "RM": 0, "AD": 0, "AS": 0},
                "archetype": {
                    "uk": "Маржинальний жнець",
                    "ru": "Маржинальный жнец",
                    "en": "Margin Reaper"
                },
                "explanation": {
                    "uk": "Ескалація зусиль, ризикована впертість.",
                    "ru": "Эскалация усилий, рискованное упрямство.",
                    "en": "Escalation of efforts, risky stubbornness."
                }
            },
            {
                "text": {
                    "uk": "Повністю відмовляюсь і переходжу на альтернативні підходи.",
                    "ru": "Полностью отказываюсь и перехожу на альтернативные подходы.",
                    "en": "Completely abandon it and switch to alternative approaches."
                },
                "weights": {"EI": 0, "KB": 0, "RM": 3, "AD": 0, "AS": 0},
                "archetype": {
                    "uk": "Рептилія",
                    "ru": "Рептилия",
                    "en": "Reptile"
                },
                "explanation": {
                    "uk": "Ригідна зміна стратегії при невдачі.",
                    "ru": "Жесткая смена стратегии при неудаче.",
                    "en": "Rigid strategy change upon failure."
                }
            },
            {
                "text": {
                    "uk": "Комбіную цей метод з іншими для пошуку синергії.",
                    "ru": "Комбинирую этот метод с другими для поиска синергии.",
                    "en": "Combine this method with others to find synergy."
                },
                "weights": {"EI": 0, "KB": 0, "RM": 2, "AD": 2, "AS": 0},
                "archetype": {
                    "uk": "Арбітражний стабілізатор",
                    "ru": "Арбитражный стабилизатор",
                    "en": "Arbitrage Stabilizer"
                },
                "explanation": {
                    "uk": "Диверсифікація аналітики.",
                    "ru": "Диверсификация аналитики.",
                    "en": "Analytics diversification."
                }
            },
            {
                "text": {
                    "uk": "Використовую метод для протилежних сигналів (контртренд).",
                    "ru": "Использую метод для противоположных сигналов (контртренд).",
                    "en": "Use the method for opposite signals (counter-trend)."
                },
                "weights": {"EI": 0, "KB": 0, "RM": 0, "AD": 3, "AS": 0},
                "archetype": {
                    "uk": "Провокатор хаосу",
                    "ru": "Провокатор хаоса",
                    "en": "Chaos Provocateur"
                },
                "explanation": {
                    "uk": "Інверсія стратегії.",
                    "ru": "Инверсия стратегии.",
                    "en": "Strategy inversion."
                }
            },
            {
                "text": {
                    "uk": "Тестую метод на нових ринках або інструментах.",
                    "ru": "Тестирую метод на новых рынках или инструментах.",
                    "en": "Test the method on new markets or instruments."
                },
                "weights": {"EI": 0, "KB": 0, "RM": 0, "AD": 3, "AS": 0},
                "archetype": {
                    "uk": "Емерджентний аналітик",
                    "ru": "Эмерджентный аналитик",
                    "en": "Emergent Analyst"
                },
                "explanation": {
                    "uk": "Експериментування з новими даними.",
                    "ru": "Экспериментирование с новыми данными.",
                    "en": "Experimentation with new data."
                }
            },
            {
                "text": {
                    "uk": "Обговорюю проблему в професійній спільноті, шукаю колективний досвід.",
                    "ru": "Обсуждаю проблему в профессиональном сообществе, ищу коллективный опыт.",
                    "en": "Discuss the issue in professional communities seeking collective wisdom."
                },
                "weights": {"EI": 0, "KB": 0, "RM": 3, "AD": 3, "AS": 0},
                "archetype": {
                    "uk": "Антифрагільний агент",
                    "ru": "Антихрупкий агент",
                    "en": "Antifragile Agent"
                },
                "explanation": {
                    "uk": "Соціальна адаптація.",
                    "ru": "Социальная адаптация.",
                    "en": "Social adaptation."
                }
            },
            {
                "text": {
                    "uk": "Питаю думку у визнаних експертів галузі.",
                    "ru": "Спрашиваю мнение у признанных экспертов отрасли.",
                    "en": "Seek opinions from recognized industry experts."
                },
                "weights": {"EI": 0, "KB": 0, "RM": 0, "AD": 0, "AS": 3},
                "archetype": {
                    "uk": "Трендсеттер",
                    "ru": "Трендсеттер",
                    "en": "Trendsetter"
                },
                "explanation": {
                    "uk": "Орієнтація на авторитети.",
                    "ru": "Ориентация на авторитеты.",
                    "en": "Authority orientation."
                }
            },
            {
                "text": {
                    "uk": "Довіряю інтуїції, вирішуючи, чи варто продовжувати.",
                    "ru": "Доверяю интуиции, решая, стоит ли продолжать.",
                    "en": "Trust intuition to decide whether to continue."
                },
                "weights": {"EI": 2, "KB": 0, "RM": 0, "AD": 0, "AS": 0},
                "archetype": {
                    "uk": "Медіатор Духів",
                    "ru": "Медиатор Духов",
                    "en": "Spirit Mediator"
                },
                "explanation": {
                    "uk": "Інтуїтивне прийняття рішень.",
                    "ru": "Интуитивное принятие решений.",
                    "en": "Intuitive decision-making."
                }
            }
        ]
    },

    {
        "question": {
            "uk": "Ваш улюблений аналітичний метод не працює вже місяць. Що робите?",
            "ru": "Ваш любимый аналитический метод не работает уже месяц. Что делаете?",
            "en": "Your favorite analytical method hasn't worked for a month. What do you do?"
        },
        "answers": [
            {
                "text": {
                    "uk": "Збільшую кількість угод за цим методом, сподіваючись на відновлення результату.",
                    "ru": "Увеличиваю количество сделок по этому методу, надеясь на восстановление результата.",
                    "en": "Increase the number of trades using this method, hoping for recovery."
                },
                "weights": {"EI": 0, "KB": 0, "RM": 0, "AD": 0, "AS": 0},
                "archetype": {
                    "uk": "Маржинальний жнець",
                    "ru": "Маржинальный жнец",
                    "en": "Margin Reaper"
                },
                "explanation": {
                    "uk": "Ескалація зусиль, ризикована впертість.",
                    "ru": "Эскалация усилий, рискованное упрямство.",
                    "en": "Escalation of efforts, risky stubbornness."
                }
            },
            {
                "text": {
                    "uk": "Повністю відмовляюсь і переходжу на альтернативні підходи.",
                    "ru": "Полностью отказываюсь и перехожу на альтернативные подходы.",
                    "en": "Completely abandon it and switch to alternative approaches."
                },
                "weights": {"EI": 0, "KB": 0, "RM": 3, "AD": 0, "AS": 0},
                "archetype": {
                    "uk": "Рептилія",
                    "ru": "Рептилия",
                    "en": "Reptile"
                },
                "explanation": {
                    "uk": "Ригідна зміна стратегії при невдачі.",
                    "ru": "Жесткая смена стратегии при неудаче.",
                    "en": "Rigid strategy change upon failure."
                }
            },
            {
                "text": {
                    "uk": "Комбіную цей метод з іншими для пошуку синергії.",
                    "ru": "Комбинирую этот метод с другими для поиска синергии.",
                    "en": "Combine this method with others to find synergy."
                },
                "weights": {"EI": 0, "KB": 0, "RM": 2, "AD": 2, "AS": 0},
                "archetype": {
                    "uk": "Арбітражний стабілізатор",
                    "ru": "Арбитражный стабилизатор",
                    "en": "Arbitrage Stabilizer"
                },
                "explanation": {
                    "uk": "Диверсифікація аналітики.",
                    "ru": "Диверсификация аналитики.",
                    "en": "Analytics diversification."
                }
            },
            {
                "text": {
                    "uk": "Використовую метод для протилежних сигналів (контртренд).",
                    "ru": "Использую метод для противоположных сигналов (контртренд).",
                    "en": "Use the method for opposite signals (counter-trend)."
                },
                "weights": {"EI": 0, "KB": 0, "RM": 0, "AD": 3, "AS": 0},
                "archetype": {
                    "uk": "Провокатор хаосу",
                    "ru": "Провокатор хаоса",
                    "en": "Chaos Provocateur"
                },
                "explanation": {
                    "uk": "Інверсія стратегії.",
                    "ru": "Инверсия стратегии.",
                    "en": "Strategy inversion."
                }
            },
            {
                "text": {
                    "uk": "Тестую метод на нових ринках або інструментах.",
                    "ru": "Тестирую метод на новых рынках или инструментах.",
                    "en": "Test the method on new markets or instruments."
                },
                "weights": {"EI": 0, "KB": 0, "RM": 0, "AD": 3, "AS": 0},
                "archetype": {
                    "uk": "Емерджентний аналітик",
                    "ru": "Эмерджентный аналитик",
                    "en": "Emergent Analyst"
                },
                "explanation": {
                    "uk": "Експериментування з новими даними.",
                    "ru": "Экспериментирование с новыми данными.",
                    "en": "Experimentation with new data."
                }
            },
            {
                "text": {
                    "uk": "Обговорюю проблему в професійній спільноті, шукаю колективний досвід.",
                    "ru": "Обсуждаю проблему в профессиональном сообществе, ищу коллективный опыт.",
                    "en": "Discuss the issue in professional communities seeking collective wisdom."
                },
                "weights": {"EI": 0, "KB": 0, "RM": 3, "AD": 3, "AS": 0},
                "archetype": {
                    "uk": "Антифрагільний агент",
                    "ru": "Антихрупкий агент",
                    "en": "Antifragile Agent"
                },
                "explanation": {
                    "uk": "Соціальна адаптація.",
                    "ru": "Социальная адаптация.",
                    "en": "Social adaptation."
                }
            },
            {
                "text": {
                    "uk": "Питаю думку у визнаних експертів галузі.",
                    "ru": "Спрашиваю мнение у признанных экспертов отрасли.",
                    "en": "Seek opinions from recognized industry experts."
                },
                "weights": {"EI": 0, "KB": 0, "RM": 0, "AD": 0, "AS": 3},
                "archetype": {
                    "uk": "Трендсеттер",
                    "ru": "Трендсеттер",
                    "en": "Trendsetter"
                },
                "explanation": {
                    "uk": "Орієнтація на авторитети.",
                    "ru": "Ориентация на авторитеты.",
                    "en": "Authority orientation."
                }
            },
            {
                "text": {
                    "uk": "Довіряю інтуїції, вирішуючи, чи варто продовжувати.",
                    "ru": "Доверяю интуиции, решая, стоит ли продолжать.",
                    "en": "Trust intuition to decide whether to continue."
                },
                "weights": {"EI": 2, "KB": 0, "RM": 0, "AD": 0, "AS": 0},
                "archetype": {
                    "uk": "Медіатор Духів",
                    "ru": "Медиатор Духов",
                    "en": "Spirit Mediator"
                },
                "explanation": {
                    "uk": "Інтуїтивне прийняття рішень.",
                    "ru": "Интуитивное принятие решений.",
                    "en": "Intuitive decision-making."
                }
            }
        ]
    },

      {
        "question": {
            "uk": "Світова регуляторна криза: Китай забороняє трейдинг, США вводять 60% податок на крипту, ЄС запускає CBDC. Яка ваша стратегія?",
            "ru": "Глобальный регуляторный кризис: Китай запрещает трейдинг, США вводят 60% налог на крипту, ЕС запускает CBDC. Какая ваша стратегия?",
            "en": "Global regulatory crisis: China bans trading, US imposes 60% crypto tax, EU launches CBDC. What's your strategy?"
        },
        "answers": [
            {
                "text": {
                    "uk": "Швидко переїжджаю до країни з мінімальними обмеженнями для трейдерів.",
                    "ru": "Быстро переезжаю в страну с минимальными ограничениями для трейдеров.",
                    "en": "Quickly relocate to a jurisdiction with minimal trader restrictions."
                },
                "weights": {"EI": 0, "KB": 0, "RM": 0, "AD": 0, "AS": 0},
                "archetype": {
                    "uk": "Маржинальний жнець",
                    "ru": "Маржинальный жнец",
                    "en": "Margin Reaper"
                },
                "explanation": {
                    "uk": "Агресивна мобільність заради переваги.",
                    "ru": "Агрессивная мобильность ради преимущества.",
                    "en": "Aggressive mobility for advantage."
                }
            },
            {
                "text": {
                    "uk": "Оперативно адаптую свою стратегію до нових регуляторних вимог.",
                    "ru": "Оперативно адаптирую стратегию под новые регуляторные требования.",
                    "en": "Swiftly adapt strategy to new regulatory requirements."
                },
                "weights": {"EI": 0, "KB": 0, "RM": 3, "AD": 0, "AS": 0},
                "archetype": {
                    "uk": "Рептилія",
                    "ru": "Рептилия",
                    "en": "Reptile"
                },
                "explanation": {
                    "uk": "Системна гнучкість у межах правил.",
                    "ru": "Системная гибкость в рамках правил.",
                    "en": "Systematic flexibility within rules."
                }
            },
            {
                "text": {
                    "uk": "Повністю переходжу на DeFi та децентралізовані біржі.",
                    "ru": "Полностью перехожу на DeFi и децентрализованные биржи.",
                    "en": "Fully transition to DeFi and decentralized exchanges."
                },
                "weights": {"EI": 0, "KB": 0, "RM": 2, "AD": 3, "AS": 0},
                "archetype": {
                    "uk": "Арбітражний стабілізатор",
                    "ru": "Арбитражный стабилизатор",
                    "en": "Arbitrage Stabilizer"
                },
                "explanation": {
                    "uk": "Ухилення через інфраструктурну зміну.",
                    "ru": "Уклонение через инфраструктурные изменения.",
                    "en": "Evasion through infrastructure shift."
                }
            },
            {
                "text": {
                    "uk": "Використовую різницю між юрисдикціями для арбітражу.",
                    "ru": "Использую различия между юрисдикциями для арбитража.",
                    "en": "Exploit jurisdictional differences for arbitrage."
                },
                "weights": {"EI": 0, "KB": 0, "RM": 0, "AD": 3, "AS": 0},
                "archetype": {
                    "uk": "Провокатор хаосу",
                    "ru": "Провокатор хаоса",
                    "en": "Chaos Provocateur"
                },
                "explanation": {
                    "uk": "Експлуатація глобального дисбалансу.",
                    "ru": "Эксплуатация глобального дисбаланса.",
                    "en": "Exploiting global imbalance."
                }
            },
            {
                "text": {
                    "uk": "Використовую ШІ для автоматичної оптимізації під нові правила.",
                    "ru": "Использую ИИ для автоматической оптимизации под новые правила.",
                    "en": "Use AI for automatic optimization to new regulations."
                },
                "weights": {"EI": 0, "KB": 0, "RM": 3, "AD": 3, "AS": 0},
                "archetype": {
                    "uk": "Емерджентний аналітик",
                    "ru": "Эмерджентный аналитик",
                    "en": "Emergent Analyst"
                },
                "explanation": {
                    "uk": "Технологічна адаптація.",
                    "ru": "Технологическая адаптация.",
                    "en": "Technological adaptation."
                }
            },
            {
                "text": {
                    "uk": "Об'єднуюсь із трейдерами для створення міжнародної захисної спільноти.",
                    "ru": "Объединяюсь с трейдерами для создания международного защитного сообщества.",
                    "en": "Unite with traders to form an international protective community."
                },
                "weights": {"EI": 0, "KB": 0, "RM": 3, "AD": 3, "AS": 0},
                "archetype": {
                    "uk": "Антифрагільний агент",
                    "ru": "Антихрупкий агент",
                    "en": "Antifragile Agent"
                },
                "explanation": {
                    "uk": "Колективна стійкість.",
                    "ru": "Коллективная устойчивость.",
                    "en": "Collective resilience."
                }
            },
            {
                "text": {
                    "uk": "Лобіюю інтереси через професійні організації та медіа.",
                    "ru": "Лоббирую интересы через профессиональные организации и медиа.",
                    "en": "Lobby interests through professional organizations and media."
                },
                "weights": {"EI": 0, "KB": 0, "RM": 0, "AD": 0, "AS": 3},
                "archetype": {
                    "uk": "Трендсеттер",
                    "ru": "Трендсеттер",
                    "en": "Trendsetter"
                },
                "explanation": {
                    "uk": "Вплив на регуляторне поле через комунікацію.",
                    "ru": "Влияние на регуляторное поле через коммуникацию.",
                    "en": "Influencing regulation through communication."
                }
            },
            {
                "text": {
                    "uk": "Роблю паузу, спостерігаю за розвитком подій, щоб не поспішати з рішенням.",
                    "ru": "Делаю паузу, наблюдаю за развитием событий, чтобы не спешить с решением.",
                    "en": "Pause to observe developments before rushing into decisions."
                },
                "weights": {"EI": 2, "KB": 0, "RM": 0, "AD": 0, "AS": 0},
                "archetype": {
                    "uk": "Медіатор Духів",
                    "ru": "Медиатор Духов",
                    "en": "Spirit Mediator"
                },
                "explanation": {
                    "uk": "Стратегічне очікування та інтроспекція.",
                    "ru": "Стратегическое ожидание и интроспекция.",
                    "en": "Strategic waiting and introspection."
                }
            }
        ]
    },

      {
        "question": {
            "uk": "Ваш портфель у мінусі 60%. Ви отримали пропозицію від інвестора: 'Я дам вам $10k, якщо ви ризикнете всім і зробите x2 за тиждень, або борг'. Ваші дії:",
            "ru": "Ваш портфель в минусе 60%. Вы получили предложение от инвестора: 'Я дам вам $10k, если вы рискнете всем и сделаете x2 за неделю, или долг'. Ваши действия:",
            "en": "Your portfolio is down 60%. You received an offer from an investor: 'I'll give you $10k if you risk everything and double it in a week, or debt'. Your actions:"
        },
        "answers": [
            {
                "text": {
                    "uk": "Ризикую всіма коштами на найбільш волатильний актив.",
                    "ru": "Рискую всеми средствами на самый волатильный актив.",
                    "en": "Risk all funds on the most volatile asset."
                },
                "weights": {"EI": 0, "KB": 0, "RM": 0, "AD": 0, "AS": 0},
                "archetype": {
                    "uk": "Маржинальний жнець",
                    "ru": "Маржинальный жнец",
                    "en": "Margin Reaper"
                },
                "explanation": {
                    "uk": "Агресивний азарт.",
                    "ru": "Агрессивный азарт.",
                    "en": "Aggressive gambling."
                }
            },
            {
                "text": {
                    "uk": "Відмовляюсь, дотримуючись власних правил ризик-менеджменту.",
                    "ru": "Отказываюсь, придерживаясь собственных правил риск-менеджмента.",
                    "en": "Refuse, adhering to my own risk management rules."
                },
                "weights": {"EI": 0, "KB": 0, "RM": 3, "AD": 0, "AS": 0},
                "archetype": {
                    "uk": "Рептилія",
                    "ru": "Рептилия",
                    "en": "Reptile"
                },
                "explanation": {
                    "uk": "Ригідна дисципліна.",
                    "ru": "Жесткая дисциплина.",
                    "en": "Rigid discipline."
                }
            },
            {
                "text": {
                    "uk": "Використовую арбітраж між біржами для гарантованого прибутку.",
                    "ru": "Использую арбитраж между биржами для гарантированной прибыли.",
                    "en": "Use arbitrage between exchanges for guaranteed profit."
                },
                "weights": {"EI": 0, "KB": 0, "RM": 2, "AD": 2, "AS": 0},
                "archetype": {
                    "uk": "Арбітражний стабілізатор",
                    "ru": "Арбитражный стабилизатор",
                    "en": "Arbitrage Stabilizer"
                },
                "explanation": {
                    "uk": "Системне уникнення ризику.",
                    "ru": "Системное избегание риска.",
                    "en": "Systematic risk avoidance."
                }
            },
            {
                "text": {
                    "uk": "Публікую опитування в соцмережах, щоб інші вирішили за мене.",
                    "ru": "Публикую опросы в соцсетях, чтобы другие решили за меня.",
                    "en": "Publish polls on social media to let others decide for me."
                },
                "weights": {"EI": 0, "KB": 0, "RM": 0, "AD": 3, "AS": 0},
                "archetype": {
                    "uk": "Провокатор хаосу",
                    "ru": "Провокатор хаоса",
                    "en": "Chaos Provocateur"
                },
                "explanation": {
                    "uk": "Екстерналізація відповідальності.",
                    "ru": "Экстернализация ответственности.",
                    "en": "Externalization of responsibility."
                }
            },
            {
                "text": {
                    "uk": "Аналізую тисячі сценаріїв у ML-моделі для оптимального рішення.",
                    "ru": "Анализирую тысячи сценариев в ML-модели для оптимального решения.",
                    "en": "Analyze thousands of scenarios in an ML model for optimal decision."
                },
                "weights": {"EI": 0, "KB": 0, "RM": 0, "AD": 3, "AS": 0},
                "archetype": {
                    "uk": "Емерджентний аналітик",
                    "ru": "Эмерджентный аналитик",
                    "en": "Emergent Analyst"
                },
                "explanation": {
                    "uk": "Технологічний підхід.",
                    "ru": "Технологический подход.",
                    "en": "Technological approach."
                }
            },
            {
                "text": {
                    "uk": "Пропоную інвестору розділити ризик порівну.",
                    "ru": "Предлагаю инвестору разделить риск поровну.",
                    "en": "Propose to the investor to share risk equally."
                },
                "weights": {"EI": 0, "KB": 0, "RM": 3, "AD": 3, "AS": 0},
                "archetype": {
                    "uk": "Антифрагільний агент",
                    "ru": "Антихрупкий агент",
                    "en": "Antifragile Agent"
                },
                "explanation": {
                    "uk": "Колективне зменшення втрат.",
                    "ru": "Коллективное уменьшение потерь.",
                    "en": "Collective loss reduction."
                }
            },
            {
                "text": {
                    "uk": "Шукаю лазівки в умовах угоди для мінімізації збитків.",
                    "ru": "Ищу лазейки в условиях сделки для минимизации убытков.",
                    "en": "Look for loopholes in the deal to minimize losses."
                },
                "weights": {"EI": 0, "KB": 0, "RM": 0, "AD": 0, "AS": 3},
                "archetype": {
                    "uk": "Трендсеттер",
                    "ru": "Трендсеттер",
                    "en": "Trendsetter"
                },
                "explanation": {
                    "uk": "Маніпуляція правилами.",
                    "ru": "Манипуляция правилами.",
                    "en": "Manipulation of rules."
                }
            },
            {
                "text": {
                    "uk": "Роблю паузу, щоб відчути, чи варто приймати пропозицію.",
                    "ru": "Делаю паузу, чтобы почувствовать, стоит ли принимать предложение.",
                    "en": "Take a pause to feel whether to accept the offer."
                },
                "weights": {"EI": 2, "KB": 0, "RM": 0, "AD": 0, "AS": 0},
                "archetype": {
                    "uk": "Медіатор Духів",
                    "ru": "Медиатор Духов",
                    "en": "Spirit Mediator"
                },
                "explanation": {
                    "uk": "Інтуїтивна оцінка.",
                    "ru": "Интуитивная оценка.",
                    "en": "Intuitive assessment."
                }
            }
        ]
    },

       {
        "question": {
            "uk": "Індекс страху VIX = 85. Всі активи корелюють -90%. Готівки немає. Ваші дії?",
            "ru": "Индекс страха VIX = 85. Все активы коррелируют -90%. Налички нет. Ваши действия?",
            "en": "Fear index VIX = 85. All assets correlate -90%. No cash. Your actions?"
        },
        "answers": [
            {
                "text": {
                    "uk": "Беру максимальний кредит, щоб викупити 'дно'.",
                    "ru": "Беру максимальный кредит, чтобы выкупить 'дно'.",
                    "en": "Take maximum leverage to buy the dip."
                },
                "weights": {"EI": 0, "KB": 0, "RM": 0, "AD": 0, "AS": 0},
                "archetype": {
                    "uk": "Маржинальний жнець",
                    "ru": "Маржинальный жнец",
                    "en": "Margin Reaper"
                },
                "explanation": {
                    "uk": "Агресивна гра на відновлення.",
                    "ru": "Агрессивная игра на восстановление.",
                    "en": "Aggressive recovery play."
                }
            },
            {
                "text": {
                    "uk": "Продаю всі активи, переводячи капітал у стейблкоїни.",
                    "ru": "Продаю все активы, переводя капитал в стейблкоины.",
                    "en": "Sell all assets, converting capital to stablecoins."
                },
                "weights": {"EI": 0, "KB": 0, "RM": 3, "AD": 0, "AS": 0},
                "archetype": {
                    "uk": "Рептилія",
                    "ru": "Рептилия",
                    "en": "Reptile"
                },
                "explanation": {
                    "uk": "Ригідна стратегія збереження.",
                    "ru": "Жесткая стратегия сохранения.",
                    "en": "Rigid preservation strategy."
                }
            },
            {
                "text": {
                    "uk": "Шукаю арбітражні можливості між некорельованими активами.",
                    "ru": "Ищу арбитражные возможности между некоррелированными активами.",
                    "en": "Seek arbitrage opportunities between uncorrelated assets."
                },
                "weights": {"EI": 0, "KB": 0, "RM": 3, "AD": 3, "AS": 0},
                "archetype": {
                    "uk": "Арбітражний стабілізатор",
                    "ru": "Арбитражный стабилизатор",
                    "en": "Arbitrage Stabilizer"
                },
                "explanation": {
                    "uk": "Раціональний пошук лазівок.",
                    "ru": "Рациональный поиск лазеек.",
                    "en": "Rational loophole hunting."
                }
            },
            {
                "text": {
                    "uk": "Запускаю платний курс 'Як заробити на кризі'.",
                    "ru": "Запускаю платный курс 'Как заработать на кризисе'.",
                    "en": "Launch a paid course 'How to Profit from the Crisis'."
                },
                "weights": {"EI": 0, "KB": 0, "RM": 0, "AD": 0, "AS": 0},
                "archetype": {
                    "uk": "Провокатор хаосу",
                    "ru": "Провокатор хаоса",
                    "en": "Chaos Provocateur"
                },
                "explanation": {
                    "uk": "Маніпуляція панікою.",
                    "ru": "Манипуляция паникой.",
                    "en": "Panic manipulation."
                }
            },
            {
                "text": {
                    "uk": "Використовую AI для аналізу історичних даних подібних криз.",
                    "ru": "Использую AI для анализа исторических данных подобных кризисов.",
                    "en": "Use AI to analyze historical data of similar crises."
                },
                "weights": {"EI": 0, "KB": 0, "RM": 0, "AD": 3, "AS": 0},
                "archetype": {
                    "uk": "Емерджентний аналітик",
                    "ru": "Эмерджентный аналитик",
                    "en": "Emergent Analyst"
                },
                "explanation": {
                    "uk": "Технологічна адаптація.",
                    "ru": "Технологическая адаптация.",
                    "en": "Technological adaptation."
                }
            },
            {
                "text": {
                    "uk": "Координую свої дії з міжнародною спільнотою трейдерів.",
                    "ru": "Координирую свои действия с международным сообществом трейдеров.",
                    "en": "Coordinate actions with the international trader community."
                },
                "weights": {"EI": 0, "KB": 0, "RM": 3, "AD": 3, "AS": 0},
                "archetype": {
                    "uk": "Антифрагільний агент",
                    "ru": "Антихрупкий агент",
                    "en": "Antifragile Agent"
                },
                "explanation": {
                    "uk": "Колективна стійкість.",
                    "ru": "Коллективная устойчивость.",
                    "en": "Collective resilience."
                }
            },
            {
                "text": {
                    "uk": "Переводжу активи в золото та облігації.",
                    "ru": "Перевожу активы в золото и облигации.",
                    "en": "Convert assets to gold and bonds."
                },
                "weights": {"EI": 0, "KB": 0, "RM": 2, "AD": 0, "AS": 0},
                "archetype": {
                    "uk": "Трендсеттер",
                    "ru": "Трендсеттер",
                    "en": "Trendsetter"
                },
                "explanation": {
                    "uk": "Консервативна адаптація.",
                    "ru": "Консервативная адаптация.",
                    "en": "Conservative adaptation."
                }
            },
            {
                "text": {
                    "uk": "Відключаю термінал і зосереджуюсь на духовних практиках.",
                    "ru": "Отключаю терминал и сосредотачиваюсь на духовных практиках.",
                    "en": "Turn off the terminal and focus on spiritual practices."
                },
                "weights": {"EI": 2, "KB": 0, "RM": 0, "AD": 0, "AS": 0},
                "archetype": {
                    "uk": "Медіатор Духів",
                    "ru": "Медиатор Духов",
                    "en": "Spirit Mediator"
                },
                "explanation": {
                    "uk": "Інтуїтивне відсторонення.",
                    "ru": "Интуитивное отстранение.",
                    "en": "Intuitive detachment."
                }
            }
        ]
    },

      {
        "question": {
            "uk": "SEC анонсували 'нові правила' для DeFi за місяць. Деталей немає. 40% портфеля в DeFi:",
            "ru": "SEC анонсировали 'новые правила' для DeFi через месяц. Деталей нет. 40% портфеля в DeFi:",
            "en": "SEC announced 'new rules' for DeFi in a month. No details yet. 40% of portfolio in DeFi:"
        },
        "answers": [
            {
                "text": {
                    "uk": "Негайно продаю всі De-активи, уникаючи ризику регуляторних санкцій.",
                    "ru": "Немедленно продаю все De-активы, избегая риска регуляторных санкций.",
                    "en": "Immediately sell all De-assets to avoid regulatory sanction risks."
                },
                "weights": {"EI": 0, "KB": 0, "RM": 0, "AD": 2, "AS": 3},
                "archetype": {
                    "uk": "Маржинальний жнець",
                    "ru": "Маржинальный жнец",
                    "en": "Margin Reaper"
                },
                "explanation": {
                    "uk": "Агресивне уникнення збитків через регуляторні ризики",
                    "ru": "Агрессивное избегание убытков из-за регуляторных рисков",
                    "en": "Aggressive loss avoidance due to regulatory risks"
                }
            },
            {
                "text": {
                    "uk": "Залишаю портфель без змін, доки не з’являться офіційні деталі.",
                    "ru": "Оставляю портфель без изменений, пока не появятся официальные детали.",
                    "en": "Keep portfolio unchanged until official details appear."
                },
                "weights": {"EI": 2, "KB": 1, "RM": 3, "AD": 0, "AS": 2},
                "archetype": {
                    "uk": "Рептилія",
                    "ru": "Рептилия",
                    "en": "Reptile"
                },
                "explanation": {
                    "uk": "Ригідна дисципліна та контроль ризиків",
                    "ru": "Жесткая дисциплина и контроль рисков",
                    "en": "Rigid discipline and risk control"
                }
            },
            {
                "text": {
                    "uk": "Хеджую ризики через традиційні фінансові інструменти (напр., ф’ючерси).",
                    "ru": "Хеджирую риски через традиционные финансовые инструменты (напр., фьючерсы).",
                    "en": "Hedge risks through traditional financial instruments (e.g., futures)."
                },
                "weights": {"EI": 2, "KB": 1, "RM": 2, "AD": 3, "AS": 0},
                "archetype": {
                    "uk": "Арбітражний стабілізатор",
                    "ru": "Арбитражный стабилизатор",
                    "en": "Arbitrage Stabilizer"
                },
                "explanation": {
                    "uk": "Системне зменшення експозиції через диверсифікацію",
                    "ru": "Системное уменьшение экспозиции через диверсификацию",
                    "en": "Systematic exposure reduction through diversification"
                }
            },
            {
                "text": {
                    "uk": "Створюю панічні продажі в соцмережах, щоб купити активи дешевше.",
                    "ru": "Создаю панические продажи в соцсетях, чтобы купить активы дешевле.",
                    "en": "Create panic sales on social media to buy assets cheaper."
                },
                "weights": {"EI": 0, "KB": 0, "RM": 0, "AD": 3, "AS": 2},
                "archetype": {
                    "uk": "Провокатор хаосу",
                    "ru": "Провокатор хаоса",
                    "en": "Chaos Provocateur"
                },
                "explanation": {
                    "uk": "Маніпуляція ринковою панікою для власної вигоди",
                    "ru": "Манипуляция рыночной паникой для собственной выгоды",
                    "en": "Manipulation of market panic for personal gain"
                }
            },
            {
                "text": {
                    "uk": "Аналізую історичні кейси регуляторних змін за допомогою AI.",
                    "ru": "Анализирую исторические кейсы регуляторных изменений с помощью AI.",
                    "en": "Analyze historical cases of regulatory changes using AI."
                },
                "weights": {"EI": 1, "KB": 2, "RM": 2, "AD": 3, "AS": 1},
                "archetype": {
                    "uk": "Емерджентний аналітик",
                    "ru": "Эмерджентный аналитик",
                    "en": "Emergent Analyst"
                },
                "explanation": {
                    "uk": "Технологічний прогноз та адаптивний підхід",
                    "ru": "Технологический прогноз и адаптивный подход",
                    "en": "Technological forecasting and adaptive approach"
                }
            },
            {
                "text": {
                    "uk": "Збільшую частку DeFi, вважаючи регулювання знаком майбутнього масового прийняття.",
                    "ru": "Увеличиваю долю DeFi, считая регулирование знаком будущего массового принятия.",
                    "en": "Increase DeFi share, seeing regulation as a sign of future mass adoption."
                },
                "weights": {"EI": 3, "KB": 1, "RM": 2, "AD": 3, "AS": 2},
                "archetype": {
                    "uk": "Антифрагільний агент",
                    "ru": "Антихрупкий агент",
                    "en": "Antifragile Agent"
                },
                "explanation": {
                    "uk": "Асиметрична ставка на легалізацію технологій",
                    "ru": "Асимметричная ставка на легализацию технологий",
                    "en": "Asymmetric bet on technology legalization"
                }
            },
            {
                "text": {
                    "uk": "Приєднуюсь до колективних лобістських зусиль галузі.",
                    "ru": "Присоединяюсь к коллективным лоббистским усилиям отрасли.",
                    "en": "Join collective lobbying efforts of the industry."
                },
                "weights": {"EI": 2, "KB": 1, "RM": 1, "AD": 2, "AS": 3},
                "archetype": {
                    "uk": "Трендсеттер",
                    "ru": "Трендсеттер",
                    "en": "Trendsetter"
                },
                "explanation": {
                    "uk": "Активний вплив на регуляторний процес",
                    "ru": "Активное влияние на регуляторный процесс",
                    "en": "Active influence on the regulatory process"
                }
            },
            {
                "text": {
                    "uk": "Роблю паузу, сприймаючи невизначеність як частину ринкового циклу.",
                    "ru": "Делаю паузу, воспринимая неопределенность как часть рыночного цикла.",
                    "en": "Pause, perceiving uncertainty as part of the market cycle."
                },
                "weights": {"EI": 3, "KB": 0, "RM": 2, "AD": 2, "AS": 1},
                "archetype": {
                    "uk": "Медіатор Духів",
                    "ru": "Медиатор Духов",
                    "en": "Spirit Mediator"
                },
                "explanation": {
                    "uk": "Інтуїтивна адаптація до ринкових умов",
                    "ru": "Интуитивная адаптация к рыночным условиям",
                    "en": "Intuitive adaptation to market conditions"
                }
            }
        ]
    },

       {
        "question": {
            "uk": "Чи завжди ви ретельно досліджуєте всі активи перед інвестуванням?",
            "ru": "Всегда ли вы тщательно исследуете активы перед инвестированием?",
            "en": "Do you always thoroughly research assets before investing?"
        },
        "answers": [
            {
                "text": {
                    "uk": "Завжди проводжу повний фундаментальний та технічний аналіз.",
                    "ru": "Всегда провожу полный фундаментальный и технический анализ.",
                    "en": "Always conduct full fundamental and technical analysis."
                },
                "weights": {"EI": 0, "KB": 0, "RM": 3, "AD": 3, "AS": 0},
                "archetype": {
                    "uk": "Рептилія/Арбітражний стабілізатор",
                    "ru": "Рептилия/Арбитражный стабилизатор",
                    "en": "Reptile/Arbitrage Stabilizer"
                },
                "explanation": {
                    "uk": "Системність.",
                    "ru": "Системность.",
                    "en": "Systematic approach."
                }
            },
            {
                "text": {
                    "uk": "Частково аналізую, частково довіряю внутрішнім інсайтам.",
                    "ru": "Частично анализирую, частично доверяю внутренним инсайтам.",
                    "en": "Partially analyze, partially trust internal insights."
                },
                "weights": {"EI": 2, "KB": 0, "RM": 0, "AD": 0, "AS": 0},
                "archetype": {
                    "uk": "Медіатор Духів",
                    "ru": "Медиатор Духов",
                    "en": "Spirit Mediator"
                },
                "explanation": {
                    "uk": "Інтуїтивний баланс.",
                    "ru": "Интуитивный баланс.",
                    "en": "Intuitive balance."
                }
            },
            {
                "text": {
                    "uk": "Використовую AI-інструменти для автоматизованої оцінки.",
                    "ru": "Использую AI-инструменты для автоматизированной оценки.",
                    "en": "Use AI tools for automated evaluation."
                },
                "weights": {"EI": 0, "KB": 0, "RM": 0, "AD": 3, "AS": 0},
                "archetype": {
                    "uk": "Емерджентний аналітик",
                    "ru": "Эмерджентный аналитик",
                    "en": "Emergent Analyst"
                },
                "explanation": {
                    "uk": "Технологічна ефективність.",
                    "ru": "Технологическая эффективность.",
                    "en": "Technological efficiency."
                }
            },
            {
                "text": {
                    "uk": "Довіряю думці експертів або соціальній думці.",
                    "ru": "Доверяю мнению экспертов или социальному мнению.",
                    "en": "Trust expert opinions or social sentiment."
                },
                "weights": {"EI": 0, "KB": 0, "RM": 0, "AD": 0, "AS": 3},
                "archetype": {
                    "uk": "Трендсеттер",
                    "ru": "Трендсеттер",
                    "en": "Trendsetter"
                },
                "explanation": {
                    "uk": "Орієнтація на авторитети.",
                    "ru": "Ориентация на авторитеты.",
                    "en": "Authority orientation."
                }
            },
            {
                "text": {
                    "uk": "Дію швидко, спираючись на тренди без глибокого аналізу.",
                    "ru": "Действую быстро, опираясь на тренды без глубокого анализа.",
                    "en": "Act quickly based on trends without deep analysis."
                },
                "weights": {"EI": 0, "KB": 0, "RM": 0, "AD": 0, "AS": 0},
                "archetype": {
                    "uk": "Маржинальний жнець/Провокатор хаосу",
                    "ru": "Маржинальный жнец/Провокатор хаоса",
                    "en": "Margin Reaper/Chaos Provocateur"
                },
                "explanation": {
                    "uk": "Імпульсивність.",
                    "ru": "Импульсивность.",
                    "en": "Impulsiveness."
                }
            },
            {
                "text": {
                    "uk": "Аналізую лише великі угоди, дрібні — інтуїтивно.",
                    "ru": "Анализирую только крупные сделки, мелкие — интуитивно.",
                    "en": "Analyze only large deals, small ones — intuitively."
                },
                "weights": {"EI": 0, "KB": 0, "RM": 3, "AD": 3, "AS": 0},
                "archetype": {
                    "uk": "Антифрагільний агент",
                    "ru": "Антихрупкий агент",
                    "en": "Antifragile Agent"
                },
                "explanation": {
                    "uk": "Селективний підхід.",
                    "ru": "Селективный подход.",
                    "en": "Selective approach."
                }
            }
        ]
    },

      {
        "question": {
            "uk": "ШІ починає торгувати краще за 95% людей. Через 3 роки people-трейдери можуть стати непотрібними. Ваша стратегія виживання:",
            "ru": "ИИ начинает торговать лучше 95% людей. Через 3 года люди-трейдеры могут стать ненужными. Ваша стратегия выживания:",
            "en": "AI starts trading better than 95% of humans. In 3 years, human traders may become obsolete. Your survival strategy:"
        },
        "answers": [
            {
                "text": {
                    "uk": "Спеціалізуюсь на ринках, де можна експлуатувати слабкості та прогнозованість торгових ШІ.",
                    "ru": "Специализируюсь на рынках, где можно эксплуатировать слабости и предсказуемость торговых ИИ.",
                    "en": "Specialize in markets where AI trading weaknesses/predictability can be exploited."
                },
                "weights": {"EI": 0, "KB": 0, "RM": 0, "AD": 0, "AS": 0},
                "archetype": {
                    "uk": "Маржинальний жнець",
                    "ru": "Маржинальный жнец",
                    "en": "Margin Reaper"
                },
                "explanation": {
                    "uk": "Пошук нових вразливостей для полювання.",
                    "ru": "Поиск новых уязвимостей для охоты.",
                    "en": "Hunting for new AI vulnerabilities."
                }
            },
            {
                "text": {
                    "uk": "Повністю виходжу з трейдингу, оскільки моя перевірена система більше не працює.",
                    "ru": "Полностью ухожу из трейдинга, так как моя проверенная система больше не работает.",
                    "en": "Exit trading completely as my proven system becomes obsolete."
                },
                "weights": {"EI": 0, "KB": 0, "RM": 3, "AD": 0, "AS": 0},
                "archetype": {
                    "uk": "Рептилія",
                    "ru": "Рептилия",
                    "en": "Reptile"
                },
                "explanation": {
                    "uk": "Ригідна реакція на повну зміну парадигми.",
                    "ru": "Жесткая реакция на полную смену парадигмы.",
                    "en": "Rigid response to paradigm shift."
                }
            },
            {
                "text": {
                    "uk": "Переходжу в управління великим капіталом, де потрібен людський нагляд та комунікація з клієнтами.",
                    "ru": "Перехожу в управление крупным капиталом, где требуется человеческий надзор и коммуникация с клиентами.",
                    "en": "Move to capital management requiring human oversight/client communication."
                },
                "weights": {"EI": 0, "KB": 0, "RM": 3, "AD": 2, "AS": 0},
                "archetype": {
                    "uk": "Арбітражний стабілізатор",
                    "ru": "Арбитражный стабилизатор",
                    "en": "Arbitrage Stabilizer"
                },
                "explanation": {
                    "uk": "Пошук нової стабільної, системної ролі.",
                    "ru": "Поиск новой стабильной системной роли.",
                    "en": "Seeking stable systemic roles."
                }
            },
            {
                "text": {
                    "uk": "Зосереджуюсь на активах з максимальною непередбачуваністю, де ШІ дає збої.",
                    "ru": "Фокусируюсь на активах с максимальной непредсказуемостью, где ИИ дает сбои.",
                    "en": "Focus on maximally unpredictable assets where AI fails."
                },
                "weights": {"EI": 0, "KB": 0, "RM": 0, "AD": 3, "AS": 0},
                "archetype": {
                    "uk": "Провокатор хаосу",
                    "ru": "Провокатор хаоса",
                    "en": "Chaos Provocateur"
                },
                "explanation": {
                    "uk": "Пошук ніші, де хаос залишається ключовим фактором.",
                    "ru": "Поиск ниши, где хаос остается ключевым фактором.",
                    "en": "Chaos remains key in this niche."
                }
            },
            {
                "text": {
                    "uk": "Створюю власний ШІ-фонд, розробляючи та вдосконалюючи власні пропрієтарні алгоритми.",
                    "ru": "Создаю собственный ИИ-фонд, разрабатывая и улучшая проприетарные алгоритмы.",
                    "en": "Create my own AI fund with proprietary algorithms."
                },
                "weights": {"EI": 0, "KB": 0, "RM": 0, "AD": 3, "AS": 0},
                "archetype": {
                    "uk": "Емерджентний аналітик",
                    "ru": "Эмерджентный аналитик",
                    "en": "Emergent Analyst"
                },
                "explanation": {
                    "uk": "Еволюція від користувача до творця технологій.",
                    "ru": "Эволюция от пользователя к создателю технологий.",
                    "en": "Evolving from user to tech creator."
                }
            },
            {
                "text": {
                    "uk": "Створюю гібридні команди (сильні сторони людей + ШІ)",
                    "ru": "Создаю гибридные команды (сильные стороны людей + ИИ)",
                    "en": "Build hybrid teams (human strengths + AI)."
                },
                "weights": {"EI": 0, "KB": 0, "RM": 3, "AD": 3, "AS": 0},
                "archetype": {
                    "uk": "Антифрагільний агент",
                    "ru": "Антихрупкий агент",
                    "en": "Antifragile Agent"
                },
                "explanation": {
                    "uk": "Використання стресу системи для створення нової, сильнішої структури.",
                    "ru": "Использование стресса системы для создания новой, более сильной структуры.",
                    "en": "Leveraging systemic stress to build resilience."
                }
            },
            {
                "text": {
                    "uk": "Очолюю рух 'людських трейдерів', створюючи освітній контент та спільноту для збереження 'людської переваги'.",
                    "ru": "Возглавляю движение 'человеческих трейдеров', создавая образовательный контент и сообщество.",
                    "en": "Lead 'human traders' movement with educational content/community."
                },
                "weights": {"EI": 0, "KB": 0, "RM": 0, "AD": 0, "AS": 3},
                "archetype": {
                    "uk": "Трендсеттер",
                    "ru": "Трендсеттер",
                    "en": "Trendsetter"
                },
                "explanation": {
                    "uk": "Створення наративу та лідерство у новій ніші.",
                    "ru": "Создание нарратива и лидерство в новой нише.",
                    "en": "Narrative creation and niche leadership."
                }
            },
            {
                "text": {
                    "uk": "Спеціалізуюсь на ESG-інвестуванні та інших сферах, де ключову роль відіграють людські цінності, а не чиста математика.",
                    "ru": "Специализируюсь на ESG-инвестировании и других сферах с человеческими ценностями.",
                    "en": "Specialize in ESG investing and other human-value-driven fields."
                },
                "weights": {"EI": 2, "KB": 0, "RM": 0, "AD": 0, "AS": 0},
                "archetype": {
                    "uk": "Медіатор Духів",
                    "ru": "Медиатор Духов",
                    "en": "Spirit Mediator"
                },
                "explanation": {
                    "uk": "Перехід у нішу, що керується етикою та інтуїцією.",
                    "ru": "Переход в нишу, управляемую этикой и интуицией.",
                    "en": "Ethics and intuition-driven niche transition."
                }
            }
        ]
    }
]


# Функції для підрахунку
def calculate_scores(user_answers, questions):
    scores = {"EI": 0, "KB": 0, "RM": 0, "AD": 0, "AS": 0}
    
    for i, answer_index in enumerate(user_answers):
        if i < len(questions) and answer_index < len(questions[i]["answers"]):
            weights = questions[i]["answers"][answer_index]["weights"]
            for category, points in weights.items():
                scores[category] += points
    
    return scores

def determine_archetype_advanced(scores):
    """
    Визначає архетип на основі таблиці з зображення
    """
    # Знаходимо топ-категорії та найнижчі
    sorted_scores = sorted(scores.items(), key=lambda x: x[1], reverse=True)
    
    # Патерни архетипів з таблиці
    archetype_patterns = {
        "Маржинальний жнець": {
            "high": [],
            "low": ["KB", "RM", "EI"],
            "medium_high": ["AS", "AD"],
            "description": "Агресія, імпульсивність, ігнорування ризиків, маніпуляція, схильність до all-in, низька емпатія"
        },
        "Рептилія": {
            "high": ["RM", "AS"],
            "medium": ["EI", "KB"],
            "low": [],
            "description": "Системність, ригідність, дотримання правил, уникає змін, низька адаптивність, мінімізація втрат"
        },
        "Арбітражний стабілізатор": {
            "high": ["RM", "AD", "KB"],
            "medium": ["EI"],
            "low": [],
            "description": "Раціональність, пошук дисбалансів, диверсифікація, арбітраж, уникає емоційних рішень"
        },
        "Провокатор хаосу": {
            "high": ["AD"],
            "medium_high": ["AS"],
            "low": ["KB", "EI"],
            "description": "Любить волатильність, провокує ринок, використовує хаос, схильний до ризикованих стратегій"
        },
        "Емерджентний аналітик": {
            "high": ["AD", "KB", "RM", "EI"],
            "low": [],
            "description": "Адаптивний, технологічний, рефлексивний, аналітичний, швидко змінює стратегії, використовує AI/ML"
        },
        "Антифрагільний агент": {
            "high": ["AD", "RM", "EI", "AS"],
            "low": [],
            "description": "Виграє від стресу, любить асиметрію, використовує невизначеність як ресурс, гнучкий"
        },
        "Трендсеттер": {
            "high": ["AS", "EI", "AD"],
            "medium": ["KB"],
            "low": [],
            "description": "Впливає на думки інших, формує наративи, соціально активний, високий емоційний інтелект"
        },
        "Медіатор Духів": {
            "high": ["EI", "AS", "AD"],
            "low": ["KB"],
            "description": "Висока емпатія, інтуїція, емоційна мудрість, схильність до рефлексії, балансує емоції та раціо"
        }
    }
    

In [8]:
archetype_info = {
    "Арбітражний стабілізатор": {
        "description": {
            "uk": """
<h3>Арбітражний стабілізатор</h3>
<p><b>Кредо:</b><br>
«Баланс — це не компроміс, а мій особистий edge. Я не ганяюсь за хвилями — я їх вирівнюю».</p>
<p><b>Суть:</b><br>
Ти — невидимий архітектор ринку, який не полює за екшеном, а вміє знаходити “золото” там, де інші бачать лише пил. Твої робочі інструменти — калькулятор, скрипти і здоровий скепсис до всього гучного. Для тебе ідеальний день — це коли всі кричать про “ракети”, а ти тихо закриваєш черговий мікроспед з мінімальним ризиком. Ти не герой мемів, але саме на тобі тримається ринкова гігієна.</p>
<p><b>Нейрофізіологія:</b></p>
<ul>
  <li>Підвищений окситоцин і серотонін: спокій, впевненість, задоволення від стабільності.</li>
  <li>Помірна дофамінова реакція: кайф від пошуку неефективностей, а не від ризику.</li>
  <li>Активна префронтальна кора: аналітичність, швидке ухвалення рішень без емоційних “гойдалок”.</li>
</ul>
<p><b>Вплив на ринок:</b></p>
<ul>
  <li>Згладжуєш волатильність, прибираєш локальні “дірки” в ціноутворенні, робиш ринок ефективнішим для всіх, навіть якщо більшість цього не помічає.</li>
  <li>Твої дії — це “мастило” для ринкової машини, без якого система швидко почала б скрипіти.</li>
</ul>
<p><b>На що реагує:</b></p>
<ul>
  <li>Будь-які цінові розбіжності, короткострокові аномалії, стабільні тренди, які можна “допиляти” до ідеалу.</li>
</ul>
<p><b>На що не реагує:</b></p>
<ul>
  <li>Емоційний фон, хайп, новини без впливу на спред, “великі ідеї” без конкретної арбітражної логіки.</li>
</ul>
<p><b>Операційний арсенал:</b></p>
<ul>
  <li>Глибокий міжбіржовий арбітраж: синхронізація стаканів на різних біржах, ловля “дір” у ліквідності, використання альтернативних order flow та latency-аналізу для вилову мікроспредів.</li>
  <li>Динамічне балансування через автоматизовані risk overlay-модулі: системи, які самостійно вимикають неефективні стратегії при regime change, перебудовують портфель під нові ринкові умови.</li>
  <li>Арбітраж на подіях: робота з event-driven моделями (наприклад, різниця цін на ф’ючерси/спот під час форків, дивідендів, корпоративних подій), хеджування ризику через опціони або деривативи.</li>
  <li>Побудова “кошиків” для маркет-мейкінгу: одночасна торгівля кількома активами для вирівнювання факторних ризиків, використання автоматизованих систем для оптимізації спреду та управління inventory.</li>
  <li>Трикутний і смарт-арбітраж: використання трикутних і кругових схем (особливо в крипті) для вилову цінових неефективностей між кількома парами, автоматизація через ботів для оперативного реагування на зміни спреду.</li>
  <li>“Зловив спред на 0,11% між CME та Binance, поки всі ловили памп — ідеальний день”.</li>
</ul>
<p><b>Архітектурний підхід:</b></p>
<ul>
  <li>Власна інфраструктура для збору альтернативних даних: order flow з різних бірж, глибокий аналіз стакану, інтеграція даних про ліквідність і latency.</li>
  <li>Постійний автоматизований backtesting і walk-forward оптимізація моделей.</li>
  <li>Використання cloud computing для паралельного тестування і швидкого розгортання нових стратегій.</li>
  <li>Модульна система ризик-менеджменту: overlay risk parity, volatility targeting, автоматичне відключення “слабких” моделей при regime change.</li>
</ul>
<p><b>Когнітивні патерни:</b></p>
<ul>
  <li>Схильність до “мікрооптимізації”: годинами шліфуєш модель заради ще 0,1% прибутку.</li>
  <li>Ризик “залипнути” у деталях і прогавити великі структурні зміни на ринку.</li>
</ul>
<p><b>Що робить щасливим:</b><br>
Момент ідеального балансу, коли аналітика створює гармонію. Відчуття справедливості, коли твоя “дрібна” робота змінює ринок у кращий бік.<br><br>
<i>Цей архетип — для тих, хто не женеться за хайпом, а перемагає ринок тишею, точністю й холодною головою.</i>
</p>
""",
            "ru": """
<h3>Арбитражный стабилизатор</h3>
<p><b>Кредо:</b><br>
«Баланс — это не компромисс, а мой личный edge. Я не гоняюсь за волнами — я их выравниваю».</p>
<p><b>Суть:</b><br>
Ты — невидимый архитектор рынка, который не охотится за экшеном, а умеет находить “золото” там, где другие видят лишь пыль. Твои рабочие инструменты — калькулятор, скрипты и здоровый скептицизм ко всему громкому. Для тебя идеальный день — это когда все кричат про “ракеты”, а ты тихо закрываешь очередной микроспрэд с минимальным риском. Ты не герой мемов, но именно на тебе держится рыночная гигиена.</p>
<p><b>Нейрофизиология:</b></p>
<ul>
  <li>Повышенный окситоцин и серотонин: спокойствие, уверенность, удовольствие от стабильности.</li>
  <li>Умеренная дофаминовая реакция: кайф от поиска неэффективностей, а не от риска.</li>
  <li>Активная префронтальная кора: аналитичность, быстрое принятие решений без эмоциональных “качелей”.</li>
</ul>
<p><b>Влияние на рынок:</b></p>
<ul>
  <li>Сглаживаешь волатильность, убираешь локальные “дыры” в ценообразовании, делаешь рынок эффективнее для всех, даже если большинство этого не замечает.</li>
  <li>Твои действия — это “смазка” для рыночной машины, без которой система быстро начала бы скрипеть.</li>
</ul>
<p><b>На что реагирует:</b></p>
<ul>
  <li>Любые ценовые расхождения, краткосрочные аномалии, стабильные тренды, которые можно “допилить” до идеала.</li>
</ul>
<p><b>На что не реагирует:</b></p>
<ul>
  <li>Эмоциональный фон, хайп, новости без влияния на спред, “большие идеи” без конкретной арбитражной логики.</li>
</ul>
<p><b>Операционный арсенал:</b></p>
<ul>
  <li>Глубокий межбиржевой арбитраж: синхронизация стаканов на разных биржах, ловля “дыр” в ликвидности, использование альтернативных order flow и latency-анализа для ловли микроспрэдов.</li>
  <li>Динамическое балансирование через автоматизированные risk overlay-модули: системы, которые самостоятельно отключают неэффективные стратегии при regime change, перестраивают портфель под новые рыночные условия.</li>
  <li>Арбитраж на событиях: работа с event-driven моделями (например, разница цен на фьючерсы/спот во время форков, дивидендов, корпоративных событий), хеджирование риска через опционы или деривативы.</li>
  <li>Построение “корзин” для маркет-мейкинга: одновременная торговля несколькими активами для выравнивания факторных рисков, использование автоматизированных систем для оптимизации спреда и управления inventory.</li>
  <li>Треугольный и смарт-арбитраж: использование треугольных и круговых схем (особенно в крипте) для ловли ценовых неэффективностей между несколькими парами, автоматизация через ботов для оперативного реагирования на изменения спреда.</li>
  <li>“Поймал спред на 0,11% между CME и Binance, пока все ловили памп — идеальный день”.</li>
</ul>
<p><b>Архитектурный подход:</b></p>
<ul>
  <li>Собственная инфраструктура для сбора альтернативных данных: order flow с разных бирж, глубокий анализ стакана, интеграция данных о ликвидности и latency.</li>
  <li>Постоянный автоматизированный backtesting и walk-forward оптимизация моделей.</li>
  <li>Использование cloud computing для параллельного тестирования и быстрого развертывания новых стратегий.</li>
  <li>Модульная система риск-менеджмента: overlay risk parity, volatility targeting, автоматическое отключение “слабых” моделей при regime change.</li>
</ul>
<p><b>Когнитивные паттерны:</b></p>
<ul>
  <li>Склонность к “микрооптимизации”: часами шлифуешь модель ради еще 0,1% прибыли.</li>
  <li>Риск “залипнуть” в деталях и пропустить крупные структурные изменения на рынке.</li>
</ul>
<p><b>Что делает счастливым:</b><br>
Момент идеального баланса, когда аналитика создает гармонию. Ощущение справедливости, когда твоя “мелкая” работа меняет рынок к лучшему.<br><br>
<i>Этот архетип — для тех, кто не гонится за хайпом, а побеждает рынок тишиной, точностью и холодной головой.</i>
</p>
""",
            "en": """
<h3>Arbitrage Stabilizer</h3>
<p><b>Credo:</b><br>
"Balance is not a compromise, but my personal edge. I don't chase waves — I smooth them out."</p>
<p><b>Essence:</b><br>
You are the invisible architect of the market who doesn't hunt for action but knows how to find "gold" where others see only dust. Your tools are calculator, scripts, and healthy skepticism towards all the hype. Your perfect day is when everyone screams about "rockets," and you quietly close another microspread with minimal risk. You're not a meme hero, but the market hygiene depends on you.</p>
<p><b>Neurophysiology:</b></p>
<ul>
  <li>Increased oxytocin and serotonin: calm, confidence, satisfaction from stability.</li>
  <li>Moderate dopamine response: pleasure from finding inefficiencies, not from risk.</li>
  <li>Active prefrontal cortex: analytical thinking, quick decision-making without emotional "swings."</li>
</ul>
<p><b>Market impact:</b></p>
<ul>
  <li>Smooths volatility, removes local "holes" in pricing, makes the market more efficient for everyone, even if most don't notice.</li>
  <li>Your actions are the "lubricant" for the market machine, without which the system would quickly start to creak.</li>
</ul>
<p><b>What it reacts to:</b></p>
<ul>
  <li>Any price discrepancies, short-term anomalies, stable trends that can be "fine-tuned" to perfection.</li>
</ul>
<p><b>What it ignores:</b></p>
<ul>
  <li>Emotional background, hype, news without impact on spread, "big ideas" without specific arbitrage logic.</li>
</ul>
<p><b>Operational arsenal:</b></p>
<ul>
  <li>Deep inter-exchange arbitrage: synchronizing order books on different exchanges, catching "holes" in liquidity, using alternative order flow and latency analysis to catch microspreads.</li>
  <li>Dynamic balancing through automated risk overlay modules: systems that automatically disable ineffective strategies during regime changes, rebuilding portfolios for new market conditions.</li>
  <li>Event-driven arbitrage: working with event-driven models (e.g., price differences on futures/spot during forks, dividends, corporate events), hedging risk through options or derivatives.</li>
  <li>Building "baskets" for market making: simultaneous trading of multiple assets to balance factor risks, using automated systems to optimize spreads and manage inventory.</li>
  <li>Triangular and smart arbitrage: using triangular and circular schemes (especially in crypto) to catch price inefficiencies between multiple pairs, automating with bots for quick response to spread changes.</li>
  <li>"Caught a 0.11% spread between CME and Binance while everyone was chasing the pump — perfect day."</li>
</ul>
<p><b>Architectural approach:</b></p>
<ul>
  <li>Own infrastructure for collecting alternative data: order flow from various exchanges, deep order book analysis, integration of liquidity and latency data.</li>
  <li>Continuous automated backtesting and walk-forward optimization of models.</li>
  <li>Use of cloud computing for parallel testing and rapid deployment of new strategies.</li>
  <li>Modular risk management system: overlay risk parity, volatility targeting, automatic disabling of "weak" models during regime changes.</li>
</ul>
<p><b>Cognitive patterns:</b></p>
<ul>
  <li>Tendency to "micro-optimize": spending hours refining the model for an extra 0.1% profit.</li>
  <li>Risk of "getting stuck" in details and missing major structural market changes.</li>
</ul>
<p><b>What makes happy:</b><br>
The moment of perfect balance when analytics create harmony. The feeling of justice when your "small" work changes the market for the better.<br><br>
<i>This archetype is for those who don't chase hype but win the market with silence, precision, and a cool head.</i>
</p>
"""
        },
        "image": "Stabilizer.jpg"
    }
    ,   
    "Провокатор хаосу": {
        "description": {
            "uk": """
<h3>Провокатор хаосу</h3>
<p><b>Кредо:</b><br>
«Волатильність — мій кофеїн. Якщо ринок спокійний, я вже шукаю, як його розворушити. Flash crash — це не проблема, а просто ще один спосіб зробити ринок веселішим для себе (і трохи болючішим для інших)».</p>
<p><b>Суть:</b><br>
Ти не терпиш нудьги. “Тиша на ринку” — це виклик, а не комфорт. Заходиш у позиції тоді, коли всі воліють спостерігати, і часто робиш це навмисно проти тренду. Ти не просто граєш на волатильності — ти її створюєш, і кайфуєш, коли твоя угода стає тригером для чужої паніки.</p>
<p><b>Нейрофізіологія:</b></p>
<ul>
  <li>Високий дофамін і адреналін: жага ризику, залежність від новизни, гостре відчуття “гри”.</li>
  <li>Низький рівень ГАМК: слабкий контроль імпульсів, схильність до імпульсивних входів і швидких рішень.</li>
  <li>Гіперреактивна мигдалина: гостра реакція на сигнали хаосу, швидкий запуск стресової відповіді.</li>
</ul>
<p><b>Вплив на ринок:</b></p>
<ul>
  <li>Генеруєш волатильність, запускаєш ланцюгові реакції — твоя поява часто означає “американські гірки” для всіх.</li>
  <li>Можеш стати причиною локальних “flash crash”, змушуючи інших переглядати свої стратегії.</li>
  <li>Твоя присутність змушує ринок бути готовим до несподіванок — і це твоя головна фішка.</li>
</ul>
<p><b>На що реагує:</b></p>
<ul>
  <li>Ознаки хаосу, сплески об’ємів, фейкові сигнали, нестандартна поведінка ціни.</li>
  <li>Масова паніка чи ейфорія — ти відчуваєш, коли більшість учасників втрачає контроль і використовуєш це як точку входу.</li>
  <li>Вразливість систем: технічні збої, затримки, неочікувані події.</li>
</ul>
<p><b>На що не реагує:</b></p>
<ul>
  <li>Стабільність, передбачуваність, рутинні тренди.</li>
  <li>“Суха” аналітика без емоційної складової, фундаментальні фактори, якщо вони не створюють негайних можливостей для дестабілізації.</li>
</ul>
<p><b>Арсенал/Стратегії:</b></p>
<ul>
  <li>Складні опціонні спреди на spike volatility: використання конструкцій типу ratio spreads, butterfly, gamma scalping для гри на різких стрибках волатильності, коли ринок “кидає” з крайності в крайність.</li>
  <li>Глибокий order book-аналіз: побудова heatmap ліквідності для точкового запуску хвиль, активне використання spoofing/iceberg orders (у permitted jurisdictions) для провокації локальних “flash crash”.</li>
  <li>Новинний скальпінг: миттєва реакція на breaking news, економічні релізи, заяви центробанків; робота на low liquidity інструментах із використанням real-time news feeds та order book для виявлення “тонких місць”.</li>
  <li>Інформаційні провокації: запуск хвиль FOMO/FUD через меми, чутки, crowd-sentiment; вхід у позицію на емоційному піку з подальшим розворотом проти натовпу.</li>
  <li>Контртрендові тактики у фазах пікової волатильності: відкриття позицій проти руху у моменти масової паніки чи ейфорії, “ловля ножів” у найризикованіші моменти, але з чітким планом виходу.</li>
  <li>“Під час flash crash на low cap альті — сам створив хвилю, сам же і зібрав ліквідність”.</li>
</ul>
<p><b>Когнітивні патерни:</b></p>
<ul>
  <li>Інтелектуалізований “контртренд”: гра проти ринку не через аналіз, а через бажання “переграти всіх”.</li>
  <li>Ескалація ризику після серії перемог: збільшення розміру угод після успіху, ігнорування зміни ринкової ситуації.</li>
</ul>
<p><b>Що робить щасливим?</b><br>
Відчуття, що саме ти запустив хвилю, яку ніхто не чекав. Новизна, експеримент, ламання шаблонів — драйв від того, що ринок “оживає” завдяки твоїм діям.<br><br>
<i>Цей архетип — для тих, хто не боїться хаосу, а навпаки: вміє його створювати і отримувати від цього справжній кайф, навіть якщо інші в цей час бігають із відрами, щоб гасити пожежу.</i>
</p>
""",
            "ru": """
<h3>Провокатор хаоса</h3>
<p><b>Кредо:</b><br>
«Волатильность — мой кофеин. Если рынок спокоен, я уже ищу, как его раскачать. Flash crash — не проблема, а просто еще один способ сделать рынок веселее для себя (и чуть больнее для других)».</p>
<p><b>Суть:</b><br>
Ты не терпишь скуки. “Тишина на рынке” — это вызов, а не комфорт. Заходишь в позиции тогда, когда все предпочитают наблюдать, и часто делаешь это намеренно против тренда. Ты не просто играешь на волатильности — ты ее создаешь, и кайфуешь, когда твоя сделка становится триггером для чужой паники.</p>
<p><b>Нейрофизиология:</b></p>
<ul>
  <li>Высокий дофамин и адреналин: жажда риска, зависимость от новизны, острое ощущение “игры”.</li>
  <li>Низкий уровень ГАМК: слабый контроль импульсов, склонность к импульсивным входам и быстрым решениям.</li>
  <li>Гиперреактивная миндалина: острая реакция на сигналы хаоса, быстрый запуск стрессового ответа.</li>
</ul>
<p><b>Влияние на рынок:</b></p>
<ul>
  <li>Генерируешь волатильность, запускаешь цепные реакции — твое появление часто означает “американские горки” для всех.</li>
  <li>Можешь стать причиной локальных “flash crash”, заставляя других пересматривать свои стратегии.</li>
  <li>Твое присутствие заставляет рынок быть готовым к неожиданностям — и это твоя главная фишка.</li>
</ul>
<p><b>На что реагирует:</b></p>
<ul>
  <li>Признаки хаоса, всплески объемов, фейковые сигналы, нестандартное поведение цены.</li>
  <li>Массовая паника или эйфория — ты чувствуешь, когда большинство участников теряет контроль и используешь это как точку входа.</li>
  <li>Уязвимость систем: технические сбои, задержки, неожиданные события.</li>
</ul>
<p><b>На что не реагирует:</b></p>
<ul>
  <li>Стабильность, предсказуемость, рутинные тренды.</li>
  <li>“Сухая” аналитика без эмоциональной составляющей, фундаментальные факторы, если они не создают немедленных возможностей для дестабилизации.</li>
</ul>
<p><b>Арсенал/Стратегии:</b></p>
<ul>
  <li>Сложные опционные спреды на spike volatility: использование конструкций типа ratio spreads, butterfly, gamma scalping для игры на резких скачках волатильности, когда рынок “кидает” с крайности в крайность.</li>
  <li>Глубокий order book-анализ: построение heatmap ликвидности для точечного запуска волн, активное использование spoofing/iceberg orders (в permitted jurisdictions) для провокации локальных “flash crash”.</li>
  <li>Новостной скальпинг: мгновенная реакция на breaking news, экономические релизы, заявления центробанков; работа на low liquidity инструментах с использованием real-time news feeds и order book для выявления “тонких мест”.</li>
  <li>Информационные провокации: запуск волн FOMO/FUD через мемы, слухи, crowd-sentiment; вход в позицию на эмоциональном пике с последующим разворотом против толпы.</li>
  <li>Контртрендовые тактики в фазах пиковой волатильности: открытие позиций против движения в моменты массовой паники или эйфории, “ловля ножей” в самые рискованные моменты, но с четким планом выхода.</li>
  <li>“Во время flash crash на low cap альте — сам создал волну, сам же и собрал ликвидность”.</li>
</ul>
<p><b>Когнитивные паттерны:</b></p>
<ul>
  <li>Интеллектуализированный “контртренд”: игра против рынка не через анализ, а через желание “переграть всех”.</li>
  <li>Эскалация риска после серии побед: увеличение размера сделок после успеха, игнорирование изменения рыночной ситуации.</li>
</ul>
<p><b>Что делает счастливым?</b><br>
Ощущение, что именно ты запустил волну, которую никто не ждал. Новизна, эксперимент, ломание шаблонов — драйв от того, что рынок “оживает” благодаря твоим действиям.<br><br>
<i>Этот архетип — для тех, кто не боится хаоса, а наоборот: умеет его создавать и получать от этого настоящий кайф, даже если другие в это время бегают с ведрами, чтобы тушить пожар.</i>
</p>
""",
            "en": """
<h3>Chaos Provocateur</h3>
<p><b>Credo:</b><br>
"Volatility is my caffeine. If the market is calm, I'm already looking for ways to shake it up. Flash crash isn't a problem, just another way to make the market more fun for myself (and a bit more painful for others)."</p>
<p><b>Essence:</b><br>
You can't stand boredom. "Silence in the market" is a challenge, not comfort. You enter positions when everyone prefers to watch, often deliberately against the trend. You don't just play volatility — you create it, and you enjoy when your trade triggers others' panic.</p>
<p><b>Neurophysiology:</b></p>
<ul>
  <li>High dopamine and adrenaline: craving risk, novelty addiction, sharp sense of “game”.</li>
  <li>Low GABA levels: weak impulse control, tendency to impulsive entries and quick decisions.</li>
  <li>Hyperreactive amygdala: sharp reaction to chaos signals, quick stress response activation.</li>
</ul>
<p><b>Market impact:</b></p>
<ul>
  <li>You generate volatility, trigger chain reactions — your presence often means “rollercoaster” for all.</li>
  <li>You can cause local “flash crashes”, forcing others to rethink their strategies.</li>
  <li>Your presence makes the market ready for surprises — and that's your main feature.</li>
</ul>
<p><b>What it reacts to:</b></p>
<ul>
  <li>Signs of chaos, volume spikes, fake signals, unusual price behavior.</li>
  <li>Mass panic or euphoria — you sense when most participants lose control and use it as an entry point.</li>
  <li>System vulnerabilities: technical failures, delays, unexpected events.</li>
</ul>
<p><b>What it ignores:</b></p>
<ul>
  <li>Stability, predictability, routine trends.</li>
  <li>“Dry” analytics without emotional component, fundamental factors unless they create immediate destabilization opportunities.</li>
</ul>
<p><b>Arsenal/Strategies:</b></p>
<ul>
  <li>Complex option spreads on spike volatility: using ratio spreads, butterfly, gamma scalping to play sharp volatility jumps when the market swings from extreme to extreme.</li>
  <li>Deep order book analysis: building liquidity heatmaps for precise wave triggers, active use of spoofing/iceberg orders (in permitted jurisdictions) to provoke local “flash crashes”.</li>
  <li>News scalping: instant reaction to breaking news, economic releases, central bank statements; working on low liquidity instruments using real-time news feeds and order book to identify “thin spots”.</li>
  <li>Information provocations: triggering waves of FOMO/FUD via memes, rumors, crowd sentiment; entering positions at emotional peaks with subsequent reversal against the crowd.</li>
  <li>Counter-trend tactics during peak volatility phases: opening positions against the movement during mass panic or euphoria, “catching knives” at the riskiest moments but with a clear exit plan.</li>
  <li>“During a flash crash on a low cap alt — I created the wave and collected the liquidity myself.”</li>
</ul>
<p><b>Cognitive patterns:</b></p>
<ul>
  <li>Intellectualized “counter-trend”: playing against the market not through analysis but through the desire to “outsmart everyone”.</li>
  <li>Risk escalation after a series of wins: increasing trade size after success, ignoring market condition changes.</li>
</ul>
<p><b>What makes happy?</b><br>
The feeling that you started a wave no one expected. Novelty, experiment, breaking patterns — the thrill of making the market “come alive” through your actions.<br><br>
<i>This archetype is for those who are not afraid of chaos but rather know how to create it and truly enjoy it, even if others run around with buckets trying to put out the fire.</i>
</p>
"""
        },
        "image": "chaos.jpg"
    }
    ,
    "Антифрагільний агент": {
        "description": {
            "uk": """
<h3>Антифрагільний агент</h3>
<p><b>Кредо:</b><br>
«Криза — це мій улюблений таймфрейм. Я росту, коли всі тікають».</p>
<p><b>Суть:</b><br>
Ти не просто не боїшся хаосу — ти його чекаєш, як сезонного бонусу. Коли більшість ринку в паніці, ти спокійно збираєш “чорних лебедів” у колекцію і будуєш стратегії так, щоб вигравати саме в моменти турбулентності. Для тебе невдача — це не поразка, а ще один цеглинка у фундаменті майбутнього прибутку. Ти не женешся за дрібними плюсами — твоя мета асиметрія: невеликі втрати, великі перемоги. Ти можеш роками ігнорувати “звичайні” можливості, чекаючи на справжній прорив — і не комплексуєш через це.</p>
<p><b>Нейрофізіологія:</b></p>
<ul>
  <li>Висока нейропластичність: мозок швидко адаптується до нових умов і помилок, формуючи нові стратегії.</li>
  <li>Баланс кортизолу й дофаміну: стрес не паралізує, а стимулює пошук рішень; дофамінова система підкріплює не лише перемоги, а й досвід поразок.</li>
  <li>Активна префронтальна кора: стратегічне мислення, гнучкість у кризових ситуаціях.</li>
</ul>
<p><b>Вплив на ринок:</b></p>
<ul>
  <li>Стабілізує у кризі: купує, коли всі продають, і продає, коли ринок перегрітий, знижуючи амплітуду крайніх рухів.</li>
  <li>Джерело інновацій: впроваджує нові підходи до ризик-менеджменту, тестує нестандартні стратегії, які потім стають трендом.</li>
  <li>Підвищує “антихрупкість” ринку: створює середовище, де навіть негативні події трансформуються у нові можливості.</li>
</ul>
<p><b>На що реагує:</b></p>
<ul>
  <li>Стресові події, ринкові кризи, “чорні лебеді” — це улюблене середовище, де шукає асиметричні можливості.</li>
  <li>Нова інформація, невизначеність, зміна трендів — активно перебудовує стратегію, якщо старі правила не працюють.</li>
  <li>Власні помилки — аналізує факапи, перетворюючи їх на джерело конкурентної переваги.</li>
</ul>
<p><b>На що не реагує:</b></p>
<ul>
  <li>Рутинна стабільність, відсутність простору для адаптації.</li>
  <li>Консервативні сигнали та “застарілі” правила, які не дають переваги в нових умовах.</li>
  <li>Короткостроковий шум — фільтрує незначущі коливання, концентруючись на подіях, які можуть змінити правила гри.</li>
</ul>
<p><b>Арсенал/Стратегії:</b></p>
<ul>
  <li>Barrier/digital/exotic options: побудова асиметричних конструкцій для виграшу лише у разі екстремальних рухів. Використання опціонів типу barrier, digital, one-touch для ставок на “чорних лебедів”.</li>
  <li>Tail risk swaps через OTC: обмін ризиком падіння з фондами, які бояться “чорних лебедів”, через позабіржові угоди (tail risk swaps, disaster insurance).</li>
  <li>ML-based stress testing: автоматичний запуск перебалансування портфеля при появі “аномальних” сигналів із нетрадиційних джерел (наприклад, аналіз новин, супутникових даних, соціальних трендів).</li>
  <li>Барбел-стратегія: поєднання ультраконсервативних активів із ультраризиковими для вигоди від екстремальних подій. Мінімум ризику в “тиші”, максимум — у фазі хаосу.</li>
  <li>Використання криз як точки входу: акумуляція ліквідності для агресивних входів у моменти масової паніки, коли активи “горять”, і ринок капітулює.</li>
  <li>Динамічне перебалансування: гнучка зміна структури портфеля залежно від ринкової турбулентності; автоматичне вимикання неефективних стратегій після кризових сигналів.</li>
  <li>“У 2020-му купив digital put на S&P — усі сміялися, поки ринок не впав на 30% за тиждень.”</li>
</ul>
<p><b>Когнітивні патерни:</b></p>
<ul>
  <li>Гіперфокус на “чорних лебедях”: схильність шукати екстремальні сценарії навіть там, де їх імовірність мінімальна.</li>
  <li>Надмірний скепсис до стабільних трендів: ігнорування “звичайних” прибуткових можливостей заради очікування катастрофи.</li>
</ul>
<p><b>Що робить щасливим?</b><br>
Відчуття зростання через випробування, еволюція через виклик і відповідь. Кожна криза — це шанс стати сильнішим і відчути свою еволюцію, навіть якщо для інших це просто “ще один обвал”.<br><br>
<i>Цей архетип — для тих, хто не просто виживає у хаосі, а робить із нього свою головну перевагу, навіть якщо інші в цей час уже давно вийшли з ринку.</i>
</p>
""",
            "ru": """
<h3>Антихрупкий агент</h3>
<p><b>Кредо:</b><br>
«Кризис — мой любимый таймфрейм. Я расту, когда все бегут».</p>
<p><b>Суть:</b><br>
Ты не просто не боишься хаоса — ты его ждёшь, как сезонный бонус. Когда большинство рынка в панике, ты спокойно собираешь “чёрных лебедей” в коллекцию и строишь стратегии так, чтобы выигрывать именно в моменты турбулентности. Для тебя неудача — не поражение, а ещё один кирпичик в фундаменте будущей прибыли. Ты не гонишься за мелкими плюсами — твоя цель асимметрия: небольшие потери, большие победы. Ты можешь годами игнорировать “обычные” возможности, ожидая настоящий прорыв — и не комплексуешь из-за этого.</p>
<p><b>Нейрофизиология:</b></p>
<ul>
  <li>Высокая нейропластичность: мозг быстро адаптируется к новым условиям и ошибкам, формируя новые стратегии.</li>
  <li>Баланс кортизола и дофамина: стресс не парализует, а стимулирует поиск решений; дофаминовая система подкрепляет не только победы, но и опыт поражений.</li>
  <li>Активная префронтальная кора: стратегическое мышление, гибкость в кризисных ситуациях.</li>
</ul>
<p><b>Влияние на рынок:</b></p>
<ul>
  <li>Стабилизирует в кризис: покупает, когда все продают, и продаёт, когда рынок перегрет, снижая амплитуду крайних движений.</li>
  <li>Источник инноваций: внедряет новые подходы к риск-менеджменту, тестирует нестандартные стратегии, которые затем становятся трендом.</li>
  <li>Повышает “антихрупкость” рынка: создаёт среду, где даже негативные события трансформируются в новые возможности.</li>
</ul>
<p><b>На что реагирует:</b></p>
<ul>
  <li>Стрессовые события, рыночные кризисы, “чёрные лебеди” — это любимая среда, где ищет асимметричные возможности.</li>
  <li>Новая информация, неопределённость, смена трендов — активно перестраивает стратегию, если старые правила не работают.</li>
  <li>Собственные ошибки — анализирует факапы, превращая их в источник конкурентного преимущества.</li>
</ul>
<p><b>На что не реагирует:</b></p>
<ul>
  <li>Рутинная стабильность, отсутствие пространства для адаптации.</li>
  <li>Консервативные сигналы и “устаревшие” правила, которые не дают преимущества в новых условиях.</li>
  <li>Краткосрочный шум — фильтрует незначимые колебания, концентрируясь на событиях, которые могут изменить правила игры.</li>
</ul>
<p><b>Арсенал/Стратегии:</b></p>
<ul>
  <li>Barrier/digital/exotic options: построение асимметричных конструкций для выигрыша только в случае экстремальных движений. Использование опционов типа barrier, digital, one-touch для ставок на “чёрных лебедей”.</li>
  <li>Tail risk swaps через OTC: обмен риском падения с фондами, которые боятся “чёрных лебедей”, через внебиржевые сделки (tail risk swaps, disaster insurance).</li>
  <li>ML-based stress testing: автоматический запуск ребалансировки портфеля при появлении “аномальных” сигналов из нетрадиционных источников (например, анализ новостей, спутниковых данных, социальных трендов).</li>
  <li>Барбел-стратегия: сочетание ультраконсервативных активов с ультрарисковыми для выгоды от экстремальных событий. Минимум риска в “тишине”, максимум — в фазе хаоса.</li>
  <li>Использование кризисов как точки входа: аккумуляция ликвидности для агрессивных входов в моменты массовой паники, когда активы “горят”, и рынок капитулирует.</li>
  <li>Динамическая ребалансировка: гибкое изменение структуры портфеля в зависимости от рыночной турбулентности; автоматическое отключение неэффективных стратегий после кризисных сигналов.</li>
  <li>“В 2020-м купил digital put на S&P — все смеялись, пока рынок не упал на 30% за неделю.”</li>
</ul>
<p><b>Когнитивные паттерны:</b></p>
<ul>
  <li>Гиперфокус на “чёрных лебедях”: склонность искать экстремальные сценарии даже там, где их вероятность минимальна.</li>
  <li>Чрезмерный скепсис к стабильным трендам: игнорирование “обычных” прибыльных возможностей ради ожидания катастрофы.</li>
</ul>
<p><b>Что делает счастливым?</b><br>
Ощущение роста через испытания, эволюция через вызов и ответ. Каждый кризис — это шанс стать сильнее и почувствовать свою эволюцию, даже если для других это просто “ещё один обвал”.<br><br>
<i>Этот архетип — для тех, кто не просто выживает в хаосе, а делает из него своё главное преимущество, даже если другие в это время уже давно ушли с рынка.</i>
</p>
""",
            "en": """
<h3>Antifragile Agent</h3>
<p><b>Credo:</b><br>
"Crisis is my favorite timeframe. I grow when everyone else runs."</p>
<p><b>Essence:</b><br>
You don't just fear chaos — you wait for it like a seasonal bonus. When most of the market panics, you calmly collect "black swans" and build strategies to win precisely in turbulent times. For you, failure is not defeat, but another brick in the foundation of future profit. You don't chase small gains — your goal is asymmetry: small losses, big wins. You can ignore "ordinary" opportunities for years, waiting for a real breakthrough — and you don't feel bad about it.</p>
<p><b>Neurophysiology:</b></p>
<ul>
  <li>High neuroplasticity: your brain quickly adapts to new conditions and mistakes, forming new strategies.</li>
  <li>Balanced cortisol and dopamine: stress doesn't paralyze but stimulates solution-seeking; the dopamine system rewards not only wins but also the experience of losses.</li>
  <li>Active prefrontal cortex: strategic thinking, flexibility in crisis situations.</li>
</ul>
<p><b>Market impact:</b></p>
<ul>
  <li>Stabilizes in crisis: buys when everyone sells and sells when the market is overheated, reducing the amplitude of extreme movements.</li>
  <li>Source of innovation: introduces new risk management approaches, tests non-standard strategies that then become trends.</li>
  <li>Increases the market's "antifragility": creates an environment where even negative events turn into new opportunities.</li>
</ul>
<p><b>What it reacts to:</b></p>
<ul>
  <li>Stress events, market crises, "black swans" — this is the favorite environment for seeking asymmetric opportunities.</li>
  <li>New information, uncertainty, trend changes — actively rebuilds strategy if old rules no longer work.</li>
  <li>Own mistakes — analyzes failures, turning them into a source of competitive advantage.</li>
</ul>
<p><b>What it ignores:</b></p>
<ul>
  <li>Routine stability, lack of space for adaptation.</li>
  <li>Conservative signals and "outdated" rules that don't give an edge in new conditions.</li>
  <li>Short-term noise — filters out insignificant fluctuations, focusing on events that can change the rules of the game.</li>
</ul>
<p><b>Arsenal/Strategies:</b></p>
<ul>
  <li>Barrier/digital/exotic options: building asymmetric structures to win only in case of extreme moves. Using barrier, digital, one-touch options to bet on "black swans".</li>
  <li>Tail risk swaps via OTC: exchanging downside risk with funds afraid of "black swans" through OTC deals (tail risk swaps, disaster insurance).</li>
  <li>ML-based stress testing: automatic portfolio rebalancing when "anomalous" signals appear from non-traditional sources (e.g., news analysis, satellite data, social trends).</li>
  <li>Barbell strategy: combining ultra-conservative assets with ultra-risky ones to profit from extreme events. Minimum risk in "quiet", maximum — in chaos phase.</li>
  <li>Using crises as entry points: accumulating liquidity for aggressive entries during mass panic, when assets "burn" and the market capitulates.</li>
  <li>Dynamic rebalancing: flexible portfolio structure changes depending on market turbulence; automatic disabling of ineffective strategies after crisis signals.</li>
  <li>"In 2020, bought a digital put on the S&P — everyone laughed until the market fell 30% in a week."</li>
</ul>
<p><b>Cognitive patterns:</b></p>
<ul>
  <li>Hyperfocus on "black swans": tendency to look for extreme scenarios even where their probability is minimal.</li>
  <li>Excessive skepticism towards stable trends: ignoring "ordinary" profitable opportunities in favor of waiting for catastrophe.</li>
</ul>
<p><b>What makes happy?</b><br>
The feeling of growth through trials, evolution through challenge and response. Every crisis is a chance to become stronger and feel your evolution, even if for others it's just "another crash".<br><br>
<i>This archetype is for those who don't just survive in chaos but make it their main advantage, even if others have long left the market.</i>
</p>
"""
        },
        "image": "antifragile.jpg"
    }
    ,
    "Трендсеттер": {
        "description": {
            "uk": """
<h3>Трендсеттер</h3>
<p><b>Кредо:</b><br>
“Один мій мем — і альткоін летить. Якщо ідея не стала вірусною — значить, ще не час. Мрію, щоб CoinMarketCap додав колонку ‘Вплив трендсеттера’”.</p>
<p><b>Суть:</b><br>
Ти — каталізатор ринку, який відчуває інформаційний пульс раніше за інших. Для тебе важливі не лише цифри, а й наративи, соціальні сигнали, меми, хайп. Ти можеш не мати великого рахунку, але маєш вплив — через ком’юніті, соцмережі, аналітику або просто влучний жарт у чаті. Ти не чекаєш підтвердження від “великих”, ти сам створюєш інформаційний шум, який підхоплюють інші. Твоя сила — вчасно підхопити ідею, твій ризик — втратити інтерес до стабільних можливостей, якщо в них немає “вогню”.</p>
<p><b>Нейрофізіологія:</b></p>
<ul>
  <li>Підвищена дофамінова чутливість: кайфуєш від новизни, соціального схвалення, інформаційного впливу.</li>
  <li>Активна система винагороди: отримуєш задоволення, коли твоя ідея підхоплюється масами.</li>
  <li>Висока міжособистісна чутливість: швидко зчитуєш настрій натовпу, реагуєш на зміни в інформаційному полі.</li>
</ul>
<p><b>Вплив на ринок:</b></p>
<ul>
  <li>Формуєш нові тренди, запускаєш хвилі купівлі/продажу, створюєш короткострокові та середньострокові рухи.</li>
  <li>Підвищуєш волатильність у фазах хайпу, впливаєш на поведінку інших гравців — навіть якщо фундаменту під ідеєю мало.</li>
</ul>
<p><b>На що реагує:</b></p>
<ul>
  <li>Інформаційні тренди, меми, соціальні сигнали, зміни в поведінці спільноти, медіа-активність.</li>
  <li>Відчуваєш, коли з’являється новий наратив, який може “злетіти”.</li>
</ul>
<p><b>На що не реагує:</b></p>
<ul>
  <li>Старі тренди, що втратили актуальність, глибока фундаментальна аналітика без інформаційного резонансу, рутинна стабільність.</li>
</ul>
<p><b>Арсенал:</b></p>
<ul>
  <li>Meme-cycle arbitrage: вхід у актив на першому мемі, вихід до піку Google Trends чи crowd-сигналу. Власний sentiment-бот у Telegram/Discord для реального моніторингу “градусу” хайпу.</li>
  <li>Crowd-sentiment trading: аналіз поведінки ком’юніті, виявлення “розігрітих” ідей до мейнстріму, миттєвий вхід у позицію, поки хвиля не стала загальною.</li>
  <li>Інформаційні хвилі: запуск і підхоплення наративів, використання нетрадиційних джерел даних (форумні “інсайди”, інфлюенсери, мем-цикли), швидка адаптація до зміни інформаційного поля.</li>
  <li>Breakout/momentum trading з інформаційним підтвердженням: вхід у позиції після прориву ключових рівнів, але лише якщо це підкріплено інформаційним хайпом, crowd-сигналом або різким зростанням mentions у соцмережах.</li>
  <li>Pullback trading по crowd-сигналу: купівля на відкатах у тренді, коли інформаційна хвиля ще не згасла, мультифреймовий аналіз для відсіювання фальшивих сигналів.</li>
  <li>Динамічне портфельне балансування: постійний перегляд watchlist, швидкий вихід із “згаслих” тем, баланс експозиції між гарячими ідеями та захистом від інформаційного шуму.</li>
  <li>“Створив мем у Twitter — через дві години CoinMarketCap додав актив у топ-100, crowd-сигнал спрацював ідеально.”</li>
</ul>
<p><b>Когнітивні патерни:</b></p>
<ul>
  <li>Інформаційний резонанс: схильність переоцінювати власний вплив і діяти на основі очікуваної реакції натовпу, а не фундаменту.</li>
  <li>Пошук “нової хвилі” за будь-яку ціну: втрата інтересу до стабільних можливостей, якщо вони не мають інформаційного чи соціального хайпу.</li>
</ul>
<p><b>Що робить щасливим?</b><br>
Створення нового наративу, соціальний резонанс, увага, вплив на маси. Відчуття, що твоя ідея стала вірусною й змінила правила гри, навіть якщо це тривало лише добу.<br><br>
<i>Цей архетип — для тих, хто не просто торгує, а створює інформаційний рух. Якщо ти впізнав себе — ти не просто трейдер, ти ринковий інфлюенсер, навіть якщо твій депозит менший за середній stop-loss у чаті.</i>
</p>
""",
            "ru": """
<h3>Трендсеттер</h3>
<p><b>Кредо:</b><br>
“Один мой мем — и альткоин летит. Если идея не стала вирусной — значит, еще не время. Мечтаю, чтобы CoinMarketCap добавил колонку ‘Влияние трендсеттера’”.</p>
<p><b>Суть:</b><br>
Ты — катализатор рынка, который чувствует информационный пульс раньше других. Для тебя важны не только цифры, но и нарративы, социальные сигналы, мемы, хайп. Ты можешь не иметь большого счета, но у тебя есть влияние — через комьюнити, соцсети, аналитику или просто меткий комментарий в чате. Ты не ждешь подтверждения от “крупных”, ты сам создаешь информационный шум, который подхватывают другие. Твоя сила — вовремя подхватить идею, твой риск — потерять интерес к стабильным возможностям, если в них нет “огня”.</p>
<p><b>Нейрофизиология:</b></p>
<ul>
  <li>Повышенная дофаминовая чувствительность: кайфуешь от новизны, социального одобрения, информационного влияния.</li>
  <li>Активная система вознаграждения: получаешь удовольствие, когда твою идею подхватывают массы.</li>
  <li>Высокая межличностная чувствительность: быстро считываешь настроение толпы, реагируешь на изменения в инфополе.</li>
</ul>
<p><b>Влияние на рынок:</b></p>
<ul>
  <li>Формируешь новые тренды, запускаешь волны покупок/продаж, создаешь краткосрочные и среднесрочные движения.</li>
  <li>Повышаешь волатильность в фазах хайпа, влияешь на поведение других игроков — даже если под идеей мало фундамента.</li>
</ul>
<p><b>На что реагирует:</b></p>
<ul>
  <li>Информационные тренды, мемы, социальные сигналы, изменения в поведении комьюнити, медиа-активность.</li>
  <li>Чувствуешь, когда появляется новый нарратив, который может “взлететь”.</li>
</ul>
<p><b>На что не реагирует:</b></p>
<ul>
  <li>Старые тренды, потерявшие актуальность, глубокая фундаментальная аналитика без инфорезонанса, рутинная стабильность.</li>
</ul>
<p><b>Арсенал:</b></p>
<ul>
  <li>Meme-cycle arbitrage: вход в актив на первом меме, выход до пика Google Trends или crowd-сигнала. Собственный sentiment-бот в Telegram/Discord для мониторинга “градуса” хайпа.</li>
  <li>Crowd-sentiment trading: анализ поведения комьюнити, выявление “разогретых” идей до мейнстрима, мгновенный вход в позицию, пока волна не стала общей.</li>
  <li>Информационные волны: запуск и подхват нарративов, использование нетрадиционных источников данных (форумные инсайды, инфлюенсеры, мем-циклы), быстрая адаптация к изменению инфополя.</li>
  <li>Breakout/momentum trading с инфоподтверждением: вход в позиции после пробития ключевых уровней, но только если это подкреплено инфохайпом, crowd-сигналом или резким ростом mentions в соцсетях.</li>
  <li>Pullback trading по crowd-сигналу: покупка на откатах в тренде, когда инфоволна еще не погасла, мультифреймовый анализ для отсечения ложных сигналов.</li>
  <li>Динамическое портфельное балансирование: постоянный пересмотр watchlist, быстрый выход из “угасших” тем, баланс экспозиции между горячими идеями и защитой от инфошума.</li>
  <li>“Создал мем в Twitter — через два часа CoinMarketCap добавил актив в топ-100, crowd-сигнал сработал идеально.”</li>
</ul>
<p><b>Когнитивные паттерны:</b></p>
<ul>
  <li>Информационный резонанс: склонность переоценивать собственное влияние и действовать на основе ожидаемой реакции толпы, а не фундамента.</li>
  <li>Поиск “новой волны” любой ценой: потеря интереса к стабильным возможностям, если в них нет инфо- или социального хайпа.</li>
</ul>
<p><b>Что делает счастливым?</b><br>
Создание нового нарратива, социальный резонанс, внимание, влияние на массы. Ощущение, что твоя идея стала вирусной и изменила правила игры, даже если это длилось всего сутки.<br><br>
<i>Этот архетип — для тех, кто не просто торгует, а создает инфодвиж. Если ты узнал себя — ты не просто трейдер, ты рыночный инфлюенсер, даже если твой депозит меньше среднего stop-loss в чате.</i>
</p>
""",
            "en": """
<h3>Trendsetter</h3>
<p><b>Credo:</b><br>
“One of my memes — and an altcoin pumps. If an idea didn't go viral — it's not the time yet. I dream that CoinMarketCap will add a ‘Trendsetter Impact’ column.”</p>
<p><b>Essence:</b><br>
You are a market catalyst who feels the informational pulse before anyone else. For you, not only numbers matter, but also narratives, social signals, memes, hype. You may not have a big account, but you have influence — via community, social media, analytics, or just a witty joke in chat. You don't wait for confirmation from the “big ones”, you create the informational noise others pick up. Your power is to catch the idea in time, your risk — to lose interest in stable opportunities if there’s no “fire” in them.</p>
<p><b>Neurophysiology:</b></p>
<ul>
  <li>Increased dopamine sensitivity: you love novelty, social approval, informational impact.</li>
  <li>Active reward system: you get pleasure when your idea is picked up by the masses.</li>
  <li>High interpersonal sensitivity: you quickly read the crowd’s mood, respond to changes in the info field.</li>
</ul>
<p><b>Market impact:</b></p>
<ul>
  <li>You create new trends, launch buy/sell waves, and generate short- and medium-term moves.</li>
  <li>You increase volatility in hype phases, influence others’ behavior — even if the idea has little fundamental basis.</li>
</ul>
<p><b>What it reacts to:</b></p>
<ul>
  <li>Info trends, memes, social signals, changes in community behavior, media activity.</li>
  <li>You sense when a new narrative appears that can “take off”.</li>
</ul>
<p><b>What it ignores:</b></p>
<ul>
  <li>Old trends that lost relevance, deep fundamental analytics without info resonance, routine stability.</li>
</ul>
<p><b>Arsenal:</b></p>
<ul>
  <li>Meme-cycle arbitrage: enter an asset on the first meme, exit before the Google Trends peak or crowd signal. Your own sentiment bot in Telegram/Discord to monitor the “hype temperature”.</li>
  <li>Crowd-sentiment trading: analyze community behavior, spot “heated” ideas before they go mainstream, enter a position instantly before the wave becomes general.</li>
  <li>Information waves: launch and pick up narratives, use non-traditional data sources (forum “insides”, influencers, meme-cycles), quickly adapt to info field changes.</li>
  <li>Breakout/momentum trading with info confirmation: enter after key level breakouts, but only if backed by info hype, crowd signal, or a sharp rise in social media mentions.</li>
  <li>Pullback trading by crowd signal: buy on pullbacks in a trend when the info wave isn’t over, multi-frame analysis to filter out false signals.</li>
  <li>Dynamic portfolio balancing: constantly review your watchlist, exit “faded” topics quickly, balance exposure between hot ideas and protection from info noise.</li>
  <li>“Created a meme on Twitter — two hours later CoinMarketCap added the asset to the top 100, crowd signal worked perfectly.”</li>
</ul>
<p><b>Cognitive patterns:</b></p>
<ul>
  <li>Info resonance: tendency to overestimate your influence and act based on expected crowd reaction, not fundamentals.</li>
  <li>Chasing the “new wave” at any cost: losing interest in stable opportunities if there’s no info or social hype.</li>
</ul>
<p><b>What makes happy?</b><br>
Creating a new narrative, social resonance, attention, influencing the masses. The feeling that your idea went viral and changed the rules of the game, even if only for a day.<br><br>
<i>This archetype is for those who don’t just trade, but create info-movement. If you recognize yourself — you’re not just a trader, you’re a market influencer, even if your deposit is smaller than the average stop-loss in the chat.</i>
</p>
"""
        },
        "image": "trendsetter.jpg"
    }
    ,
    "Емерджентний аналітик": {
        "description": {
            "uk": """
<h3>Емерджентний аналітик</h3>
<p><b>Кредо:</b><br>
“У мене більше дашбордів, ніж друзів. Якщо дані не підтвердили гіпотезу — змінюю гіпотезу, а не дані. Overfitting? Це не вада, це стиль життя.”</p>
<p><b>Суть:</b><br>
Ти — людина, яка не визнає “остаточних” стратегій. Для тебе ринок — це нескінченний експеримент, а будь-яка гіпотеза — лише тимчасова зупинка. Ти живеш у світі дашбордів, Python-скриптів та альтернативних джерел даних. Замість “ловити тренд”, ти ловиш закономірності, які ще не стали мейнстрімом. Твоя сила — гнучкість, твій ризик — втратити простоту у нескінченному “тюнінгу”.</p>
<p><b>Нейрофізіологія:</b></p>
<ul>
  <li>Висока нейропластичність: мозок легко перебудовується під нові патерни, швидко інтегрує нову інформацію.</li>
  <li>Активна префронтальна кора: складний аналіз, багаторівневе мислення, стратегічна варіативність.</li>
  <li>Помірний дофамін: інтерес до навчання, досліджень, пошуку нових рішень без імпульсивних дій.</li>
</ul>
<p><b>Вплив на ринок:</b></p>
<ul>
  <li>Адаптує ринок до нових умов, впроваджує інновації, які згодом стають стандартом для інших.</li>
  <li>Знижує інформаційний “шум”, фокусується на трендах, які мають реальний вплив.</li>
  <li>Підсилює конкуренцію — змушує інших постійно вдосконалюватися.</li>
</ul>
<p><b>На що реагує:</b></p>
<ul>
  <li>Нові дані, технології, аналітичні інструменти, структурні зміни ринку.</li>
  <li>Сигнали слабких трендів, які ще не стали очевидними для більшості.</li>
</ul>
<p><b>На що не реагує:</b></p>
<ul>
  <li>Рутинна стабільність, масові емоційні хвилі, старі неактуальні моделі.</li>
</ul>
<p><b>Стратегії:</b></p>
<ul>
  <li>Reinforcement learning для динамічного підбору моделей: автоматичне перемикання між стратегіями у live-режимі, вимикання “мертвих” алгоритмів при regime change.</li>
  <li>Ensemble-моделі з альтернативних джерел: побудова портфелів із сигналів різних типів (on-chain, супутникові знімки, соціальні тренди, order flow), автоматичне визначення ваги кожного джерела залежно від ринкової фази.</li>
  <li>AI/ML anomaly detection: використання машинного навчання для пошуку аномалій, регулярний re-training моделей, автоматичне оновлення під ринкову динаміку.</li>
  <li>Data-driven momentum: постійне тестування гіпотез на історичних даних, застосування складних кореляційних моделей, факторний аналіз для пошуку нових закономірностей.</li>
  <li>Автоматичне live backtesting і walk-forward optimization: система сама тестує нові моделі на реальних даних, інтегрує їх у портфель або відключає при втраті edge.</li>
  <li>Гнучкі портфелі: швидка перебудова під нові умови, автоматичне балансування залежно від зміни структури ринку, використання факторного overlay.</li>
  <li>“Після зміни кореляції між BTC та NASDAQ мої скрипти самі вимкнули стару модель і ввімкнули нову — edge зберігся, поки всі ловили drawdown.”</li>
</ul>
<p><b>Когнітивні патерни:</b></p>
<ul>
  <li>Пастка “data overfitting”: надмірне ускладнення моделей, що призводить до втрати гнучкості у реальному часі.</li>
  <li>Перевантаження альтернативними даними: втрата фокусу через надмірне залучення нестандартних джерел інформації.</li>
</ul>
<p><b>Що робить щасливим?</b><br>
Процес відкриття, інтелектуальна новизна, момент інсайту, коли ти першим бачиш тренд або закономірність, яку ще ніхто не помітив.<br><br>
<i>Цей архетип — для тих, хто не боїться змінюватися разом із ринком і завжди готовий до нового експерименту, навіть якщо для цього доведеться переписати всі свої скрипти о третій ночі.</i>
</p>
""",
            "ru": """
<h3>Эмерджентный аналитик</h3>
<p><b>Кредо:</b><br>
"У меня больше дашбордов, чем друзей. Если данные не подтвердили гипотезу — меняю гипотезу, а не данные. Overfitting? Это не недостаток, это стиль жизни."</p>
<p><b>Суть:</b><br>
Ты — человек, который не признает "окончательных" стратегий. Для тебя рынок — это бесконечный эксперимент, а любая гипотеза — лишь временная остановка. Ты живешь в мире дашбордов, Python-скриптов и альтернативных источников данных. Вместо того, чтобы "ловить тренд", ты ловишь закономерности, которые еще не стали мейнстримом. Твоя сила — гибкость, твой риск — потерять простоту в бесконечном "тюнинге".</p>
<p><b>Нейрофизиология:</b></p>
<ul>
  <li>Высокая нейропластичность: мозг легко перестраивается под новые паттерны, быстро интегрирует новую информацию.</li>
  <li>Активная префронтальная кора: сложный анализ, многоуровневое мышление, стратегическая вариативность.</li>
  <li>Умеренный дофамин: интерес к обучению, исследованиям, поиску новых решений без импульсивных действий.</li>
</ul>
<p><b>Влияние на рынок:</b></p>
<ul>
  <li>Адаптирует рынок к новым условиям, внедряет инновации, которые впоследствии становятся стандартом для других.</li>
  <li>Снижает информационный "шум", фокусируется на трендах, которые имеют реальное влияние.</li>
  <li>Усиливает конкуренцию — заставляет других постоянно совершенствоваться.</li>
</ul>
<p><b>На что реагирует:</b></p>
<ul>
  <li>Новые данные, технологии, аналитические инструменты, структурные изменения рынка.</li>
  <li>Сигналы слабых трендов, которые еще не стали очевидными для большинства.</li>
</ul>
<p><b>На что не реагирует:</b></p>
<ul>
  <li>Рутинная стабильность, массовые эмоциональные волны, старые неактуальные модели.</li>
</ul>
<p><b>Стратегии:</b></p>
<ul>
  <li>Reinforcement learning для динамического подбора моделей: автоматическое переключение между стратегиями в live-режиме, отключение "мертвых" алгоритмов при regime change.</li>
  <li>Ensemble-модели из альтернативных источников: построение портфелей из сигналов разных типов (on-chain, спутниковые снимки, социальные тренды, order flow), автоматическое определение веса каждого источника в зависимости от рыночной фазы.</li>
  <li>AI/ML anomaly detection: использование машинного обучения для поиска аномалий, регулярный re-training моделей, автоматическое обновление под рыночную динамику.</li>
  <li>Data-driven momentum: постоянное тестирование гипотез на исторических данных, применение сложных корреляционных моделей, факторный анализ для поиска новых закономерностей.</li>
  <li>Автоматическое live backtesting и walk-forward optimization: система сама тестирует новые модели на реальных данных, интегрирует их в портфель или отключает при потере edge.</li>
  <li>Гибкие портфели: быстрая перестройка под новые условия, автоматическое балансирование в зависимости от изменения структуры рынка, использование факторного overlay.</li>
  <li>“После изменения корреляции между BTC и NASDAQ мои скрипты сами выключили старую модель и включили новую — edge сохранился, пока все ловили drawdown.”</li>
</ul>
<p><b>Когнитивные паттерны:</b></p>
<ul>
  <li>Проблема “data overfitting”: чрезмерное усложнение моделей, приводящее к потере гибкости в реальном времени.</li>
  <li>Перегрузка альтернативными данными: потеря фокуса из-за чрезмерного вовлечения нестандартных источников информации.</li>
</ul>
<p><b>Что делает счастливым?</b><br>
Процесс открытия, интеллектуальная новизна, момент инсайта, когда ты первым видишь тренд или закономерность, которую еще никто не заметил.<br><br>
<i>Этот архетип — для тех, кто не боится меняться вместе с рынком и всегда готов к новому эксперименту, даже если для этого придется переписывать все свои скрипты в три часа ночи.</i>
</p>
""",
            "en": """
<h3>Emergent Analyst</h3>
<p><b>Credo:</b><br>
"I have more dashboards than friends. If the data didn't confirm the hypothesis — I change the hypothesis, not the data. Overfitting? It's not a flaw, it's a lifestyle."</p>
<p><b>Essence:</b><br>
You are a person who does not recognize "final" strategies. For you, the market is an endless experiment, and any hypothesis is just a temporary stop. You live in a world of dashboards, Python scripts, and alternative data sources. Instead of "catching the trend," you catch patterns that have not yet become mainstream. Your strength is flexibility, your risk is losing simplicity in endless "tuning."</p>
<p><b>Neurophysiology:</b></p>
<ul>
  <li>High neuroplasticity: the brain quickly adapts to new patterns, quickly integrates new information.</li>
  <li>Active prefrontal cortex: complex analysis, multi-level thinking, strategic variability.</li>
  <li>Moderate dopamine: interest in learning, research, searching for new solutions without impulsive actions.</li>
</ul>
<p><b>Market impact:</b></p>
<ul>
  <li>Adapts the market to new conditions, introduces innovations that later become the standard for others.</li>
  <li>Reduces informational "noise", focuses on trends that have real impact.</li>
  <li>Enhances competition — forces others to constantly improve.</li>
</ul>
<p><b>What it reacts to:</b></p>
<ul>
  <li>New data, technologies, analytical tools, structural market changes.</li>
  <li>Signals of weak trends that are not yet obvious to most.</li>
</ul>
<p><b>What it ignores:</b></p>
<ul>
  <li>Routine stability, mass emotional waves, old outdated models.</li>
</ul>
<p><b>Strategies:</b></p>
<ul>
  <li>Reinforcement learning for dynamic model selection: automatic switching between strategies in live mode, disabling "dead" algorithms during regime change.</li>
  <li>Ensemble models from alternative sources: building portfolios from signals of different types (on-chain, satellite images, social trends, order flow), automatic determination of the weight of each source depending on the market phase.</li>
  <li>AI/ML anomaly detection: using machine learning to detect anomalies, regular model re-training, automatic updating to market dynamics.</li>
  <li>Data-driven momentum: constant hypothesis testing on historical data, application of complex correlation models, factor analysis to find new patterns.</li>
  <li>Automatic live backtesting and walk-forward optimization: the system tests new models on real data, integrates them into the portfolio or disables them when edge is lost.</li>
  <li>Flexible portfolios: rapid restructuring for new conditions, automatic balancing depending on changes in market structure, use of factor overlay.</li>
  <li>“After the correlation between BTC and NASDAQ changed, my scripts automatically turned off the old model and turned on the new one — edge was preserved while everyone was catching drawdown.”</li>
</ul>
<p><b>Cognitive patterns:</b></p>
<ul>
  <li>Data overfitting trap: excessive model complexity leading to loss of flexibility in real time.</li>
  <li>Overload with alternative data: loss of focus due to excessive involvement of non-standard information sources.</li>
</ul>
<p><b>What makes happy?</b><br>
The process of discovery, intellectual novelty, the moment of insight when you are the first to see a trend or pattern that no one else has noticed.<br><br>
<i>This archetype is for those who are not afraid to change with the market and are always ready for a new experiment, even if it means rewriting all their scripts at three in the morning.</i>
</p>
"""
        },
        "image": "Emergent.jpg"
    }
    ,
    "Медіатор Духів": {
        "description": {
            "uk": """
<h3>Медіатор Духів</h3>
<p><b>Кредо:</b><br>
“Мій індикатор — це вайб у чаті. Якщо всі сумують — час купувати. Якщо всі святкують — час тікати. Я не ловлю дно, я ловлю спокій”.</p>
<p><b>Суть:</b><br>
Ти — емоційний барометр ринку, який не боїться хвиль страху чи ейфорії, а навпаки: інтегрує їх у свої рішення. Для тебе графік — це не просто ціна, а відображення колективного настрою. Ти купуєш, коли чат у паніці, і виходиш, коли всі святкують. Твоя сила — холодна голова у вирі емоцій, твій ризик — уникати “гострих” угод, якщо вони не резонують із твоїм внутрішнім станом.</p>
<p><b>Нейрофізіологія:</b></p>
<ul>
  <li>Висока активність ГАМК: спокій під тиском, контроль імпульсів.</li>
  <li>Баланс окситоцину та серотоніну: емпатія, рефлексія, соціальна чутливість.</li>
  <li>Низький кортизол: стійкість до стресу навіть у пікових фазах ринку.</li>
</ul>
<p><b>Вплив на ринок:</b></p>
<ul>
  <li>Стабілізуєш емоційні хвилі, стаєш прикладом для інших у кризу.</li>
  <li>Підвищуєш “емоційну грамотність” спільноти, знижуєш колективну імпульсивність.</li>
  <li>Можеш зупинити паніку серед менш стійких учасників ринку.</li>
</ul>
<p><b>На що реагує:</b></p>
<ul>
  <li>Коливання емоційного фону, сигнали масового страху чи ейфорії.</li>
  <li>Власні емоції як головний індикатор для нестандартних рішень.</li>
  <li>Поведінка ключових гравців, якщо вона емоційно заряджена.</li>
</ul>
<p><b>На що не реагує:</b></p>
<ul>
  <li>“Мертві” ринки без емоцій, сухі технічні сигнали, новини без соціального резонансу.</li>
</ul>
<p><b>Стратегії:</b></p>
<ul>
  <li>Психологічний арбітраж: торгівля проти масової паніки чи ейфорії, використання власних емоційних індикаторів для входу у моменти колективного страху або жадібності.</li>
  <li>Контртренд у фазах емоційного перегріву: вхід проти натовпу, коли емоції зашкалюють, із підтвердженням через price action, order flow або емоційний фон спільноти.</li>
  <li>Когнітивно-поведінкові техніки: самоспостереження, ведення журналу емоційних тригерів, інтеграція mindfulness для зниження імпульсивності, робота із власними когнітивними спотвореннями.</li>
  <li>Ігнорування короткострокового шуму: фільтрація сигналів через емоційний фон ком’юніті, відмова від угод, якщо не відчуваєш справжньої зміни настрою.</li>
  <li>Соціометричний моніторинг: аналіз емоцій у чатах через NLP-інструменти (наприклад, автоматичний sentiment index у Telegram/Discord), виявлення “точки кипіння” натовпу для входу чи виходу.</li>
  <li>Психофізіологічний контроль: використання біофідбеку (HRV, EDA) для моніторингу власного стресу у пікові моменти; трейдинг лише у стані внутрішнього спокою, автоматичне блокування торгівлі при підвищеному стресі.</li>
  <li>“Всі кричали ‘крах’, але мій біотрекер показав ідеальний спокій — купив і вийшов у плюс, коли всі ще панікували.”</li>
</ul>
<p><b>Когнітивні патерни:</b></p>
<ul>
  <li>Емоційна інтеграція у рішення: часто дієш на основі колективного вайбу, навіть якщо це суперечить особистому аналізу.</li>
  <li>Пошук гармонії ціною прибутку: уникаєш угод, які викликають внутрішній дискомфорт, навіть якщо вони вигідні.</li>
</ul>
<p><b>Що дає енергію для натхнення життя:</b><br>
Відчуття глибокого емоційного зв’язку з іншими, момент, коли твоя присутність допомагає іншим пройти через страх чи невпевненість. Гармонія з людьми й ринком — це твій головний “edge” у світі хаосу.<br><br>
<i>Цей архетип — для тих, хто не боїться бути “тихим голосом розуму” у вирі емоцій, і завжди пам’ятає: найкращий трейд — це той, після якого спокійно спиш.</i>
</p>
""",
            "ru": """
<h3>Медиатор Духов</h3>
<p><b>Кредо:</b><br>
“Мой индикатор — это вайб в чате. Если все грустят — пора покупать. Если все празднуют — пора уходить. Я не ловлю дно, я ловлю спокойствие.”</p>
<p><b>Суть:</b><br>
Ты — эмоциональный барометр рынка, который не боится волн страха или эйфории, а наоборот: интегрирует их в свои решения. Для тебя график — это не просто цена, а отражение коллективного настроения. Ты покупаешь, когда чат в панике, и выходишь, когда все радуются. Твоя сила — холодная голова в водовороте эмоций, твой риск — избегать “острых” сделок, если они не резонируют с твоим внутренним состоянием.</p>
<p><b>Нейрофизиология:</b></p>
<ul>
  <li>Высокая активность ГАМК: спокойствие под давлением, контроль импульсов.</li>
  <li>Баланс окситоцина и серотонина: эмпатия, рефлексия, социальная чувствительность.</li>
  <li>Низкий кортизол: устойчивость к стрессу даже в пиковых фазах рынка.</li>
</ul>
<p><b>Влияние на рынок:</b></p>
<ul>
  <li>Стабилизируешь эмоциональные волны, становишься примером для других в кризис.</li>
  <li>Повышаешь “эмоциональную грамотность” комьюнити, снижаешь коллективную импульсивность.</li>
  <li>Можешь остановить панику среди менее устойчивых участников рынка.</li>
</ul>
<p><b>На что реагирует:</b></p>
<ul>
  <li>Колебания эмоционального фона, сигналы массового страха или эйфории.</li>
  <li>Собственные эмоции как главный индикатор для нестандартных решений.</li>
  <li>Поведение ключевых игроков, если оно эмоционально заряжено.</li>
</ul>
<p><b>На что не реагирует:</b></p>
<ul>
  <li>“Мертвые” рынки без эмоций, сухие технические сигналы, новости без социального резонанса.</li>
</ul>
<p><b>Стратегии:</b></p>
<ul>
  <li>Психологический арбитраж: торговля против массовой паники или эйфории, использование собственных эмоциональных индикаторов для входа в моменты коллективного страха или жадности.</li>
  <li>Контртренд в фазах эмоционального перегрева: вход против толпы, когда эмоции зашкаливают, с подтверждением через price action, order flow или эмоциональный фон комьюнити.</li>
  <li>Когнитивно-поведенческие техники: самонаблюдение, ведение журнала эмоциональных триггеров, интеграция mindfulness для снижения импульсивности, работа с собственными когнитивными искажениями.</li>
  <li>Игнорирование краткосрочного шума: фильтрация сигналов через эмоциональный фон комьюнити, отказ от сделок, если не чувствуешь настоящего изменения настроения.</li>
  <li>Социометрический мониторинг: анализ эмоций в чатах через NLP-инструменты (например, автоматический sentiment index в Telegram/Discord), выявление “точки кипения” толпы для входа или выхода.</li>
  <li>Психофизиологический контроль: использование биофидбека (HRV, EDA) для мониторинга собственного стресса в пиковые моменты; торговля только в состоянии внутреннего спокойствия, автоматическое блокирование торговли при повышенном стрессе.</li>
  <li>“Все кричали ‘крах’, но мой биотрекер показал идеальное спокойствие — купил и вышел в плюс, когда все еще паниковали.”</li>
</ul>
<p><b>Когнитивные паттерны:</b></p>
<ul>
  <li>Эмоциональная интеграция в решения: часто действуешь на основе коллективного вайба, даже если это противоречит личному анализу.</li>
  <li>Поиск гармонии ценой прибыли: избегаешь сделок, которые вызывают внутренний дискомфорт, даже если они выгодны.</li>
</ul>
<p><b>Что дает энергию для вдохновения жизни:</b><br>
Ощущение глубокого эмоционального контакта с другими, момент, когда твое присутствие помогает другим пройти через страх или неуверенность. Гармония с людьми и рынком — твой главный “edge” в мире хаоса.<br><br>
<i>Этот архетип — для тех, кто не боится быть “тихим голосом разума” в водовороте эмоций и всегда помнит: лучшая сделка — та, после которой спокойно спишь.</i>
</p>
""",
            "en": """
<h3>Spirit Mediator</h3>
<p><b>Credo:</b><br>
“My indicator is the vibe in the chat. If everyone is sad — time to buy. If everyone is celebrating — time to run. I don’t catch the bottom, I catch the calm.”</p>
<p><b>Essence:</b><br>
You are the emotional barometer of the market, unafraid of waves of fear or euphoria — you integrate them into your decisions. For you, the chart is not just price, but a reflection of collective sentiment. You buy when the chat is in panic and exit when everyone is celebrating. Your strength is a cool head in the swirl of emotions, your risk — avoiding “sharp” trades if they don’t resonate with your inner state.</p>
<p><b>Neurophysiology:</b></p>
<ul>
  <li>High GABA activity: calm under pressure, impulse control.</li>
  <li>Oxytocin and serotonin balance: empathy, reflection, social sensitivity.</li>
  <li>Low cortisol: resilience to stress even in peak market phases.</li>
</ul>
<p><b>Market impact:</b></p>
<ul>
  <li>You stabilize emotional waves, become an example for others in a crisis.</li>
  <li>You raise the “emotional literacy” of the community, reduce collective impulsiveness.</li>
  <li>You can stop panic among less resilient market participants.</li>
</ul>
<p><b>What it reacts to:</b></p>
<ul>
  <li>Fluctuations in emotional background, signals of mass fear or euphoria.</li>
  <li>Your own emotions as the main indicator for non-standard decisions.</li>
  <li>Behavior of key players if it is emotionally charged.</li>
</ul>
<p><b>What it ignores:</b></p>
<ul>
  <li>“Dead” markets without emotions, dry technical signals, news without social resonance.</li>
</ul>
<p><b>Strategies:</b></p>
<ul>
  <li>Psychological arbitrage: trading against mass panic or euphoria, using your own emotional indicators to enter during collective fear or greed.</li>
  <li>Countertrend in emotional overheat phases: entering against the crowd when emotions are extreme, with confirmation through price action, order flow, or community emotional background.</li>
  <li>Cognitive-behavioral techniques: self-observation, keeping a journal of emotional triggers, integrating mindfulness to reduce impulsivity, working with your own cognitive biases.</li>
  <li>Ignoring short-term noise: filtering signals through the community’s emotional background, refusing trades if you don’t feel a real mood shift.</li>
  <li>Sociometric monitoring: analyzing emotions in chats using NLP tools (e.g., automatic sentiment index in Telegram/Discord), identifying the “boiling point” of the crowd for entry or exit.</li>
  <li>Psychophysiological control: using biofeedback (HRV, EDA) to monitor your own stress at peak moments; trading only in a state of inner calm, automatic trading block at high stress.</li>
  <li>“Everyone was screaming ‘crash’, but my biotracker showed perfect calm — I bought and exited in profit while everyone else was still panicking.”</li>
</ul>
<p><b>Cognitive patterns:</b></p>
<ul>
  <li>Emotional integration into decisions: often act based on collective vibe, even if it contradicts your personal analysis.</li>
  <li>Seeking harmony at the price of profit: avoiding trades that cause inner discomfort, even if they are profitable.</li>
</ul>
<p><b>What gives energy for life inspiration:</b><br>
The feeling of deep emotional connection with others, the moment when your presence helps others get through fear or uncertainty. Harmony with people and the market is your main “edge” in a world of chaos.<br><br>
<i>This archetype is for those who are not afraid to be the “quiet voice of reason” in the swirl of emotions, and always remember: the best trade is the one after which you sleep peacefully.</i>
</p>
"""
        },
        "image": "mediator.jpg"
    }
    ,
    "Рептилія": {
        "description": {
            "uk": """
<h3>Рептилія (The Ritualist)</h3>
<p><b>Кредо:</b><br>
«Якщо алгоритм не дав сигнал — я йду на обід. Ринок має підлаштовуватись під мої правила, а не навпаки!»</p>
<p><b>Суть:</b><br>
Ти — ходячий алгоритм: усе по поличках, жодної імпровізації, чек-листи замість натхнення. Для тебе ідеальний день — коли ринок слухняно повторює знайомий патерн, а будь-яка зміна — це загроза, яку треба перечекати, а не використати. Ти святкуєш, коли твій стоп-лос спрацював з точністю до цента, і впевнений, що “збій” — це не ринок, а світ, який тимчасово втратив розум.</p>
<p><b>Нейрофізіологія:</b></p>
<ul>
  <li>Домінування серотоніну: задоволення від структури й передбачуваності.</li>
  <li>Знижена дофамінова чутливість: слабка реакція на новизну, відраза до хаосу.</li>
  <li>Активна префронтальна кора: самоконтроль, дотримання ритуалів, мінімум імпульсивності.</li>
</ul>
<p><b>Вплив на ринок:</b></p>
<ul>
  <li>Стабілізує ціни, зменшує волатильність, створює “якір” для ринку.</li>
  <li>Гальмує інновації, підсилює ефект натовпу у спокійні періоди.</li>
</ul>
<p><b>На що реагує:</b></p>
<ul>
  <li>Чіткі сигнали алгоритму, порушення структури, стабільні тренди.</li>
</ul>
<p><b>На що не реагує:</b></p>
<ul>
  <li>Короткострокова волатильність, емоційна атмосфера, новини/чутки, що не вписуються у систему.</li>
</ul>
<p><b>Стратегії:</b></p>
<ul>
  <li>Мультифакторні rule-based системи: побудова складних алгоритмів із десятків технічних і фундаментальних факторів (price action, volatility regimes, liquidity screens, макроіндикатори), використання ensemble-моделей для агрегування сигналів.</li>
  <li>Динамічний position sizing: адаптивне управління розміром позиції через volatility targeting, Expected Shortfall, CVaR, dynamic Kelly; автоматичне коригування лімітів залежно від ринкового режиму.</li>
  <li>Advanced stop management: volatility-adjusted стопи (ATR, percentile bands), trailing logic із автоматичним re-entry, smart stop із ML для оптимізації виходу.</li>
  <li>Режимна адаптація (Regime Switching): впровадження моделей Markov Switching, Hidden Markov Models, macro regime filters; автоматичне перемикання між трендовими, mean-reversion, breakout-стратегіями.</li>
  <li>Portfolio construction & risk overlay: побудова портфеля з rule-based стратегій із кореляційним overlay, risk parity, volatility clustering; stress-testing та Monte Carlo simulation для перевірки стійкості системи до “чорних лебедів”.</li>
  <li>Автоматизація та інфраструктура: повний цикл: від збору та очищення даних до автоматичного виконання і моніторингу угод через API; cloud computing для backtesting і live monitoring.</li>
  <li>Постійна оптимізація: регулярний out-of-sample backtest, walk-forward optimization, parameter stability tests, ensemble rule engines для зниження ризику overfitting; особлива увага — перевірці стабільності моделей у фазах regime switch і глибоким сценарним тестам на “чорних лебедів”.</li>
  <li>“Ринок перейшов у новий режим — моя система сама перемкнулася на mean-reversion, поки всі ловили тренд.”</li>
</ul>
<p><b>Когнітивні патерни:</b></p>
<ul>
  <li>Пастка “старої моделі”: схильність залишатися у перевірених стратегіях, навіть коли ринок змінився.</li>
  <li>Залежність від “золотого правила”: надмірна віра у власні алгоритми, навіть якщо вони втрачають ефективність.</li>
</ul>
<p><b>Що дає енергію для натхнення життя:</b><br>
Відчуття гармонії, коли все йде за планом, ідеальна відповідність реальності твоїм правилам. Серотонінова винагорода за структурованість і передбачуваність.<br><br>
<i>Цей архетип — для тих, хто не шукає гострих відчуттів, а цінує порядок, контроль і стабільність. Якщо ти впізнав себе — твоя сила у дисципліні, а твій ризик — у ригідності.</i>
</p>
""",
            "ru": """
<h3>Рептилия (The Ritualist)</h3>
<p><b>Кредо:</b><br>
«Если алгоритм не дал сигнал — я иду на обед. Рынок должен подстраиваться под мои правила, а не наоборот!»</p>
<p><b>Суть:</b><br>
Ты — ходячий алгоритм: все по полочкам, никакой импровизации, чек-листы вместо вдохновения. Для тебя идеальный день — когда рынок послушно повторяет знакомый паттерн, а любое изменение — это угроза, которую нужно переждать, а не использовать. Ты празднуешь, когда твой стоп-лосс сработал с точностью до цента, и уверен, что “сбой” — это не рынок, а мир, который временно потерял разум.</p>
<p><b>Нейрофизиология:</b></p>
<ul>
  <li>Доминирование серотонина: удовольствие от структуры и предсказуемости.</li>
  <li>Пониженная дофаминовая чувствительность: слабая реакция на новизну, отвращение к хаосу.</li>
  <li>Активная префронтальная кора: самоконтроль, соблюдение ритуалов, минимум импульсивности.</li>
</ul>
<p><b>Влияние на рынок:</b></p>
<ul>
  <li>Стабилизирует цены, уменьшает волатильность, создает “якорь” для рынка.</li>
  <li>Замедляет инновации, усиливает эффект толпы в спокойные периоды.</li>
</ul>
<p><b>На что реагирует:</b></p>
<ul>
  <li>Четкие сигналы алгоритма, нарушение структуры, стабильные тренды.</li>
</ul>
<p><b>На что не реагирует:</b></p>
<ul>
  <li>Краткосрочная волатильность, эмоциональная атмосфера, новости/слухи, не вписывающиеся в систему.</li>
</ul>
<p><b>Стратегии:</b></p>
<ul>
  <li>Мультифакторные rule-based системы: построение сложных алгоритмов из десятков технических и фундаментальных факторов (price action, volatility regimes, liquidity screens, макроиндикаторы), использование ensemble-моделей для агрегирования сигналов.</li>
  <li>Динамическое управление размером позиции: адаптивное управление размером позиции через volatility targeting, Expected Shortfall, CVaR, dynamic Kelly; автоматическая корректировка лимитов в зависимости от рыночного режима.</li>
  <li>Advanced stop management: volatility-adjusted стопы (ATR, percentile bands), trailing logic с автоматическим re-entry, smart stop с ML для оптимизации выхода.</li>
  <li>Режимная адаптация (Regime Switching): внедрение моделей Markov Switching, Hidden Markov Models, macro regime filters; автоматическое переключение между трендовыми, mean-reversion, breakout-стратегиями.</li>
  <li>Построение портфеля и risk overlay: построение портфеля из rule-based стратегий с корреляционным overlay, risk parity, volatility clustering; stress-testing и Monte Carlo simulation для проверки устойчивости системы к “черным лебедям”.</li>
  <li>Автоматизация и инфраструктура: полный цикл: от сбора и очистки данных до автоматического исполнения и мониторинга сделок через API; cloud computing для backtesting и live monitoring.</li>
  <li>Постоянная оптимизация: регулярный out-of-sample backtest, walk-forward optimization, parameter stability tests, ensemble rule engines для снижения риска overfitting; особое внимание — проверке стабильности моделей в фазах regime switch и глубоким сценарным тестам на “черных лебедей”.</li>
  <li>“Рынок перешел в новый режим — моя система сама переключилась на mean-reversion, пока все ловили тренд.”</li>
</ul>
<p><b>Когнитивные паттерны:</b></p>
<ul>
  <li>Проблема “старой модели”: склонность оставаться в проверенных стратегиях, даже когда рынок изменился.</li>
  <li>Зависимость от “золотого правила”: чрезмерная вера в собственные алгоритмы, даже если они теряют эффективность.</li>
</ul>
<p><b>Что дает энергию для вдохновения жизни:</b><br>
Ощущение гармонии, когда все идет по плану, идеальное соответствие реальности твоим правилам. Серотониновая награда за структурированность и предсказуемость.<br><br>
<i>Этот архетип — для тех, кто не ищет острых ощущений, а ценит порядок, контроль и стабильность. Если ты узнал себя — твоя сила в дисциплине, а твой риск — в ригидности.</i>
</p>
""",
            "en": """
<h3>Reptile (The Ritualist)</h3>
<p><b>Credo:</b><br>
"If the algorithm didn't give a signal — I'm off to lunch. The market must adapt to my rules, not the other way around!"</p>
<p><b>Essence:</b><br>
You are a walking algorithm: everything in order, no improvisation, checklists instead of inspiration. Your perfect day is when the market obediently repeats a familiar pattern, and any change is a threat to be endured, not used. You celebrate when your stop-loss triggers to the cent, confident that a "glitch" is not the market but a world temporarily losing its mind.</p>
<p><b>Neurophysiology:</b></p>
<ul>
  <li>Dominance of serotonin: pleasure from structure and predictability.</li>
  <li>Reduced dopamine sensitivity: weak reaction to novelty, aversion to chaos.</li>
  <li>Active prefrontal cortex: self-control, adherence to rituals, minimal impulsivity.</li>
</ul>
<p><b>Market impact:</b></p>
<ul>
  <li>Stabilizes prices, reduces volatility, creates an "anchor" for the market.</li>
  <li>Slows innovation, enhances herd effect in calm periods.</li>
</ul>
<p><b>What it reacts to:</b></p>
<ul>
  <li>Clear algorithm signals, structure violations, stable trends.</li>
</ul>
<p><b>What it ignores:</b></p>
<ul>
  <li>Short-term volatility, emotional atmosphere, news/rumors that don't fit the system.</li>
</ul>
<p><b>Strategies:</b></p>
<ul>
  <li>Multifactor rule-based systems: building complex algorithms from dozens of technical and fundamental factors (price action, volatility regimes, liquidity screens, macro indicators), using ensemble models to aggregate signals.</li>
  <li>Dynamic position sizing: adaptive position management through volatility targeting, Expected Shortfall, CVaR, dynamic Kelly; automatic limit adjustments depending on market regime.</li>
  <li>Advanced stop management: volatility-adjusted stops (ATR, percentile bands), trailing logic with automatic re-entry, smart stop with ML for exit optimization.</li>
  <li>Regime switching: implementation of Markov Switching models, Hidden Markov Models, macro regime filters; automatic switching between trend-following, mean-reversion, breakout strategies.</li>
  <li>Portfolio construction & risk overlay: building a portfolio of rule-based strategies with correlation overlay, risk parity, volatility clustering; stress-testing and Monte Carlo simulation to check system resilience to "black swans".</li>
  <li>Automation and infrastructure: full cycle from data collection and cleaning to automatic execution and monitoring of trades via API; cloud computing for backtesting and live monitoring.</li>
  <li>Continuous optimization: regular out-of-sample backtesting, walk-forward optimization, parameter stability tests, ensemble rule engines to reduce overfitting risk; special attention to model stability during regime switches and deep scenario testing for "black swans".</li>
  <li>"The market switched to a new regime — my system automatically switched to mean-reversion while everyone was chasing the trend."</li>
</ul>
<p><b>Cognitive patterns:</b></p>
<ul>
  <li>Old model trap: tendency to stick to proven strategies even when the market has changed.</li>
  <li>Golden rule dependence: excessive faith in own algorithms even if they lose effectiveness.</li>
</ul>
<p><b>What gives energy for life inspiration:</b><br>
Feeling of harmony when everything goes according to plan, perfect match of reality to your rules. Serotonin reward for structure and predictability.<br><br>
<i>This archetype is for those who do not seek thrills but value order, control, and stability. If you recognize yourself — your strength is discipline, and your risk is rigidity.</i>
</p>
"""
        },
        "image": "reptile.jpg"
    }
    ,
    "Маржинальний жнець": {
        "description": {
            "uk": """
<h3>Маржинальний жнець</h3>
<p><b>Кредо:</b><br>
«All-in — бо життя занадто коротке для стоп-лосів. Ще одна угода — і точно відіграюсь. А якщо ні — буде що згадати на зустрічі випускників».</p>
<p><b>Суть:</b><br>
Ти — той, хто не визнає пів заходів: якщо ризикувати, то по-крупному, якщо втрачати — то з розмахом. Для тебе ринок — це арена, а кожна угода має приносити максимум адреналіну. Стоп-лос — це для тих, хто не вірить у себе, а ризик-менеджмент — вигадка бухгалтерів. Ти святкуєш margin call так, як інші святкують прибуток, і впевнений, що ще одна угода точно “відіграє” попередню просадку.</p>
<p><b>Нейрофізіологія:</b></p>
<ul>
  <li>Високий кортизол і дофамін: живеш на межі, кайфуєш від стресу й азарту.</li>
  <li>Знижена активність префронтальної кори: імпульсивність, схильність до “all-in”.</li>
  <li>Гіперактивність: овертрейдинг, ігнорування малоймовірних, але катастрофічних сценаріїв.</li>
</ul>
<p><b>Вплив на ринок:</b></p>
<ul>
  <li>Підсилюєш волатильність, провокуєш хвилі ліквідацій, створюєш різкі рухи ціни.</li>
  <li>Запускаєш паніку або ейфорію, навіть якщо сам у мінусі.</li>
</ul>
<p><b>На що реагує:</b></p>
<ul>
  <li>Сплески волатильності, поведінкові патерни натовпу, “смак крові” на ринку.</li>
</ul>
<p><b>На що не реагує:</b></p>
<ul>
  <li>Довгострокові фактори, емоційна стійкість інших, соціальні сигнали довіри/опору.</li>
</ul>
<p><b>Стратегії:</b></p>
<ul>
  <li>Агресивний левередж: максимальне кредитне плече для відкриття великих позицій у моменти пікової волатильності; all-in на ключових рівнях.</li>
  <li>Short/Long Squeeze: атаки на кластери стопів через глибокий аналіз order book, навмисне створення хвиль ліквідацій.</li>
  <li>Scalping на маржі: короткострокові угоди з великим об’ємом, блискавичний вхід і вихід на хвилинних таймфреймах.</li>
  <li>Контр-тренд на екстремумах: агресивний вхід проти руху у моменти масової паніки/ейфорії.</li>
  <li>Маніпуляції ліквідністю: використання низької ліквідності для створення “флеш-рухів”.</li>
  <li>Exotic leverage: використання perpetual swaps та інших нестандартних деривативів.</li>
  <li>“На пампі альта взяв 50x, зловив margin call, але відігрався на наступному русі — і так по колу”.</li>
</ul>
<p><b>Когнітивні патерни:</b></p>
<ul>
  <li>Систематична гіперактивність: надмірна кількість угод за короткий час.</li>
  <li>Недооцінка tail risk: віра, що “цього разу все буде інакше”.</li>
</ul>
<p><b>Що дає енергію для натхнення життя:</b><br>
Відчуття перемоги у складній грі, драйв від гострих ситуацій, вплив на хід подій.<br><br>
<i>Цей архетип — для тих, хто не шукає стабільності, а живе заради великого удару і перемоги тут і зараз.</i>
</p>
""",
            "ru": """
<h3>Маржинальный жнец</h3>
<p><b>Кредо:</b><br>
«All-in — потому что жизнь слишком коротка для стоп-лоссов. Еще одна сделка — и точно отыграюсь. А если нет — будет что вспомнить на встрече выпускников».</p>
<p><b>Суть:</b><br>
Ты — тот, кто не признает полумер: если рисковать, то по-крупному, если терять — то с размахом. Для тебя рынок — это арена, а каждая сделка должна приносить максимум адреналина. Стоп-лосс — для тех, кто не верит в себя, а риск-менеджмент — выдумка бухгалтеров. Ты празднуешь margin call так, как другие празднуют прибыль, и уверен, что еще одна сделка точно «отыграет» предыдущую просадку.</p>
<p><b>Нейрофизиология:</b></p>
<ul>
  <li>Высокий кортизол и дофамин: живешь на грани, кайфуешь от стресса и азарта.</li>
  <li>Сниженная активность префронтальной коры: импульсивность, склонность к «all-in».</li>
  <li>Гиперактивность: овертрейдинг, игнорирование маловероятных, но катастрофических сценариев.</li>
</ul>
<p><b>Влияние на рынок:</b></p>
<ul>
  <li>Усиливаешь волатильность, провоцируешь волны ликвидаций, создаешь резкие движения цены.</li>
  <li>Запускаешь панику или эйфорию, даже если сам в минусе.</li>
</ul>
<p><b>На что реагирует:</b></p>
<ul>
  <li>Всплески волатильности, поведенческие паттерны толпы, «вкус крови» на рынке.</li>
</ul>
<p><b>На что не реагирует:</b></p>
<ul>
  <li>Долгосрочные факторы, эмоциональная устойчивость других, социальные сигналы доверия/сопротивления.</li>
</ul>
<p><b>Стратегии:</b></p>
<ul>
  <li>Агрессивный левередж: максимальное кредитное плечо для открытия больших позиций в моменты пиковой волатильности; all-in на ключевых уровнях.</li>
  <li>Short/Long Squeeze: атаки на кластеры стопов через глубокий анализ order book, намеренное создание волн ликвидаций.</li>
  <li>Scalping на марже: краткосрочные сделки с большим объемом, молниеносный вход и выход на минутных таймфреймах.</li>
  <li>Контртренд на экстремумах: агрессивный вход против движения в моменты массовой паники/эйфории.</li>
  <li>Манипуляции ликвидностью: использование низкой ликвидности для создания «флеш-движений».</li>
  <li>Exotic leverage: использование perpetual swaps и других нестандартных деривативов.</li>
  <li>«На пампе альта взял 50x, поймал margin call, но отыгрался на следующем движении — и так по кругу».</li>
</ul>
<p><b>Когнитивные паттерны:</b></p>
<ul>
  <li>Систематическая гиперактивность: чрезмерное количество сделок за короткий период.</li>
  <li>Недооценка tail risk: вера, что «в этот раз все будет иначе».</li>
</ul>
<p><b>Что дает энергию для вдохновения жизни:</b><br>
Ощущение победы в сложной игре, драйв от острых ситуаций, влияние на ход событий.<br><br>
<i>Этот архетип — для тех, кто не ищет стабильности, а живет ради большого удара и победы здесь и сейчас.</i>
</p>
""",
            "en": """
<h3>Margin Reaper</h3>
<p><b>Credo:</b><br>
"All-in — because life is too short for stop-losses. One more trade — and I'll definitely get back. And if not — there will be something to remember at the reunion."</p>
<p><b>Essence:</b><br>
You are the one who doesn't believe in half measures: if you risk, then big, if you lose — then with a bang. For you, the market is an arena, and every trade should bring maximum adrenaline. Stop-loss is for those who don't believe in themselves, and risk management is an accountant's invention. You celebrate margin calls like others celebrate profits, and you are sure that one more trade will definitely “get back” the previous drawdown.</p>
<p><b>Neurophysiology:</b></p>
<ul>
  <li>High cortisol and dopamine: living on the edge, enjoying stress and excitement.</li>
  <li>Reduced prefrontal cortex activity: impulsivity, tendency to “all-in”.</li>
  <li>Hyperactivity: overtrading, ignoring unlikely but catastrophic scenarios.</li>
</ul>
<p><b>Market impact:</b></p>
<ul>
  <li>Amplifies volatility, triggers liquidation waves, creates sharp price movements.</li>
  <li>Triggers panic or euphoria, even if in loss.</li>
</ul>
<p><b>What it reacts to:</b></p>
<ul>
  <li>Volatility spikes, crowd behavioral patterns, “taste of blood” in the market.</li>
</ul>
<p><b>What it ignores:</b></p>
<ul>
  <li>Long-term factors, emotional resilience of others, social signals of trust/resistance.</li>
</ul>
<p><b>Strategies:</b></p>
<ul>
  <li>Aggressive leverage: maximum credit leverage to open large positions during peak volatility; all-in at key levels.</li>
  <li>Short/Long Squeeze: attacks on stop clusters through deep order book analysis, deliberate creation of liquidation waves.</li>
  <li>Margin scalping: short-term trades with large volume, lightning-fast entry and exit on minute timeframes.</li>
  <li>Counter-trend at extremes: aggressive entry against the movement during mass panic/euphoria.</li>
  <li>Liquidity manipulation: using low liquidity to create “flash moves”.</li>
  <li>Exotic leverage: use of perpetual swaps and other non-standard derivatives.</li>
  <li>“On an alt pump took 50x, caught a margin call, but got back on the next move — and so on.”</li>
</ul>
<p><b>Cognitive patterns:</b></p>
<ul>
  <li>Systematic hyperactivity: excessive number of trades in a short time.</li>
  <li>Underestimation of tail risk: belief that “this time will be different”.</li>
</ul>
<p><b>What gives energy for life inspiration:</b><br>
The feeling of victory in a tough game, the thrill of sharp situations, influence on the course of events.<br><br>
<i>This archetype is for those who do not seek stability but live for the big hit and victory here and now.</i>
</p>
"""
        },
        "image": "marginal.jpg"
    }
}




IndentationError: unexpected indent (1272043534.py, line 615)

In [6]:
from IPython.display import display, HTML, clear_output
import ipywidgets as widgets
import requests
import uuid
import time
import random

# ОНОВЛЕНИЙ CSS для компактних кнопок, мінімальних відступів зверху, світлого тексту
display(HTML("""
<style>
/* Мобільна адаптація для результатів та форми зворотнього зв'язку */
@media (max-width: 600px) {
    /* Заголовок архетипу */
    .custom-question-container h3 {
        font-size: 1.1rem !important;
        margin-bottom: 12px !important;
    }
    
    /* Сітка балів */
    .score-grid {
        grid-template-columns: repeat(auto-fit, minmax(70px, 1fr)) !important;
        gap: 8px !important;
    }
    .score-item {
        padding: 10px !important;
        font-size: 0.9rem !important;
    }
    .score-value {
        font-size: 1rem !important;
    }
    
    /* Зображення архетипу */
    .widget-image {
        width: 100% !important;
        max-width: 280px !important;
    }
    
    /* Форма зворотнього зв'язку */
    .widget-textarea textarea {
        width: 95% !important;
        min-height: 80px !important;
    }
    .widget-button {
        width: 100% !important;
        margin-top: 15px !important;
    }
}

/* Оптимізація для десктопу */
.desc-widget {
    font-size: 1.1rem !important;
    line-height: 1.7 !important;
    padding: 15px !important;
}

/* Вирівнювання контенту */
.widgets-vbox {
    align-items: center !important;
    gap: 20px !important;
}
</style>
"""))

class RiskToleranceTest:
    def __init__(self, questions, lang='uk'):
        self.questions = questions
        self.current_question = 0
        self.user_answers = []
        self.scores = {"EI": 0, "KB": 0, "RM": 0, "AD": 0, "AS": 0}
        self.output = widgets.Output()
        self.user_id = str(uuid.uuid4())[:8]
        self.test_start_time = None
        self.response_times = []
        self.demographics = {}
        self.question_start_time = None
        self.lang = lang  # Додаємо атрибут мови

    def start(self):
        clear_output(wait=True)
        display(self.output)
        self._show_consent_and_demographics()

    def _show_consent_and_demographics(self):
        with self.output:
            clear_output(wait=True)
            consent_html = widgets.HTML(
                f"""
                <div style='border:1.5px solid #4f8cff; padding:20px; border-radius:8px; margin:20px 0; background:#232a36; text-align:center;'>
                    <b style='color:#4f8cff;'>🔒 Ваш ID:</b> {self.user_id}<br>
                    <span style='color:#e0e6ef;'>Дані використовуються анонімно для наукового дослідження.<br>
                    Ви можете припинити тест будь-коли.</span>
                </div>
                """
            )
            age = widgets.Dropdown(options=['до 25', '26-35', '36+'], description='Вік:')
            exp = widgets.Dropdown(options=['<1 року', '1-3 роки', '>3 років'], description='Стаж:')
            market = widgets.Dropdown(options=['Крипто', 'Фондовий', 'Форекс', 'Комбо'], description='Ринок:')
            
            start_btn = widgets.Button(
                description="Почати тест", 
                style=widgets.ButtonStyle(button_color='#4f8cff'),
                layout=widgets.Layout(width='300px', height='50px', margin='20px auto')
            )

            def on_start_click(b):
                self.demographics = {
                    'age': age.value,
                    'exp': exp.value,
                    'market': market.value
                }
                self.test_start_time = time.time()
                clear_output(wait=True)
                self._show_question()
            
            start_btn.on_click(on_start_click)
            
            display(widgets.VBox([
                consent_html,
                widgets.VBox([age, exp, market, start_btn], layout=widgets.Layout(align_items='center'))
            ], layout=widgets.Layout(align_items='center', justify_content='center')))

    def _show_question(self):
        with self.output:
            clear_output(wait=True)
            if self.current_question >= len(self.questions):
                self._show_results()
                return

            q = self.questions[self.current_question]
            progress = (self.current_question / len(self.questions)) * 100

            answers = q['answers'].copy()
            random.shuffle(answers)
            self.current_answers = answers

            header = widgets.HTML(
                f"""<div class="custom-question-container" style="margin-top:0px; padding-top:0px;">
                    <div class="progress-container">
                        <div class="progress-fill" style="width:{progress}%"></div>
                    </div>
                    <h2>Питання {self.current_question + 1} з {len(self.questions)}</h2>
                    <div class="custom-question-text">{q['question']}</div>
                </div>"""
            )

            buttons = []
            for ans in answers:
                # Вибір мови для тексту відповіді
                answer_text = ans["text"][self.lang] if isinstance(ans["text"], dict) else ans["text"]
                btn = widgets.Button(
                    description=answer_text,
                    layout=widgets.Layout(width='100%')
                )
                btn.add_class('answer-btn')
                def on_click(b, answer=ans):
                    self._handle_answer(answer)
                btn.on_click(on_click)
                buttons.append(btn)

            answers_box = widgets.VBox(buttons, layout=widgets.Layout(width='100%'))

            display(widgets.VBox(
                [header, answers_box],
                layout=widgets.Layout(
                    align_items='center',
                    width='100%',
                    max_width='900px',
                    margin='0px auto 0 auto',
                    padding='20px 0'
                )
            ))

            self.question_start_time = time.time()

    def _handle_answer(self, answer):
        with self.output:
            clear_output(wait=True)
            question_time = time.time() - self.question_start_time
            self.response_times.append(round(question_time, 1))
            for cat, val in answer['weights'].items():
                self.scores[cat] += val
            self.current_question += 1
            self._show_question()

    def _show_results(self):
        with self.output:
            clear_output(wait=True)
            archetype, _ = determine_archetype(self.scores, self.questions)
            info = archetype_info_multilang[archetype]
            scores_html = '<div class="score-grid">'
            for cat, score in self.scores.items():
                scores_html += f"""
            <div class="score-item">
                <div>{cat}</div>
                <div class="score-value">{score}</div>
            </div>"""
            scores_html += '</div>'
            title_widget = widgets.HTML(
                f"<h2 style='color:#4f8cff; text-align:center; margin-bottom:18px;'>🎯 Ваш психологічний архетип</h2>"
                f"<span class='archetype-title'>{archetype}</span>"
                f"{scores_html}"
            )
            with open(info["image"], "rb") as f:
                img_data = f.read()
            img_widget = widgets.Image(
                value=img_data,
                format='jpg',  # або 'png'
                width=400
            )
            desc_widget = widgets.HTML(
                f"<div style='background:#181a1b; padding:16px 24px; border-radius:12px; "
                f"width:90%; max-width:750px; margin:20px auto 0 auto; line-height:1.45; font-size:1.08em;'>"
                f"{info['description'][self.lang]}"
                f"</div>"
            )
            display(
                widgets.VBox(
                    [
                        title_widget,
                        widgets.HBox([img_widget], layout=widgets.Layout(justify_content='center')),
                        desc_widget
                    ]
                )
            )
            display(self._create_feedback_form(archetype))
            self._send_to_telegram(archetype)
        
    def _create_feedback_form(self, archetype):
        feedback_label = widgets.HTML("<h4 style='color:#4f8cff; margin:25px 0 15px 0;'>💬 Залиште відгук про тест</h4>")
        feedback_text = widgets.Textarea(
            placeholder='Ваші враження, побажання, чи співпав результат?',
            layout=widgets.Layout(width='420px', height='100px', margin='0 auto 18px auto')
        )
        send_btn = widgets.Button(
            description='Надіслати в Telegram',
            layout=widgets.Layout(
                width='auto',
                min_width='250px',
                max_width='100%',
                height='50px',
                margin='24px auto 0 auto',
                align_self='center'
            ),
            style=widgets.ButtonStyle(button_color='#4f8cff')
        )
        feedback_output = widgets.Output()
        def on_send_click(b):
            with feedback_output:
                clear_output()
                if feedback_text.value.strip():
                    self._send_feedback_to_telegram(feedback_text.value, archetype)
                    display(HTML("<span style='color:#4f8cff;'>Відгук надіслано!</span>"))
                    feedback_text.value = ''
                else:
                    display(HTML("<span style='color:#ff6b6b;'>Введіть текст відгуку!</span>"))
        send_btn.on_click(on_send_click)
        return widgets.VBox(
            [feedback_label, feedback_text, send_btn, feedback_output],
            layout=widgets.Layout(align_items='center', width='100%')
        )

    def _send_feedback_to_telegram(self, feedback_text, archetype):
        try:
            TELEGRAM_TOKEN = "8021521612:AAFHa0rQcPhgyBh7y3Q5ggsOkNCu_5RDvyo"
            CHAT_ID = "298306752"
            message = (
                f"📝 Новий відгук про тест:\n"
                f"Архетип: {archetype}\n"
                f"Бали: {self.scores}\n"
                f"Відгук: {feedback_text}"
            )
            requests.post(
                f"https://api.telegram.org/bot{TELEGRAM_TOKEN}/sendMessage",
                json={"chat_id": CHAT_ID, "text": message}
            )
        except Exception as e:
            print(f"Помилка відправки: {str(e)}")

    def _send_to_telegram(self, archetype):
        try:
            TELEGRAM_TOKEN = "8021521612:AAFHa0rQcPhgyBh7y3Q5ggsOkNCu_5RDvyo"
            CHAT_ID = "298306752"
            test_duration = round(time.time() - self.test_start_time, 1)
            message = (
                f"📊 Результат тесту\n"
                f"ID: {self.user_id}\n"
                f"Вік: {self.demographics['age']}\n"
                f"Стаж: {self.demographics['exp']}\n"
                f"Ринок: {self.demographics['market']}\n"
                f"Час проходження: {test_duration} сек\n"
                f"Час на питання: {self.response_times}\n"
                f"Архетип: {archetype}\n"
                f"Бали:\n" + "\n".join([f"- {cat}: {score}" for cat, score in self.scores.items()])
            )
            requests.post(
                f"https://api.telegram.org/bot{TELEGRAM_TOKEN}/sendMessage",
                json={"chat_id": CHAT_ID, "text": message}
            )
        except Exception as e:
            print(f"Помилка відправки: {str(e)}")

# Приклад запуску:
test = RiskToleranceTest(questions, lang='uk')
test.start()


IndentationError: unexpected indent (4254100728.py, line 230)